In [1]:
# start time - notebook execution
import time
start_nb = time.time()

Set up

In [2]:
!pip install datasets

In [3]:
import gensim.downloader
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from torchtext.data import get_tokenizer

# ignore all warnings
import warnings
warnings.filterwarnings('ignore')

DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(DEVICE)

mps


In [4]:
import hw4_tests as ag

## Data

In [5]:
wiki_data_train = load_dataset("wikitext", 'wikitext-2-v1', split="train").shuffle()
wiki_data_test = load_dataset("wikitext", 'wikitext-2-v1', split="test").shuffle()
WIKI_TRAIN = pd.DataFrame(wiki_data_train)
WIKI_TEST = pd.DataFrame(wiki_data_test)
WIKI_ALL = pd.concat([WIKI_TRAIN, WIKI_TEST])

In [6]:
TOKENIZER = get_tokenizer("basic_english")

def my_tokenizer(string):
  tokens = TOKENIZER(string)
  ### BEGIN SOLUTION
  ### END SOLUTION
  return tokens

## CBOW

Hyperparameters:
CBOW_EMBED_DIMENSIONS = 100
CBOW_WINDOW = 4
CBOW_MAX_LENGTH = 50
CBOW_BATCH_SIZE = 1024
CBOW_NUM_EPOCHS = 2
CBOW_LEARNING_RATE = 5e-4 

In [7]:
# Hyperparameters; feel free to change them
CBOW_EMBED_DIMENSIONS = 100
CBOW_WINDOW = 4
CBOW_MAX_LENGTH = 50
CBOW_BATCH_SIZE = 256 #1024
CBOW_NUM_EPOCHS = 4 #2
CBOW_LEARNING_RATE = 5e-4 

In [8]:
# RUN THIS CELL BUT DO NOT EDIT IT
UNK_token = 0   # Unknown '<unk>'
UNK_symbol = '<unk>'

class Vocab:
  def __init__(self, name=''):
    self.name = name
    self._word2index = {UNK_symbol: UNK_token}
    self._word2count = {UNK_symbol: 0}
    self._index2word = {UNK_token: UNK_symbol}
    self._n_words = 1

  def get_words(self):
    return list(self._word2count.keys())

  def num_words(self):
    return self._n_words

  def word2index(self, word):
    if word in self._word2index:
      return self._word2index[word]
    else:
      return self._word2index[UNK_symbol]

  def index2word(self, word):
    return self._index2word[word]

  def word2count(self, word):
    return self._word2count[word]

  def add_sentence(self, sentence):
    for word in sentence.split(' '):
      self.add_word(word)

  def add_word(self, word):
    if word not in self._word2index:
      self._word2index[word] = self._n_words
      self._word2count[word] = 1
      self._index2word[self._n_words] = word
      self._n_words += 1
    else:
      self._word2count[word] += 1

def prep_cbow_data(data_frame, tokenizer_fn, window=2, max_length=50):
  """
    1. Tokenize each document, restricting the # of tokens to max_length.
    2. Add all tokens to the vocabulary.
    3. Iterate over each token, selecting `window` words before and after as context.
    5. Convert words to their corresponding indices.
    6. Append the (context, target) pairs to the dataset.
    7. Return the dataset and the vocabulary.
  """
  data_out = []
  vocab = Vocab()
  ### BEGIN SOLUTION
  
  for doc in data_frame['text']:
    tokens = tokenizer_fn(doc)[:max_length]
    unique_tokens = list(set(tokenizer_fn(doc)))
    vocab.add_sentence(' '.join(unique_tokens))
      
    for i in range(window, len(tokens)-window):
      # x = context words list, y = target word
      target = tokens[i]
      context = tokens[i-window : i] + tokens[i+1 : i+window+1]
      
      y = vocab.word2index(target)
      x = [vocab.word2index(w) for w in context]
      data_out.append((x,y))
          
  ### END SOLUTION
  return data_out, vocab

In [9]:
CBOW_DATA, CBOW_VOCAB = prep_cbow_data(WIKI_TRAIN, tokenizer_fn=my_tokenizer, window=CBOW_WINDOW, max_length=CBOW_MAX_LENGTH)
print("len dataframe=", len(WIKI_TRAIN), "len data=", len(CBOW_DATA))

len dataframe= 36718 len data= 625869


In [10]:
def get_batch(data, index, batch_size=10):
  ### BEGIN SOLUTION
  batch_data = data[index : index+batch_size]
  x = np.array([d[0] for d in batch_data], dtype=np.int64)
  y = np.array([d[1] for d in batch_data], dtype=np.int64)

  x = torch.tensor(x, dtype=torch.long).to(DEVICE)
  y = torch.tensor(y, dtype=torch.long).to(DEVICE)
  ### END SOLUTION
  return x, y

In [11]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, embed_size):
    super(CBOW, self).__init__()
    ### BEGIN SOLUTION
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.linear = nn.Linear(embed_size, vocab_size)
    ### END SOLUTION

  def forward(self, x):
    probs = None
    ### BEGIN SOLUTION
    # Get document embedding (average over word embeddings)
    x = self.embed(x).mean(dim=1)
    out = self.linear(x)
    probs = F.log_softmax(out, dim=1)
    ### END SOLUTION
    return probs

In [12]:
import traceback
cbow_model = CBOW(CBOW_VOCAB.num_words(), CBOW_EMBED_DIMENSIONS)
cbow_model.to(DEVICE)
CBOW_CRITERION = nn.NLLLoss()
try:
  CBOW_OPTIMIZER = torch.optim.AdamW(cbow_model.parameters(), lr=CBOW_LEARNING_RATE)
except:
  print(traceback.format_exc())

## Training

In [13]:
def train_cbow(model, data, num_epochs, batch_size, criterion, optimizer):
  for epoch in range(num_epochs):
    losses = []
    for i in range(len(data)//batch_size):
      x, y = get_batch(data, i, batch_size)
      y_hat = model(x)
      loss = criterion(y_hat, y)
      optimizer.zero_grad()
      loss.backward()
      losses.append(loss.item())
      optimizer.step()
      if i % 100 == 0:
        print('iter', i, 'loss', np.array(losses).mean())
    print('epoch', epoch, 'loss', np.array(losses).mean())

## Testing

In [14]:
def prep_test_data(data_frame, vocab, tokenizer_fn, window=2, max_length=50):
  data_out = []
  for row in data_frame['text']:
    tokens = tokenizer_fn(row)
    token_ids = [vocab.word2index(w) for w in tokens]
    if len(token_ids) >= (window*2)+1:
      token_ids = token_ids[0:min(len(token_ids), max_length)]
      for i in range(window, len(token_ids)-window):
        x = token_ids[i-window:i] + token_ids[i+1:i+window+1]
        y = token_ids[i]
        data_out.append((x, y))
  return data_out

TEST_DATA = prep_test_data(WIKI_TEST, CBOW_VOCAB, tokenizer_fn=my_tokenizer, window=CBOW_WINDOW, max_length=CBOW_MAX_LENGTH)

## Tests

In [15]:
# try:
#   train_cbow(cbow_model, CBOW_DATA, num_epochs=CBOW_NUM_EPOCHS, batch_size=CBOW_BATCH_SIZE, criterion=CBOW_CRITERION, optimizer=CBOW_OPTIMIZER)
# except:
#     print(traceback.format_exc())

In [16]:
# ag.test_cbow_performance(cbow_model, TEST_DATA, 512, get_batch_fn=get_batch)

In [18]:
import sys
import time

# Grid search for hyperparameters
CBOW_BATCH_SIZE = [128, 256, 1024]
CBOW_NUM_EPOCHS = [2, 5,10]
CBOW_LEARNING_RATE = [1e-3, 5e-4, 1e-4]

# CBOW_BATCH_SIZE = [1024]
# CBOW_NUM_EPOCHS = [2]
# CBOW_LEARNING_RATE = [5e-4]

# log results for each hyperparameter combination to a file for later reference
with open("tuning.txt", "w") as f:
    f.write("\n")

for batch_size in CBOW_BATCH_SIZE:
    for num_epochs in CBOW_NUM_EPOCHS:
        for learning_rate in CBOW_LEARNING_RATE:
                        
            print("--- TEST: batch_size=", batch_size, " num_epochs=", num_epochs, " lr=", learning_rate,"\n") 

            with open("tuning.txt", "a") as f:
                f.write(f"batch_size={batch_size}, num_epochs={num_epochs}, lr={learning_rate}\n")
            
            start = time.time()
            try:
                CBOW_OPTIMIZER = torch.optim.AdamW(cbow_model.parameters(), lr=learning_rate)
                
                train_cbow(cbow_model, CBOW_DATA, num_epochs=num_epochs, batch_size=batch_size, criterion=CBOW_CRITERION, optimizer=CBOW_OPTIMIZER)
                
                print("--- Result:")
                ag.test_cbow_performance(cbow_model, TEST_DATA, batch_size, get_batch_fn=get_batch)
                
            except:
                print(traceback.format_exc())
            
            
            print("--- Time taken:", time.time()-start)
            print("\n")
            
            with open("tuning.txt", "a") as f:
                f.write(f"Time: {time.time()-start}\n")

#closing file
f.close()
                

--- TEST: batch_size= 128  num_epochs= 2  lr= 0.001 

iter 0 loss 5.360970497131348
iter 100 loss 2.677766457642659
iter 200 loss 1.9164674759504214
iter 300 loss 1.4928958814009479
iter 400 loss 1.2169374825056651
iter 500 loss 1.023421169308845
iter 600 loss 0.8946677464960815
iter 700 loss 0.7946135978428341
iter 800 loss 0.7175528876269504
iter 900 loss 0.6540346222202997
iter 1000 loss 0.6032701997415765
iter 1100 loss 0.56194613828619
iter 1200 loss 0.5288252042952326
iter 1300 loss 0.5063149970566008
iter 1400 loss 0.49293781857908325
iter 1500 loss 0.5175314932158198
iter 1600 loss 0.6300197657451341
iter 1700 loss 0.735548725567412
iter 1800 loss 0.8295231293193966
iter 1900 loss 0.8920154790504239
iter 2000 loss 0.963949881404683
iter 2100 loss 1.0364348975314748
iter 2200 loss 1.102497021060459
iter 2300 loss 1.1639409597407782
iter 2400 loss 1.22334846886172
iter 2500 loss 1.2727208265819487
iter 2600 loss 1.3080035540741115
iter 2700 loss 1.3403830458608348
iter 2800 loss 

--- TEST: batch_size= 128  num_epochs= 2  lr= 0.001 

iter 0 loss 5.360970497131348
iter 100 loss 2.677766457642659
iter 200 loss 1.9164674759504214
iter 300 loss 1.4928958814009479
iter 400 loss 1.2169374825056651
iter 500 loss 1.023421169308845
iter 600 loss 0.8946677464960815
iter 700 loss 0.7946135978428341
iter 800 loss 0.7175528876269504
iter 900 loss 0.6540346222202997
iter 1000 loss 0.6032701997415765
iter 1100 loss 0.56194613828619
iter 1200 loss 0.5288252042952326
iter 1300 loss 0.5063149970566008
iter 1400 loss 0.49293781857908325
iter 1500 loss 0.5175314932158198
iter 1600 loss 0.6300197657451341
iter 1700 loss 0.735548725567412
iter 1800 loss 0.8295231293193966
iter 1900 loss 0.8920154790504239
iter 2000 loss 0.963949881404683
iter 2100 loss 1.0364348975314748
iter 2200 loss 1.102497021060459
iter 2300 loss 1.1639409597407782
iter 2400 loss 1.22334846886172
iter 2500 loss 1.2727208265819487
iter 2600 loss 1.3080035540741115
iter 2700 loss 1.3403830458608348
iter 2800 loss 1.3726086877911527
iter 2900 loss 1.4060285223794986
iter 3000 loss 1.4414913663935438
iter 3100 loss 1.467182132905912
iter 3200 loss 1.494224429696156
iter 3300 loss 1.5218082184355137
iter 3400 loss 1.550926740650691
iter 3500 loss 1.5710595451599698
iter 3600 loss 1.5947062574002153
iter 3700 loss 1.6102102671268501
iter 3800 loss 1.6316096395728494
iter 3900 loss 1.6475792399979317
iter 4000 loss 1.6591229734104802
iter 4100 loss 1.672532663644959
iter 4200 loss 1.6853770251782472
iter 4300 loss 1.699497145991898
iter 4400 loss 1.715552368458429
iter 4500 loss 1.724865368015566
iter 4600 loss 1.7336575739900202
iter 4700 loss 1.7444207778914105
iter 4800 loss 1.7585213443132843
epoch 0 loss 1.7705523102186311
iter 0 loss 5.84788179397583
iter 100 loss 1.3906523210577446
iter 200 loss 0.9597621563062146
iter 300 loss 0.7614983760835324
iter 400 loss 0.6678683307402746
iter 500 loss 0.5954461530178131
iter 600 loss 0.5530498342188741
iter 700 loss 0.5203772271623285
iter 800 loss 0.4886444684085477
iter 900 loss 0.46099801364868515
iter 1000 loss 0.43698208946567196
iter 1100 loss 0.41960642510495544
iter 1200 loss 0.40455889353198277
iter 1300 loss 0.3921983026363225
iter 1400 loss 0.3791967288223358
iter 1500 loss 0.37395282693301574
iter 1600 loss 0.36981206009865997
iter 1700 loss 0.36291548887433056
iter 1800 loss 0.3561232308839044
iter 1900 loss 0.3498905660415059
iter 2000 loss 0.34300383762351755
iter 2100 loss 0.3373129337472838
iter 2200 loss 0.3346032441231079
iter 2300 loss 0.33273618098110386
iter 2400 loss 0.33450619850318564
iter 2500 loss 0.32948123706859955
iter 2600 loss 0.32476986915743933
iter 2700 loss 0.3218276985610295
iter 2800 loss 0.31688241992293403
iter 2900 loss 0.3135229297539605
iter 3000 loss 0.3097314327006855
iter 3100 loss 0.30531497891440923
iter 3200 loss 0.30246291036337325
iter 3300 loss 0.29949566797076554
iter 3400 loss 0.2960856016949849
iter 3500 loss 0.2916970545331841
iter 3600 loss 0.2873804427991639
iter 3700 loss 0.28338246975950665
iter 3800 loss 0.2799641482200757
iter 3900 loss 0.2763442471986305
iter 4000 loss 0.27307167414634476
iter 4100 loss 0.27072640578904056
iter 4200 loss 0.2681135033970928
iter 4300 loss 0.26596181067795954
iter 4400 loss 0.26297298057008617
iter 4500 loss 0.2601869331846182
iter 4600 loss 0.2573233283481321
iter 4700 loss 0.2543174015701616
iter 4800 loss 0.25148207445525544
epoch 1 loss 0.2530496846705425
--- Result:
Test failed! Accuracy = 0.1864219605922699<0.3
Test G: 0/20
--- Time taken: 67.75561189651489


--- TEST: batch_size= 128  num_epochs= 2  lr= 0.0005 

iter 0 loss 2.6675734519958496
iter 100 loss 0.9569734945745751
iter 200 loss 0.644023724085656
iter 300 loss 0.4916693090996473
iter 400 loss 0.40907096665844955
iter 500 loss 0.3524556712148908
iter 600 loss 0.321249156210962
iter 700 loss 0.29761978650144094
iter 800 loss 0.2767449403821157
iter 900 loss 0.26076189466606364
iter 1000 loss 0.2497474992578918
iter 1100 loss 0.2403762932328827
iter 1200 loss 0.23367589654449022
iter 1300 loss 0.22753182722457457
iter 1400 loss 0.22440740927724137
iter 1500 loss 0.22437596032216023
iter 1600 loss 0.22658876912769566
iter 1700 loss 0.22777398756712341
iter 1800 loss 0.22647600433582865
iter 1900 loss 0.22474205463356497
iter 2000 loss 0.2236458362712674
iter 2100 loss 0.2233995897735317
iter 2200 loss 0.22603883811258718
iter 2300 loss 0.22931009384664647
iter 2400 loss 0.23314215231070862
iter 2500 loss 0.2328720312447893
iter 2600 loss 0.2325894182310889
iter 2700 loss 0.23522268822682163
iter 2800 loss 0.23570924587162084
iter 2900 loss 0.23668060511586914
iter 3000 loss 0.2370530126522637
iter 3100 loss 0.2378246808332691
iter 3200 loss 0.2403369218306555
iter 3300 loss 0.24321022846416357
iter 3400 loss 0.2444959824942589
iter 3500 loss 0.24722296670313326
iter 3600 loss 0.2485592221674142
iter 3700 loss 0.24992679213710942
iter 3800 loss 0.25080246892109637
iter 3900 loss 0.25223797182720825
iter 4000 loss 0.2523431806482693
iter 4100 loss 0.253474035911431
iter 4200 loss 0.25576741811818265
iter 4300 loss 0.2580284842105279
iter 4400 loss 0.26163807241476766
iter 4500 loss 0.2637729475370277
iter 4600 loss 0.2650223012833045
iter 4700 loss 0.26759670982270056
iter 4800 loss 0.2691983918744161
epoch 0 loss 0.2751703286194124
iter 0 loss 2.4021377563476562
iter 100 loss 0.4341270086346286
iter 200 loss 0.2619370356832274
iter 300 loss 0.19141099436140535
iter 400 loss 0.1550303141022115
iter 500 loss 0.13108722834351533
iter 600 loss 0.11704469185799807
iter 700 loss 0.10747316893663114
iter 800 loss 0.09835044026244595
iter 900 loss 0.09112926388157327
iter 1000 loss 0.08550378802854877
iter 1100 loss 0.08102605221869186
iter 1200 loss 0.07828078827790476
iter 1300 loss 0.07500971893391226
iter 1400 loss 0.07225751525441253
iter 1500 loss 0.07209038607488268
iter 1600 loss 0.07142419577096307
iter 1700 loss 0.07049954462159143
iter 1800 loss 0.06939163609102228
iter 1900 loss 0.06835801967530894
iter 2000 loss 0.06739714712943362
iter 2100 loss 0.0667219943385119
iter 2200 loss 0.06662265642857969
iter 2300 loss 0.06748108193035464
iter 2400 loss 0.06982726776822688
iter 2500 loss 0.06885171456650489
iter 2600 loss 0.06788308754912802
iter 2700 loss 0.06823833996098336
iter 2800 loss 0.0676493601596215
iter 2900 loss 0.06748338489543154
iter 3000 loss 0.06651893732724826
iter 3100 loss 0.06552416280510252
iter 3200 loss 0.06499990693391561
iter 3300 loss 0.06443675655574825
iter 3400 loss 0.06366102098301382
iter 3500 loss 0.06272511445922244
iter 3600 loss 0.06187788984213015
iter 3700 loss 0.06111971482280234
iter 3800 loss 0.06029232692437026
iter 3900 loss 0.05957217088842493
iter 4000 loss 0.05878979742891995
iter 4100 loss 0.058284525648013785
iter 4200 loss 0.05771857366130781
iter 4300 loss 0.05745469804543987
iter 4400 loss 0.05690113417897352
iter 4500 loss 0.056440436583566046
iter 4600 loss 0.05587805948918331
iter 4700 loss 0.05524729318770988
iter 4800 loss 0.05463139418690273
epoch 1 loss 0.05630958694785584
--- Result:
Test failed! Accuracy = 0.16255855560302734<0.3
Test G: 0/20
--- Time taken: 68.53828573226929


--- TEST: batch_size= 128  num_epochs= 2  lr= 0.0001 

iter 0 loss 1.2960236072540283
iter 100 loss 0.372486041692814
iter 200 loss 0.24498137965131161
iter 300 loss 0.18816249442605482
iter 400 loss 0.15164009750149493
iter 500 loss 0.12846540945940865
iter 600 loss 0.11507983920404598
iter 700 loss 0.10892381579661675
iter 800 loss 0.09999371002824714
iter 900 loss 0.09382590466281285
iter 1000 loss 0.08872577329198797
iter 1100 loss 0.0839029318948262
iter 1200 loss 0.08234060588886001
iter 1300 loss 0.07956571611278704
iter 1400 loss 0.07673217922842647
iter 1500 loss 0.07518227157653927
iter 1600 loss 0.0745444478284449
iter 1700 loss 0.0731072362177271
iter 1800 loss 0.07158493748279098
iter 1900 loss 0.07000822533335516
iter 2000 loss 0.06871758072801318
iter 2100 loss 0.06776044438320475
iter 2200 loss 0.06728985783953333
iter 2300 loss 0.06825792185900678
iter 2400 loss 0.07030699302949343
iter 2500 loss 0.06920070057270862
iter 2600 loss 0.06797247103450162
iter 2700 loss 0.06865989751713844
iter 2800 loss 0.06833498021644253
iter 2900 loss 0.06845148385947673
iter 3000 loss 0.06802952305718844
iter 3100 loss 0.06695854483470826
iter 3200 loss 0.06652508849864061
iter 3300 loss 0.06622167980883545
iter 3400 loss 0.065522176315704
iter 3500 loss 0.06480234476874144
iter 3600 loss 0.06390356936928333
iter 3700 loss 0.06328111871879702
iter 3800 loss 0.062448847214173125
iter 3900 loss 0.06187548500363245
iter 4000 loss 0.06114009914463966
iter 4100 loss 0.060547874439703026
iter 4200 loss 0.06011764606129723
iter 4300 loss 0.05980205885638084
iter 4400 loss 0.05960943226945235
iter 4500 loss 0.059637644865948
iter 4600 loss 0.05909615592944819
iter 4700 loss 0.058677275629642416
iter 4800 loss 0.058162708747553915
epoch 0 loss 0.060379572028679385
iter 0 loss 1.2545040845870972
iter 100 loss 0.27299905675325065
iter 200 loss 0.161856254182793
iter 300 loss 0.11715802171598637
iter 400 loss 0.092847191391889
iter 500 loss 0.07749083323660724
iter 600 loss 0.06795604831032369
iter 700 loss 0.061283753052906544
iter 800 loss 0.055621565886995246
iter 900 loss 0.0510349310562428
iter 1000 loss 0.04751643664077981
iter 1100 loss 0.044502887363079516
iter 1200 loss 0.043123696911320106
iter 1300 loss 0.041207251801147635
iter 1400 loss 0.039544836138717454
iter 1500 loss 0.03866157243319903
iter 1600 loss 0.03816778077944639
iter 1700 loss 0.03734276328741768
iter 1800 loss 0.03649089679221257
iter 1900 loss 0.03567351566834365
iter 2000 loss 0.03495865678233185
iter 2100 loss 0.03440396487269584
iter 2200 loss 0.03414339933464848
iter 2300 loss 0.03499993497990671
iter 2400 loss 0.037268671569806594
iter 2500 loss 0.03662730206842305
iter 2600 loss 0.0359285316451158
iter 2700 loss 0.03656384569993784
iter 2800 loss 0.036423066203701567
iter 2900 loss 0.03646948606177769
iter 3000 loss 0.036017816447054235
iter 3100 loss 0.035379811214695765
iter 3200 loss 0.0349539931539431
iter 3300 loss 0.03467955057850907
iter 3400 loss 0.03425440989501535
iter 3500 loss 0.0337366129880065
iter 3600 loss 0.03318813395130654
iter 3700 loss 0.03276763990434867
iter 3800 loss 0.03231061474728611
iter 3900 loss 0.03192532400943681
iter 4000 loss 0.03150950953358786
iter 4100 loss 0.031200251100102057
iter 4200 loss 0.03089805933921976
iter 4300 loss 0.030663425842517893
iter 4400 loss 0.030447005630098157
iter 4500 loss 0.0303352104325714
iter 4600 loss 0.030066946955586784
iter 4700 loss 0.029757282308761845
iter 4800 loss 0.029424076524718904
epoch 1 loss 0.031121309220297103
--- Result:
Test failed! Accuracy = 0.14987489581108093<0.3
Test G: 0/20
--- Time taken: 68.70097303390503


--- TEST: batch_size= 128  num_epochs= 5  lr= 0.001 

iter 0 loss 0.9960516095161438
iter 100 loss 0.14493015010168056
iter 200 loss 0.0796684243378058
iter 300 loss 0.05568839477969216
iter 400 loss 0.04313263655733085
iter 500 loss 0.03557116313495426
iter 600 loss 0.031047378997930548
iter 700 loss 0.027782120572179873
iter 800 loss 0.025297529900390723
iter 900 loss 0.023677123398607912
iter 1000 loss 0.022467502868491467
iter 1100 loss 0.02131505541357091
iter 1200 loss 0.020577696794302797
iter 1300 loss 0.020123056319150658
iter 1400 loss 0.01992923865007698
iter 1500 loss 0.01992714114913433
iter 1600 loss 0.0207566042033098
iter 1700 loss 0.021537981085180315
iter 1800 loss 0.022212669489024076
iter 1900 loss 0.02284470655556586
iter 2000 loss 0.024226067635090626
iter 2100 loss 0.025932823156544327
iter 2200 loss 0.028038518939878975
iter 2300 loss 0.03134927579128094
iter 2400 loss 0.03574627614192205
iter 2500 loss 0.037845774140653585
iter 2600 loss 0.039937191393877344
iter 2700 loss 0.04351430286509211
iter 2800 loss 0.046900777539812886
iter 2900 loss 0.05012725995988699
iter 3000 loss 0.05319371266529195
iter 3100 loss 0.05633704219593389
iter 3200 loss 0.05957982863645194
iter 3300 loss 0.06233317799908802
iter 3400 loss 0.06489373492198851
iter 3500 loss 0.06825327553950736
iter 3600 loss 0.07133204144359999
iter 3700 loss 0.0741948044755826
iter 3800 loss 0.0771524696317456
iter 3900 loss 0.08001466868336651
iter 4000 loss 0.08197087236301602
iter 4100 loss 0.08440029517426348
iter 4200 loss 0.08695939296779596
iter 4300 loss 0.0902768833935932
iter 4400 loss 0.09455920480629965
iter 4500 loss 0.09740856065935444
iter 4600 loss 0.10056735730853067
iter 4700 loss 0.10415332871560547
iter 4800 loss 0.1077527739925741
epoch 0 loss 0.11354736471915376
iter 0 loss 2.098349094390869
iter 100 loss 0.255672662233067
iter 200 loss 0.1616491576406493
iter 300 loss 0.12548453129357673
iter 400 loss 0.10602636646915999
iter 500 loss 0.09302482582227199
iter 600 loss 0.08379212795398795
iter 700 loss 0.077023757897987
iter 800 loss 0.07072800705755844
iter 900 loss 0.06616035260810009
iter 1000 loss 0.06218396774311731
iter 1100 loss 0.05866496088228558
iter 1200 loss 0.05630152765118659
iter 1300 loss 0.05341975307013327
iter 1400 loss 0.05079504815219429
iter 1500 loss 0.04914188326313486
iter 1600 loss 0.04740343274296186
iter 1700 loss 0.045769102907553975
iter 1800 loss 0.04410029432712033
iter 1900 loss 0.04259351827890104
iter 2000 loss 0.04114924612567238
iter 2100 loss 0.039893905011611656
iter 2200 loss 0.038901584954938256
iter 2300 loss 0.039251643425660745
iter 2400 loss 0.04072764250288862
iter 2500 loss 0.03954805327119117
iter 2600 loss 0.03851325488754937
iter 2700 loss 0.0383417877359664
iter 2800 loss 0.03768694676643038
iter 2900 loss 0.03706297334858243
iter 3000 loss 0.036180921632241646
iter 3100 loss 0.03531888509826943
iter 3200 loss 0.03469513900364467
iter 3300 loss 0.034050189945124536
iter 3400 loss 0.03334941566047954
iter 3500 loss 0.032617190245446376
iter 3600 loss 0.03193933076657907
iter 3700 loss 0.031293787500400565
iter 3800 loss 0.030671780875432424
iter 3900 loss 0.030101976764093062
iter 4000 loss 0.029540394392107552
iter 4100 loss 0.029054686085109128
iter 4200 loss 0.02857464789733259
iter 4300 loss 0.028236629700249632
iter 4400 loss 0.02779497032188568
iter 4500 loss 0.027391116037365597
iter 4600 loss 0.026988631762200642
iter 4700 loss 0.026564260899950863
iter 4800 loss 0.02615596700833722
epoch 1 loss 0.026586144109248755
iter 0 loss 0.5780420303344727
iter 100 loss 0.035913768653195385
iter 200 loss 0.022484877125929986
iter 300 loss 0.016492224154542153
iter 400 loss 0.013324427432345779
iter 500 loss 0.011375382982246473
iter 600 loss 0.01024517621271647
iter 700 loss 0.009679008970773126
iter 800 loss 0.008830962545376797
iter 900 loss 0.008365356581457182
iter 1000 loss 0.007875258833138304
iter 1100 loss 0.0074415751737115276
iter 1200 loss 0.007362293916600109
iter 1300 loss 0.00704255896612578
iter 1400 loss 0.006766686355595415
iter 1500 loss 0.006637295534174087
iter 1600 loss 0.006464624314392678
iter 1700 loss 0.006292287618094844
iter 1800 loss 0.006119197497981032
iter 1900 loss 0.005949625966488445
iter 2000 loss 0.005803412664495502
iter 2100 loss 0.005685076580628015
iter 2200 loss 0.005601920579932038
iter 2300 loss 0.006674146485202879
iter 2400 loss 0.00889043216647868
iter 2500 loss 0.008619904690725608
iter 2600 loss 0.008373922265307697
iter 2700 loss 0.008758304557466162
iter 2800 loss 0.008779427967009675
iter 2900 loss 0.00865388201286882
iter 3000 loss 0.008433708574738316
iter 3100 loss 0.008219972584635063
iter 3200 loss 0.008071832372038866
iter 3300 loss 0.007920188176410678
iter 3400 loss 0.007747110298104142
iter 3500 loss 0.007567651624110589
iter 3600 loss 0.007396727808128129
iter 3700 loss 0.0072451322913530715
iter 3800 loss 0.007094370880120347
iter 3900 loss 0.00695881555363024
iter 4000 loss 0.006824334402506023
iter 4100 loss 0.006710644986398039
iter 4200 loss 0.00660226174839279
iter 4300 loss 0.006553208535291138
iter 4400 loss 0.00645610657557096
iter 4500 loss 0.00637308394121926
iter 4600 loss 0.006287900537001023
iter 4700 loss 0.006184081917505222
iter 4800 loss 0.006085517133805894
epoch 2 loss 0.006279573289688549
iter 0 loss 0.23933398723602295
iter 100 loss 0.009732096062945478
iter 200 loss 0.006380743257335355
iter 300 loss 0.004618778448486917
iter 400 loss 0.0036703549011057692
iter 500 loss 0.003093961649527866
iter 600 loss 0.002762647781783743
iter 700 loss 0.0028392130013464947
iter 800 loss 0.0025660848181366866
iter 900 loss 0.002480081306588521
iter 1000 loss 0.0023220513550030603
iter 1100 loss 0.0021795788200950246
iter 1200 loss 0.0022573287674492624
iter 1300 loss 0.002153733205107498
iter 1400 loss 0.0020560208513357575
iter 1500 loss 0.0020115606502386593
iter 1600 loss 0.0019449901990380154
iter 1700 loss 0.001881840208914626
iter 1800 loss 0.0018207372654178208
iter 1900 loss 0.0017650905361635192
iter 2000 loss 0.0017167316476078141
iter 2100 loss 0.0016813330018298008
iter 2200 loss 0.0016610825541750805
iter 2300 loss 0.00271946545683215
iter 2400 loss 0.00496301117259272
iter 2500 loss 0.004786784121463569
iter 2600 loss 0.0046252593384577285
iter 2700 loss 0.005013504220649975
iter 2800 loss 0.005082730536269901
iter 2900 loss 0.004971112345182955
iter 3000 loss 0.0048229276684398515
iter 3100 loss 0.004682028608212548
iter 3200 loss 0.004568600287933792
iter 3300 loss 0.00445956465543188
iter 3400 loss 0.004344635781969232
iter 3500 loss 0.004230991996350501
iter 3600 loss 0.004123046874064055
iter 3700 loss 0.0040246163983988165
iter 3800 loss 0.0039287090942862245
iter 3900 loss 0.0038405034120105206
iter 4000 loss 0.0037547871042222787
iter 4100 loss 0.003679263616148001
iter 4200 loss 0.0036056927417892556
iter 4300 loss 0.0035658059584472083
iter 4400 loss 0.0035021737758066528
iter 4500 loss 0.003446121019145666
iter 4600 loss 0.003390265899258005
iter 4700 loss 0.003325442630196158
iter 4800 loss 0.0032645600844844647
epoch 3 loss 0.003362683309777416
iter 0 loss 0.11124134063720703
iter 100 loss 0.003126782246422723
iter 200 loss 0.002158962039419436
iter 300 loss 0.0015492945814453392
iter 400 loss 0.0012197340837159564
iter 500 loss 0.001020477881305154
iter 600 loss 0.0009035878811996607
iter 700 loss 0.0011100258707301691
iter 800 loss 0.000994766063570509
iter 900 loss 0.0009941586171938793
iter 1000 loss 0.0009220607496865376
iter 1100 loss 0.0008584662152384577
iter 1200 loss 0.0009813274964888435
iter 1300 loss 0.0009275252279026171
iter 1400 loss 0.0008786448927508214
iter 1500 loss 0.0008499236629103859
iter 1600 loss 0.0008153786604962897
iter 1700 loss 0.0007837707995330525
iter 1800 loss 0.0007535671802033176
iter 1900 loss 0.0007267894298480301
iter 2000 loss 0.0007043177553852296
iter 2100 loss 0.0006861698999863816
iter 2200 loss 0.0006774561517207963
iter 2300 loss 0.0017194801425860164
iter 2400 loss 0.003953970812437182
iter 2500 loss 0.003802907689096348
iter 2600 loss 0.003664633561598008
iter 2700 loss 0.004049524759541721
iter 2800 loss 0.004123141218096777
iter 2900 loss 0.004009074969620581
iter 3000 loss 0.0038810862024353573
iter 3100 loss 0.0037603915523396906
iter 3200 loss 0.003654590524497381
iter 3300 loss 0.003554875019570398
iter 3400 loss 0.0034557558869890755
iter 3500 loss 0.003360416036574633
iter 3600 loss 0.0032699047340417264
iter 3700 loss 0.0031858531845272643
iter 3800 loss 0.0031051606812608668
iter 3900 loss 0.003029543031151265
iter 4000 loss 0.002957051128321037
iter 4100 loss 0.0028904755551822647
iter 4200 loss 0.0028262520277038098
iter 4300 loss 0.002775272117650423
iter 4400 loss 0.0027194418578818506
iter 4500 loss 0.002671470222461751
iter 4600 loss 0.0026219394121734796
iter 4700 loss 0.002568584815955068
iter 4800 loss 0.0025183125363097652
epoch 4 loss 0.0025767974466988836
--- Result:
Test failed! Accuracy = 0.15260328352451324<0.3
Test G: 0/20
--- Time taken: 163.27268195152283


--- TEST: batch_size= 128  num_epochs= 5  lr= 0.0005 

iter 0 loss 0.05816188454627991
iter 100 loss 0.0023714788860323293
iter 200 loss 0.001995501409062365
iter 300 loss 0.0014689469983046287
iter 400 loss 0.0011432429991080136
iter 500 loss 0.0009445653263110707
iter 600 loss 0.0008255683263052316
iter 700 loss 0.0013140800263645152
iter 800 loss 0.0011696437116946034
iter 900 loss 0.001157134449783636
iter 1000 loss 0.0010738445500962782
iter 1100 loss 0.0009930235391485778
iter 1200 loss 0.0011916732110209018
iter 1300 loss 0.0011193291329276115
iter 1400 loss 0.001054591962702326
iter 1500 loss 0.0010049392731210263
iter 1600 loss 0.0009572858999705865
iter 1700 loss 0.0009142488882018908
iter 1800 loss 0.0008742522136554206
iter 1900 loss 0.0008368143109877659
iter 2000 loss 0.0008044695940912354
iter 2100 loss 0.0007823223157024008
iter 2200 loss 0.0007644607129578223
iter 2300 loss 0.001809529684197166
iter 2400 loss 0.004048146359339537
iter 2500 loss 0.003894816591819559
iter 2600 loss 0.003751577611690189
iter 2700 loss 0.004090388629005411
iter 2800 loss 0.0041780148599772195
iter 2900 loss 0.004076904754146318
iter 3000 loss 0.0039489347988950875
iter 3100 loss 0.0038270237084751216
iter 3200 loss 0.003738375671337158
iter 3300 loss 0.0036425382488344984
iter 3400 loss 0.0035431928087851297
iter 3500 loss 0.003452026836125641
iter 3600 loss 0.0033699789443636676
iter 3700 loss 0.003290415104909045
iter 3800 loss 0.0032095414739384295
iter 3900 loss 0.0031366168806888227
iter 4000 loss 0.003064416984713933
iter 4100 loss 0.003003983134923237
iter 4200 loss 0.002942758218360935
iter 4300 loss 0.002906304037530767
iter 4400 loss 0.0028622765088635693
iter 4500 loss 0.002827565137522672
iter 4600 loss 0.0027790111693924884
iter 4700 loss 0.0027251738856348567
iter 4800 loss 0.002674846619911267
epoch 0 loss 0.0027980606749385076
iter 0 loss 0.06256222724914551
iter 100 loss 0.002052925277840217
iter 200 loss 0.0012798905628183116
iter 300 loss 0.0009179363367372065
iter 400 loss 0.0007124708075358816
iter 500 loss 0.0005894838131299873
iter 600 loss 0.0005141064253127816
iter 700 loss 0.0005973530659763794
iter 800 loss 0.0005333243808330481
iter 900 loss 0.000503655705727858
iter 1000 loss 0.0004665928254498622
iter 1100 loss 0.0004322784631745414
iter 1200 loss 0.0005768404647382103
iter 1300 loss 0.0005437981932757372
iter 1400 loss 0.0005126628426032955
iter 1500 loss 0.000490487103744671
iter 1600 loss 0.0004682546565461321
iter 1700 loss 0.0004481833844251381
iter 1800 loss 0.0004292862376042295
iter 1900 loss 0.0004123657316043781
iter 2000 loss 0.00039784476821628064
iter 2100 loss 0.0003853066243522446
iter 2200 loss 0.00037805893904761865
iter 2300 loss 0.0013929028404205443
iter 2400 loss 0.003626850459189244
iter 2500 loss 0.003485423347356067
iter 2600 loss 0.0033545384696420906
iter 2700 loss 0.003655304120123814
iter 2800 loss 0.0037217396884589703
iter 2900 loss 0.0036121157452892825
iter 3000 loss 0.0034947954152449528
iter 3100 loss 0.0033837476020732804
iter 3200 loss 0.0032829169591043527
iter 3300 loss 0.003189298586879356
iter 3400 loss 0.0030982359618670387
iter 3500 loss 0.0030114239057977654
iter 3600 loss 0.002929173771067562
iter 3700 loss 0.0028524606887271615
iter 3800 loss 0.0027788857955409426
iter 3900 loss 0.0027095540197966577
iter 4000 loss 0.00264325586455237
iter 4100 loss 0.002581214472661048
iter 4200 loss 0.0025218292119760888
iter 4300 loss 0.0024690101834727303
iter 4400 loss 0.00241722800776735
iter 4500 loss 0.0023700903590012277
iter 4600 loss 0.002323118996809745
iter 4700 loss 0.0022750472945908297
iter 4800 loss 0.00222914263752994
epoch 1 loss 0.0022709592163706683
iter 0 loss 0.03915102779865265
iter 100 loss 0.0009319090102436159
iter 200 loss 0.0006257287109767629
iter 300 loss 0.00044494607521171147
iter 400 loss 0.00034224898196665243
iter 500 loss 0.0002811660830599782
iter 600 loss 0.00024367268408011195
iter 700 loss 0.00036274706646830765
iter 800 loss 0.00032156289496007286
iter 900 loss 0.00031305279470579306
iter 1000 loss 0.0002888614652058002
iter 1100 loss 0.00026581812755621973
iter 1200 loss 0.000384344158598767
iter 1300 loss 0.00036104400530000916
iter 1400 loss 0.00033852853341105827
iter 1500 loss 0.00032106404131828324
iter 1600 loss 0.000304362078903291
iter 1700 loss 0.0002893630255530062
iter 1800 loss 0.0002755672631569847
iter 1900 loss 0.00026335339144913843
iter 2000 loss 0.0002529436331334172
iter 2100 loss 0.00024386708731196087
iter 2200 loss 0.00023683430546179331
iter 2300 loss 0.001239650647321425
iter 2400 loss 0.0034690758902612214
iter 2500 loss 0.003331832054560878
iter 2600 loss 0.0032049800351959578
iter 2700 loss 0.0035139019313806994
iter 2800 loss 0.0035759240852076767
iter 2900 loss 0.0034643455345746183
iter 3000 loss 0.003350012725076516
iter 3100 loss 0.003242678129627852
iter 3200 loss 0.0031436484465713066
iter 3300 loss 0.003051081734015678
iter 3400 loss 0.002962519575302352
iter 3500 loss 0.0028785936936981246
iter 3600 loss 0.0027992202938503913
iter 3700 loss 0.00272466139730443
iter 3800 loss 0.002653630127110875
iter 3900 loss 0.002586344911130831
iter 4000 loss 0.002522267373174108
iter 4100 loss 0.002461844456687397
iter 4200 loss 0.002404117865181756
iter 4300 loss 0.002350881313493794
iter 4400 loss 0.0022996890923641486
iter 4500 loss 0.0022524111675200765
iter 4600 loss 0.002206139412243145
iter 4700 loss 0.0021598523586081027
iter 4800 loss 0.002115567373193098
epoch 2 loss 0.0021266728397671118
iter 0 loss 0.024567726999521255
iter 100 loss 0.0005075417101768763
iter 200 loss 0.00034450281909178995
iter 300 loss 0.00024467881934790204
iter 400 loss 0.0001871754717657475
iter 500 loss 0.00015288469872764822
iter 600 loss 0.00013193291087253075
iter 700 loss 0.00022638722193097236
iter 800 loss 0.00020022058237436716
iter 900 loss 0.00019106021772839107
iter 1000 loss 0.00017545927833023117
iter 1100 loss 0.00016093137619513525
iter 1200 loss 0.0002461277357433816
iter 1300 loss 0.0002310166846361582
iter 1400 loss 0.00021598298415841785
iter 1500 loss 0.00020404460157320218
iter 1600 loss 0.0001928551466118187
iter 1700 loss 0.00018281668195009438
iter 1800 loss 0.00017366874779853396
iter 1900 loss 0.00016560077903412151
iter 2000 loss 0.00015877329190305876
iter 2100 loss 0.0001525661784081962
iter 2200 loss 0.0001477210176758897
iter 2300 loss 0.001151318680887723
iter 2400 loss 0.003378468100303525
iter 2500 loss 0.003244078116145423
iter 2600 loss 0.003119927425582058
iter 2700 loss 0.0034187818394272756
iter 2800 loss 0.003475023099563824
iter 2900 loss 0.0033623930315163668
iter 3000 loss 0.0032508345394021343
iter 3100 loss 0.003146322939837747
iter 3200 loss 0.0030492049219861326
iter 3300 loss 0.0029581664837345714
iter 3400 loss 0.002871733078498737
iter 3500 loss 0.002790077544719467
iter 3600 loss 0.0027129039865907956
iter 3700 loss 0.002640272887063984
iter 3800 loss 0.0025711945742324317
iter 3900 loss 0.0025056345573933234
iter 4000 loss 0.0024432786511700082
iter 4100 loss 0.002384214757244479
iter 4200 loss 0.002327878199142841
iter 4300 loss 0.002275077026178632
iter 4400 loss 0.002224699737843228
iter 4500 loss 0.0021775717588897823
iter 4600 loss 0.0021318742876098038
iter 4700 loss 0.002086891395378687
iter 4800 loss 0.002043797368920376
epoch 3 loss 0.002038851659161164
iter 0 loss 0.01368490606546402
iter 100 loss 0.0002899187563010549
iter 200 loss 0.0002127417851557196
iter 300 loss 0.00015090202120527434
iter 400 loss 0.00011504659884357218
iter 500 loss 9.36904032255234e-05
iter 600 loss 8.067562920755056e-05
iter 700 loss 0.00014767947791285475
iter 800 loss 0.0001305471258446699
iter 900 loss 0.0001248439780273567
iter 1000 loss 0.00011445149655057891
iter 1100 loss 0.00010479278641372487
iter 1200 loss 0.00016459690613170115
iter 1300 loss 0.00015469908125561246
iter 1400 loss 0.00014447549063666387
iter 1500 loss 0.00013613536402972253
iter 1600 loss 0.0001284711068803698
iter 1700 loss 0.00012161851773784633
iter 1800 loss 0.00011543155631272681
iter 1900 loss 0.00010995435712022955
iter 2000 loss 0.00010537045733187348
iter 2100 loss 0.0001011135646404658
iter 2200 loss 9.759084837673784e-05
iter 2300 loss 0.0011004553442755269
iter 2400 loss 0.0033259856322897883
iter 2500 loss 0.003193378516257502
iter 2600 loss 0.003070920087520545
iter 2700 loss 0.0033616110470794766
iter 2800 loss 0.0034149639257899833
iter 2900 loss 0.0033023740953330803
iter 3000 loss 0.0031925923501395035
iter 3100 loss 0.0030898176936380957
iter 3200 loss 0.002993940508008301
iter 3300 loss 0.0029040006663172747
iter 3400 loss 0.0028189214531440594
iter 3500 loss 0.0027386396874197333
iter 3600 loss 0.0026627804482968545
iter 3700 loss 0.00259137070366796
iter 3800 loss 0.002523496581180502
iter 3900 loss 0.0024590086871501874
iter 4000 loss 0.0023977007766306808
iter 4100 loss 0.0023395330761804257
iter 4200 loss 0.0022840843342019665
iter 4300 loss 0.002231826079736008
iter 4400 loss 0.0021819728066418223
iter 4500 loss 0.002134981674943358
iter 4600 loss 0.0020896212436827093
iter 4700 loss 0.002045406597051897
iter 4800 loss 0.0020030251456945947
epoch 4 loss 0.0019883827318727255
--- Result:
Test failed! Accuracy = 0.14453791081905365<0.3
Test G: 0/20
--- Time taken: 165.83817172050476


--- TEST: batch_size= 128  num_epochs= 5  lr= 0.0001 

iter 0 loss 0.006560519337654114
iter 100 loss 0.00022368800188751897
iter 200 loss 0.0003198726776351229
iter 300 loss 0.000285706609325988
iter 400 loss 0.0002164351783987758
iter 500 loss 0.00017481507638295878
iter 600 loss 0.0001480446355659378
iter 700 loss 0.0004283275267164348
iter 800 loss 0.00039678482903043
iter 900 loss 0.00036197911060892335
iter 1000 loss 0.00033348982577519125
iter 1100 loss 0.00030394048446989594
iter 1200 loss 0.00031136018612447336
iter 1300 loss 0.00030080186439481707
iter 1400 loss 0.00028023724794291335
iter 1500 loss 0.00026262636537528386
iter 1600 loss 0.00024701294396288725
iter 1700 loss 0.00023313256197541107
iter 1800 loss 0.0002206420399993701
iter 1900 loss 0.00020947113690802764
iter 2000 loss 0.00019972298094267873
iter 2100 loss 0.0001908131034473573
iter 2200 loss 0.00018287558410265889
iter 2300 loss 0.0011751616383532581
iter 2400 loss 0.0033984596789398734
iter 2500 loss 0.0032628955038489223
iter 2600 loss 0.0031376500144286758
iter 2700 loss 0.003451678393907819
iter 2800 loss 0.003500660592424808
iter 2900 loss 0.0033856544794875912
iter 3000 loss 0.0032730689272802748
iter 3100 loss 0.0031676467962732
iter 3200 loss 0.0030690297917493527
iter 3300 loss 0.0029766024587411536
iter 3400 loss 0.002889304238051965
iter 3500 loss 0.0028069690738220807
iter 3600 loss 0.0027291681530319124
iter 3700 loss 0.002655814145414335
iter 3800 loss 0.0025861515016120146
iter 3900 loss 0.002519980190568907
iter 4000 loss 0.002457103959385052
iter 4100 loss 0.002397362573218211
iter 4200 loss 0.0023404540726786297
iter 4300 loss 0.002286330008831356
iter 4400 loss 0.0022349787224589684
iter 4500 loss 0.002186395617897375
iter 4600 loss 0.0021395492382148868
iter 4700 loss 0.002094285235916102
iter 4800 loss 0.002050781874172773
epoch 0 loss 0.0020509866199814345
iter 0 loss 0.0065489886328577995
iter 100 loss 0.00019357715151308735
iter 200 loss 0.00013888433790976815
iter 300 loss 0.000107011801025278
iter 400 loss 8.171839601998913e-05
iter 500 loss 6.657453723291583e-05
iter 600 loss 5.721861174086026e-05
iter 700 loss 8.28304157536297e-05
iter 800 loss 7.556810755097407e-05
iter 900 loss 7.134391969705068e-05
iter 1000 loss 6.703897746979941e-05
iter 1100 loss 6.143507403294073e-05
iter 1200 loss 0.00010472449480535681
iter 1300 loss 0.00010502368218025295
iter 1400 loss 9.811832420618716e-05
iter 1500 loss 9.233182777325025e-05
iter 1600 loss 8.715526644727976e-05
iter 1700 loss 8.249595072112814e-05
iter 1800 loss 7.828228129532108e-05
iter 1900 loss 7.452387274084711e-05
iter 2000 loss 7.137143317792371e-05
iter 2100 loss 6.844176072945738e-05
iter 2200 loss 6.58960201636901e-05
iter 2300 loss 0.0010384765499201105
iter 2400 loss 0.0032647433207806283
iter 2500 loss 0.0031344655771923546
iter 2600 loss 0.0030141323244760815
iter 2700 loss 0.003271024327586956
iter 2800 loss 0.0033248614226714214
iter 2900 loss 0.003215256523796128
iter 3000 loss 0.003108306049254803
iter 3100 loss 0.0030081807816117437
iter 3200 loss 0.002914482085527657
iter 3300 loss 0.0028266528548372884
iter 3400 loss 0.0027437345512561218
iter 3500 loss 0.0026655223278930565
iter 3600 loss 0.002591626295358311
iter 3700 loss 0.0025219631696733647
iter 3800 loss 0.0024558164686266826
iter 3900 loss 0.0023929755663097585
iter 4000 loss 0.002333264627625055
iter 4100 loss 0.002276523667341214
iter 4200 loss 0.002222473431201343
iter 4300 loss 0.002171081674258403
iter 4400 loss 0.002122282326147596
iter 4500 loss 0.0020759140950106832
iter 4600 loss 0.002031377993527467
iter 4700 loss 0.001988374684981648
iter 4800 loss 0.001947069701672257
epoch 1 loss 0.0019390946465845625
iter 0 loss 0.005505547858774662
iter 100 loss 0.0001564311817286814
iter 200 loss 0.0001018557715941648
iter 300 loss 7.700410976796044e-05
iter 400 loss 5.891552749962467e-05
iter 500 loss 4.808529174733631e-05
iter 600 loss 4.1482833984871424e-05
iter 700 loss 7.035982556727957e-05
iter 800 loss 6.440269051315764e-05
iter 900 loss 6.0301220077998615e-05
iter 1000 loss 5.644418330315408e-05
iter 1100 loss 5.17159097868719e-05
iter 1200 loss 8.49439161383218e-05
iter 1300 loss 8.512083616297772e-05
iter 1400 loss 7.95197132286036e-05
iter 1500 loss 7.48258360070711e-05
iter 1600 loss 7.063532825001024e-05
iter 1700 loss 6.686488278794636e-05
iter 1800 loss 6.346796202163809e-05
iter 1900 loss 6.0438859153926645e-05
iter 2000 loss 5.792011981124477e-05
iter 2100 loss 5.5553685774406325e-05
iter 2200 loss 5.348832637979415e-05
iter 2300 loss 0.0010255197656374223
iter 2400 loss 0.003250449475859907
iter 2500 loss 0.003120706038734003
iter 2600 loss 0.0030008804635368734
iter 2700 loss 0.003261602449902528
iter 2800 loss 0.0033148931318974367
iter 2900 loss 0.0032045712362680696
iter 3000 loss 0.0030979484872524807
iter 3100 loss 0.002998144290465075
iter 3200 loss 0.0029047195477319124
iter 3300 loss 0.002817114090589884
iter 3400 loss 0.002734448053835999
iter 3500 loss 0.002656480320796313
iter 3600 loss 0.0025828239940138757
iter 3700 loss 0.0025133983044959714
iter 3800 loss 0.002447473813975303
iter 3900 loss 0.002384836773998718
iter 4000 loss 0.002325318865798526
iter 4100 loss 0.002268752594513186
iter 4200 loss 0.0022148745705309913
iter 4300 loss 0.0021636240185545414
iter 4400 loss 0.0021149251351727637
iter 4500 loss 0.0020686124482433405
iter 4600 loss 0.0020241731525746675
iter 4700 loss 0.0019812958675426927
iter 4800 loss 0.0019401238462823872
epoch 2 loss 0.0019256926130552766
iter 0 loss 0.004678480327129364
iter 100 loss 0.0001329800796940059
iter 200 loss 8.658094954277582e-05
iter 300 loss 6.522247567803081e-05
iter 400 loss 4.9881898980981676e-05
iter 500 loss 4.069753422735668e-05
iter 600 loss 3.5088045932905564e-05
iter 700 loss 6.044062004739044e-05
iter 800 loss 5.5371667630301014e-05
iter 900 loss 5.161461911698365e-05
iter 1000 loss 4.81570950265072e-05
iter 1100 loss 4.412048804304134e-05
iter 1200 loss 7.142684908645003e-05
iter 1300 loss 7.155724842299274e-05
iter 1400 loss 6.685221305955076e-05
iter 1500 loss 6.290342427372653e-05
iter 1600 loss 5.93803391509438e-05
iter 1700 loss 5.621529325743274e-05
iter 1800 loss 5.337286354065799e-05
iter 1900 loss 5.083551281983887e-05
iter 2000 loss 4.8740593850001106e-05
iter 2100 loss 4.6757408609101524e-05
iter 2200 loss 4.500923267727395e-05
iter 2300 loss 0.001016372752947352
iter 2400 loss 0.0032400573887999837
iter 2500 loss 0.003110699797476956
iter 2600 loss 0.0029912421590005387
iter 2700 loss 0.0032513731262921565
iter 2800 loss 0.0033040294543875744
iter 2900 loss 0.003193351992058728
iter 3000 loss 0.003087082255364999
iter 3100 loss 0.0029876184357992934
iter 3200 loss 0.002894489351290029
iter 3300 loss 0.0028071349248196893
iter 3400 loss 0.0027247409101989243
iter 3500 loss 0.0026470358503222035
iter 3600 loss 0.0025736337955480893
iter 3700 loss 0.002504456680774792
iter 3800 loss 0.002438765477979226
iter 3900 loss 0.0023763432413998306
iter 4000 loss 0.0023170289268377472
iter 4100 loss 0.0022606492527815695
iter 4200 loss 0.0022069525557281322
iter 4300 loss 0.0021558552129547965
iter 4400 loss 0.00210726778831551
iter 4500 loss 0.0020610334273540583
iter 4600 loss 0.0020166793782993003
iter 4700 loss 0.001973936047718915
iter 4800 loss 0.0019329048353948933
epoch 3 loss 0.0019149152153973244
iter 0 loss 0.003986550495028496
iter 100 loss 0.00011477415337860365
iter 200 loss 7.277269617982562e-05
iter 300 loss 5.44052420695533e-05
iter 400 loss 4.163379414530664e-05
iter 500 loss 3.399020133592912e-05
iter 600 loss 2.933753120761289e-05
iter 700 loss 5.125184563086789e-05
iter 800 loss 4.700915092422172e-05
iter 900 loss 4.3674412907393094e-05
iter 1000 loss 4.064945716725726e-05
iter 1100 loss 3.725012178575483e-05
iter 1200 loss 6.025287562135674e-05
iter 1300 loss 6.0404016025527246e-05
iter 1400 loss 5.6443042907576494e-05
iter 1500 loss 5.311675502569385e-05
iter 1600 loss 5.0147834950203364e-05
iter 1700 loss 4.748550479225747e-05
iter 1800 loss 4.509961728035301e-05
iter 1900 loss 4.296634049824168e-05
iter 2000 loss 4.12208085977638e-05
iter 2100 loss 3.955457022286121e-05
iter 2200 loss 3.807200473668394e-05
iter 2300 loss 0.001008948978122657
iter 2400 loss 0.0032314908628326157
iter 2500 loss 0.003102452254796565
iter 2600 loss 0.0029832982692199
iter 2700 loss 0.003242940199161354
iter 2800 loss 0.0032949881146503743
iter 2900 loss 0.003184017156581925
iter 3000 loss 0.0030780422614189724
iter 3100 loss 0.0029788612927523833
iter 3200 loss 0.002885979502051788
iter 3300 loss 0.0027988362375030345
iter 3400 loss 0.002716668642833247
iter 3500 loss 0.002639181745559639
iter 3600 loss 0.0025659905520062708
iter 3700 loss 0.002497019042120246
iter 3800 loss 0.002431521998108705
iter 3900 loss 0.002369278356761826
iter 4000 loss 0.0023101332893210565
iter 4100 loss 0.002253908839696014
iter 4200 loss 0.002200362506138472
iter 4300 loss 0.002149393732159809
iter 4400 loss 0.0021009022029654136
iter 4500 loss 0.0020547317875533886
iter 4600 loss 0.0020104403569383296
iter 4700 loss 0.0019678076515974663
iter 4800 loss 0.0019268937829943617
epoch 4 loss 0.0019067483495439513
--- Result:
Test failed! Accuracy = 0.14425840973854065<0.3
Test G: 0/20
--- Time taken: 161.2735722064972


--- TEST: batch_size= 128  num_epochs= 10  lr= 0.001 

iter 0 loss 0.0034086357336491346
iter 100 loss 9.23321925612153e-05
iter 200 loss 5.750301836055927e-05
iter 300 loss 4.054909854752369e-05
iter 400 loss 3.081470694532324e-05
iter 500 loss 2.4981500234412985e-05
iter 600 loss 2.1292868955235172e-05
iter 700 loss 2.1967589846138067e-05
iter 800 loss 1.9437357835183743e-05
iter 900 loss 1.956359767089727e-05
iter 1000 loss 1.7863284460214093e-05
iter 1100 loss 1.638633038052255e-05
iter 1200 loss 4.630525786367095e-05
iter 1300 loss 4.322777021763395e-05
iter 1400 loss 4.031881875291867e-05
iter 1500 loss 3.783837595728001e-05
iter 1600 loss 3.5620019358849696e-05
iter 1700 loss 3.365589762376681e-05
iter 1800 loss 3.189607173301999e-05
iter 1900 loss 3.0326914485930528e-05
iter 2000 loss 2.8964859372171487e-05
iter 2100 loss 2.771380807163443e-05
iter 2200 loss 2.6625987212307605e-05
iter 2300 loss 0.0010444594234013907
iter 2400 loss 0.003255794071444854
iter 2500 loss 0.0031257004107679374
iter 2600 loss 0.003005614828368736
iter 2700 loss 0.003305237393026323
iter 2800 loss 0.003380964769027134
iter 2900 loss 0.003265941024459921
iter 3000 loss 0.0031571896621255817
iter 3100 loss 0.0030554482907670976
iter 3200 loss 0.00296017324542484
iter 3300 loss 0.0028706668022205847
iter 3400 loss 0.0027863452991571335
iter 3500 loss 0.0027068371503647416
iter 3600 loss 0.0026317628212381315
iter 3700 loss 0.002560776018023539
iter 3800 loss 0.002493496518501738
iter 3900 loss 0.0024296582108918135
iter 4000 loss 0.002368992018016958
iter 4100 loss 0.002311337710347229
iter 4200 loss 0.0022564198668017137
iter 4300 loss 0.0022052381680348243
iter 4400 loss 0.0021559911223980152
iter 4500 loss 0.0021091627653580755
iter 4600 loss 0.002065794505713948
iter 4700 loss 0.0020220023108815308
iter 4800 loss 0.001980107418786815
epoch 0 loss 0.0020704808760263304
iter 0 loss 0.004672245588153601
iter 100 loss 0.00012595998795053549
iter 200 loss 0.00011911711386455321
iter 300 loss 8.15911734036756e-05
iter 400 loss 6.176374254017019e-05
iter 500 loss 4.991618331388451e-05
iter 600 loss 4.254638988221149e-05
iter 700 loss 0.00017092823262547567
iter 800 loss 0.0001499208294552751
iter 900 loss 0.0001407819491075268
iter 1000 loss 0.0001272594115678087
iter 1100 loss 0.00011590017127953939
iter 1200 loss 0.00018335560911576692
iter 1300 loss 0.0001700803224497336
iter 1400 loss 0.00015820456744151525
iter 1500 loss 0.00014801808274934408
iter 1600 loss 0.00013905169553625054
iter 1700 loss 0.00013112466628111162
iter 1800 loss 0.00012402528811306812
iter 1900 loss 0.00011767812276709012
iter 2000 loss 0.00011206734518464168
iter 2100 loss 0.00010692922908399418
iter 2200 loss 0.00010240156454051854
iter 2300 loss 0.0011266859240320669
iter 2400 loss 0.003339508332443983
iter 2500 loss 0.003206104723259626
iter 2600 loss 0.0030829593489921318
iter 2700 loss 0.0033699218142166525
iter 2800 loss 0.003415026585048979
iter 2900 loss 0.00329959479201075
iter 3000 loss 0.0031897405564560474
iter 3100 loss 0.00308694989657631
iter 3200 loss 0.002990762935137928
iter 3300 loss 0.0029003867306873284
iter 3400 loss 0.0028152071300647096
iter 3500 loss 0.002734896359129899
iter 3600 loss 0.0026590513892892566
iter 3700 loss 0.00258745579484331
iter 3800 loss 0.0025195326194351764
iter 3900 loss 0.0024550353468768787
iter 4000 loss 0.002393742042778159
iter 4100 loss 0.002335500029668129
iter 4200 loss 0.0022800004181245084
iter 4300 loss 0.002227283169603038
iter 4400 loss 0.0021769648485637162
iter 4500 loss 0.002128995985951018
iter 4600 loss 0.002083056688354045
iter 4700 loss 0.002038846229873591
iter 4800 loss 0.0019964871779711737
epoch 1 loss 0.0019759762038617265
iter 0 loss 0.0013431175611913204
iter 100 loss 5.1799018583641567e-05
iter 200 loss 3.3494799404636875e-05
iter 300 loss 2.4083411725228248e-05
iter 400 loss 1.8606322390284578e-05
iter 500 loss 1.5380349727611666e-05
iter 600 loss 1.3809839732836468e-05
iter 700 loss 7.087409954652483e-05
iter 800 loss 6.237147616881865e-05
iter 900 loss 5.888851900154935e-05
iter 1000 loss 5.344480085433723e-05
iter 1100 loss 4.880520363773306e-05
iter 1200 loss 9.43347999103856e-05
iter 1300 loss 8.777844976639153e-05
iter 1400 loss 8.172460639886986e-05
iter 1500 loss 7.662339175139078e-05
iter 1600 loss 7.204465299036713e-05
iter 1700 loss 6.801260127728514e-05
iter 1800 loss 6.442608273360208e-05
iter 1900 loss 6.121228294959502e-05
iter 2000 loss 5.844792846047007e-05
iter 2100 loss 5.586544112670723e-05
iter 2200 loss 5.356524319356348e-05
iter 2300 loss 0.001083088253359105
iter 2400 loss 0.003299918791084569
iter 2500 loss 0.0031680993910527693
iter 2600 loss 0.0030464139670264576
iter 2700 loss 0.0033429987226438656
iter 2800 loss 0.003386707464054275
iter 2900 loss 0.003270725617460463
iter 3000 loss 0.0031618217220747433
iter 3100 loss 0.0030599270306045506
iter 3200 loss 0.00296452736799737
iter 3300 loss 0.0028748925058773574
iter 3400 loss 0.0027904494755602817
iter 3500 loss 0.0027108228561112976
iter 3600 loss 0.002635639075524135
iter 3700 loss 0.002564742262516916
iter 3800 loss 0.0024974738313931185
iter 3900 loss 0.0024335349941995614
iter 4000 loss 0.0023727684027877254
iter 4100 loss 0.002315012144313466
iter 4200 loss 0.002259989700903605
iter 4300 loss 0.002208613344385901
iter 4400 loss 0.002158826761112246
iter 4500 loss 0.0021112702212905867
iter 4600 loss 0.0020656567502406057
iter 4700 loss 0.0020217894890747624
iter 4800 loss 0.0019797724631667106
epoch 2 loss 0.0019553793138414174
iter 0 loss 0.0008705905638635159
iter 100 loss 3.660925050900176e-05
iter 200 loss 7.277302444180264e-05
iter 300 loss 5.0092662643109e-05
iter 400 loss 3.811776368956658e-05
iter 500 loss 3.097583308834022e-05
iter 600 loss 2.688607900080465e-05
iter 700 loss 6.627451062473605e-05
iter 800 loss 5.834224306168851e-05
iter 900 loss 5.4748074150992114e-05
iter 1000 loss 4.9651552394657254e-05
iter 1100 loss 4.5339117104690386e-05
iter 1200 loss 7.54378218562654e-05
iter 1300 loss 7.027097073482201e-05
iter 1400 loss 6.548479828728858e-05
iter 1500 loss 6.142454434780651e-05
iter 1600 loss 5.779055605984897e-05
iter 1700 loss 5.460520443296064e-05
iter 1800 loss 5.1786726796686096e-05
iter 1900 loss 4.9232773032077206e-05
iter 2000 loss 4.7067291578853715e-05
iter 2100 loss 4.501060436290605e-05
iter 2200 loss 4.32303139282292e-05
iter 2300 loss 0.0010705845006051724
iter 2400 loss 0.0032904746161813068
iter 2500 loss 0.00315902335035298
iter 2600 loss 0.003037675871844907
iter 2700 loss 0.0033383426417511936
iter 2800 loss 0.0033849804565590142
iter 2900 loss 0.003270182512763177
iter 3000 loss 0.003161297464650642
iter 3100 loss 0.003059418634156134
iter 3200 loss 0.0029640564838447157
iter 3300 loss 0.0028744556998861665
iter 3400 loss 0.002790027580493016
iter 3500 loss 0.002710444655828749
iter 3600 loss 0.0026352809853703568
iter 3700 loss 0.0025644785159863945
iter 3800 loss 0.0024973346408522514
iter 3900 loss 0.002433402949953615
iter 4000 loss 0.0023726403976459965
iter 4100 loss 0.0023148912006482622
iter 4200 loss 0.0022598673448583208
iter 4300 loss 0.0022075474899105876
iter 4400 loss 0.002157678015257143
iter 4500 loss 0.0021100941056253785
iter 4600 loss 0.002064491656657371
iter 4700 loss 0.0020206469156668324
iter 4800 loss 0.001978651154381234
epoch 3 loss 0.001949939184429463
iter 0 loss 0.0005016264040023088
iter 100 loss 2.3649784229701703e-05
iter 200 loss 1.683618420406787e-05
iter 300 loss 1.2522593038116261e-05
iter 400 loss 9.882511624952561e-06
iter 500 loss 8.336668860966619e-06
iter 600 loss 7.901207268499118e-06
iter 700 loss 3.497487821939587e-05
iter 800 loss 3.093200732411297e-05
iter 900 loss 2.980105193173192e-05
iter 1000 loss 2.7138367109095903e-05
iter 1100 loss 2.4865555041453774e-05
iter 1200 loss 4.702476585475722e-05
iter 1300 loss 4.3958462520493525e-05
iter 1400 loss 4.100747172158949e-05
iter 1500 loss 3.860432783262122e-05
iter 1600 loss 3.638351640114908e-05
iter 1700 loss 3.4459427670347314e-05
iter 1800 loss 3.2776997715593356e-05
iter 1900 loss 3.120821302165619e-05
iter 2000 loss 2.9952729557438926e-05
iter 2100 loss 2.8699575534244175e-05
iter 2200 loss 2.7593237501530678e-05
iter 2300 loss 0.0010544711397739
iter 2400 loss 0.003276540600850897
iter 2500 loss 0.0031456415022799514
iter 2600 loss 0.0030248078014272284
iter 2700 loss 0.0033270120533988395
iter 2800 loss 0.0033708028429865703
iter 2900 loss 0.0032552678387432156
iter 3000 loss 0.0031468698413735672
iter 3100 loss 0.0030454508092245284
iter 3200 loss 0.0029504772111116095
iter 3300 loss 0.002861237615739945
iter 3400 loss 0.002777184042133359
iter 3500 loss 0.002697928796732044
iter 3600 loss 0.002623101433381574
iter 3700 loss 0.002552659592668535
iter 3800 loss 0.002485887247814889
iter 3900 loss 0.0024222438276991095
iter 4000 loss 0.0023617519753161313
iter 4100 loss 0.0023042446295111897
iter 4200 loss 0.002249465149071375
iter 4300 loss 0.0021978410577234296
iter 4400 loss 0.0021482397767854976
iter 4500 loss 0.002100854825366492
iter 4600 loss 0.0020554156778369312
iter 4700 loss 0.002011756631569105
iter 4800 loss 0.001969936635163266
epoch 4 loss 0.001939912342094503
iter 0 loss 0.0003990154364146292
iter 100 loss 1.9191652346208766e-05
iter 200 loss 5.1717910858527675e-05
iter 300 loss 3.557344174483219e-05
iter 400 loss 2.7164323414683366e-05
iter 500 loss 2.2150424273288712e-05
iter 600 loss 1.9376403507240427e-05
iter 700 loss 3.992459373723598e-05
iter 800 loss 3.525475567182006e-05
iter 900 loss 3.322146488125358e-05
iter 1000 loss 3.017354935560183e-05
iter 1100 loss 2.761295277607007e-05
iter 1200 loss 4.521792148876727e-05
iter 1300 loss 4.224483313167832e-05
iter 1400 loss 3.9459705883264e-05
iter 1500 loss 3.712494361218758e-05
iter 1600 loss 3.49926415119603e-05
iter 1700 loss 3.317821629461041e-05
iter 1800 loss 3.1608507522475414e-05
iter 1900 loss 3.0101305005885046e-05
iter 2000 loss 2.894041288176316e-05
iter 2100 loss 2.7729689913725814e-05
iter 2200 loss 2.6656549053731083e-05
iter 2300 loss 0.0010517218812918908
iter 2400 loss 0.0032751058396125734
iter 2500 loss 0.003144255505129026
iter 2600 loss 0.003023467158757983
iter 2700 loss 0.0033249048221028646
iter 2800 loss 0.0033706273506570604
iter 2900 loss 0.0032559557037603404
iter 3000 loss 0.0031475346771971903
iter 3100 loss 0.0030460933933044044
iter 3200 loss 0.002951105577817158
iter 3300 loss 0.0028618563521182823
iter 3400 loss 0.002777785383660236
iter 3500 loss 0.0026985316262011227
iter 3600 loss 0.00262369188060605
iter 3700 loss 0.0025533172392174135
iter 3800 loss 0.0024866227782453596
iter 3900 loss 0.0024229617400617826
iter 4000 loss 0.002362452703905174
iter 4100 loss 0.002304941340437957
iter 4200 loss 0.002250145485842404
iter 4300 loss 0.0021980204157646
iter 4400 loss 0.0021483110483276656
iter 4500 loss 0.0021008835654223053
iter 4600 loss 0.002055407787270778
iter 4700 loss 0.0020117437145766963
iter 4800 loss 0.001969919175790445
epoch 5 loss 0.001938517158292904
iter 0 loss 0.00045921263517811894
iter 100 loss 1.969669113863671e-05
iter 200 loss 1.3431986314404567e-05
iter 300 loss 9.986562411864253e-06
iter 400 loss 7.94021292492069e-06
iter 500 loss 6.73630156661475e-06
iter 600 loss 6.445404189356375e-06
iter 700 loss 2.402013684817142e-05
iter 800 loss 2.1325340655503158e-05
iter 900 loss 2.038665964938949e-05
iter 1000 loss 1.8588058106478595e-05
iter 1100 loss 1.707572940793829e-05
iter 1200 loss 3.226948486734955e-05
iter 1300 loss 3.0239389100585143e-05
iter 1400 loss 2.8242355415597538e-05
iter 1500 loss 2.665109119461013e-05
iter 1600 loss 2.516413057851767e-05
iter 1700 loss 2.3954284290278704e-05
iter 1800 loss 2.2931801074314e-05
iter 1900 loss 2.186952274019105e-05
iter 2000 loss 2.110431701290647e-05
iter 2100 loss 2.024965443724484e-05
iter 2200 loss 1.948955718608749e-05
iter 2300 loss 0.0010433639397167275
iter 2400 loss 0.0032675103753527597
iter 2500 loss 0.003136960437237716
iter 2600 loss 0.003016451648865243
iter 2700 loss 0.003316484519440391
iter 2800 loss 0.0033600626887152728
iter 2900 loss 0.003244751956465081
iter 3000 loss 0.003136697863356
iter 3100 loss 0.0030356021490890723
iter 3200 loss 0.0029409089157832443
iter 3300 loss 0.0028519341437608543
iter 3400 loss 0.0027681445619252088
iter 3500 loss 0.0026891393039349073
iter 3600 loss 0.002614554544494467
iter 3700 loss 0.0025444279251737026
iter 3800 loss 0.0024779730642656692
iter 3900 loss 0.0024145304179328878
iter 4000 loss 0.002354226355957356
iter 4100 loss 0.002296893927992432
iter 4200 loss 0.002242282701588891
iter 4300 loss 0.002190619606215035
iter 4400 loss 0.0021411193798290605
iter 4500 loss 0.002093838825324331
iter 4600 loss 0.0020485124389175487
iter 4700 loss 0.0020049926741700034
iter 4800 loss 0.0019632990796861964
epoch 6 loss 0.0019312827995447478
iter 0 loss 0.00044902166700921953
iter 100 loss 1.937649934385119e-05
iter 200 loss 4.8616615533400864e-05
iter 300 loss 3.3326300429181355e-05
iter 400 loss 2.5445291090758395e-05
iter 500 loss 2.073584158790686e-05
iter 600 loss 1.80985640529763e-05
iter 700 loss 3.0764567427974133e-05
iter 800 loss 2.7224793369386543e-05
iter 900 loss 2.5429530217491735e-05
iter 1000 loss 2.3105607836570796e-05
iter 1100 loss 2.1175452762578493e-05
iter 1200 loss 3.3247524741721546e-05
iter 1300 loss 3.111262009160692e-05
iter 1400 loss 2.917948185388739e-05
iter 1500 loss 2.7526688750879497e-05
iter 1600 loss 2.59917632168746e-05
iter 1700 loss 2.4775073310735367e-05
iter 1800 loss 2.3756245938914847e-05
iter 1900 loss 2.265428312932386e-05
iter 2000 loss 2.190314705541263e-05
iter 2100 loss 2.101568091801104e-05
iter 2200 loss 2.02208228870869e-05
iter 2300 loss 0.0010423860995589455
iter 2400 loss 0.0032668368836103138
iter 2500 loss 0.003136308645823009
iter 2600 loss 0.0030158182934079303
iter 2700 loss 0.0033147175576753677
iter 2800 loss 0.0033598416917776618
iter 2900 loss 0.0032452316035892267
iter 3000 loss 0.003137162393416578
iter 3100 loss 0.003036052495652423
iter 3200 loss 0.0029413525149146373
iter 3300 loss 0.0028523750056787278
iter 3400 loss 0.00276857621864945
iter 3500 loss 0.002689576340161583
iter 3600 loss 0.0026149794976139894
iter 3700 loss 0.0025448815627206523
iter 3800 loss 0.002478461098547447
iter 3900 loss 0.0024150062644238994
iter 4000 loss 0.0023546926197458335
iter 4100 loss 0.002297364117223286
iter 4200 loss 0.0022427433348198495
iter 4300 loss 0.0021907510869569277
iter 4400 loss 0.002141163213890497
iter 4500 loss 0.002093857286317929
iter 4600 loss 0.0020484948160684295
iter 4700 loss 0.002004970697442445
iter 4800 loss 0.001963276016299066
epoch 7 loss 0.0019309430738567884
iter 0 loss 0.00040809219353832304
iter 100 loss 1.7757519135377135e-05
iter 200 loss 1.1590403259314239e-05
iter 300 loss 8.647240758025789e-06
iter 400 loss 6.911571403522029e-06
iter 500 loss 5.888635405471996e-06
iter 600 loss 5.663575845465129e-06
iter 700 loss 1.7476651745082162e-05
iter 800 loss 1.558305909379107e-05
iter 900 loss 1.4897287020856363e-05
iter 1000 loss 1.3610987154818546e-05
iter 1100 loss 1.2541573892163262e-05
iter 1200 loss 2.30239829512602e-05
iter 1300 loss 2.1628122989366477e-05
iter 1400 loss 2.023793600937065e-05
iter 1500 loss 1.916099610918426e-05
iter 1600 loss 1.8135961866887248e-05
iter 1700 loss 1.7407585222271503e-05
iter 1800 loss 1.6823817492785516e-05
iter 1900 loss 1.607653749127739e-05
iter 2000 loss 1.5624319374204324e-05
iter 2100 loss 1.5019617182561424e-05
iter 2200 loss 1.4479317115581812e-05
iter 2300 loss 0.0010355521885105896
iter 2400 loss 0.0032601018979203887
iter 2500 loss 0.0031298399621169934
iter 2600 loss 0.0030096008166884624
iter 2700 loss 0.003306211977740862
iter 2800 loss 0.0033498073610015223
iter 2900 loss 0.003234772806513876
iter 3000 loss 0.0031270481850331
iter 3100 loss 0.003026261126102932
iter 3200 loss 0.0029318416657602452
iter 3300 loss 0.0028431289649944787
iter 3400 loss 0.002759593667377265
iter 3500 loss 0.002680828767107963
iter 3600 loss 0.002606471276103285
iter 3700 loss 0.0025365757732775146
iter 3800 loss 0.002470371619315573
iter 3900 loss 0.002407123313716576
iter 4000 loss 0.0023470025504597285
iter 4100 loss 0.00228984380359072
iter 4200 loss 0.0022353966151957283
iter 4300 loss 0.0021837847448279435
iter 4400 loss 0.002134381937841382
iter 4500 loss 0.002087216794172338
iter 4600 loss 0.002042003538942016
iter 4700 loss 0.0019986169833412794
iter 4800 loss 0.0019570475027116
epoch 8 loss 0.0019243868545931592
iter 0 loss 0.0004566551651805639
iter 100 loss 1.9128992121160522e-05
iter 200 loss 3.831524003910986e-05
iter 300 loss 2.6328754592534243e-05
iter 400 loss 2.01719690885103e-05
iter 500 loss 1.649503628219852e-05
iter 600 loss 1.4496543319724582e-05
iter 700 loss 2.330460221937639e-05
iter 800 loss 2.0682217279089004e-05
iter 900 loss 1.9279558595670573e-05
iter 1000 loss 1.754292063983153e-05
iter 1100 loss 1.6111709179469163e-05
iter 1200 loss 2.4950803857023586e-05
iter 1300 loss 2.3394786786393537e-05
iter 1400 loss 2.205182455659357e-05
iter 1500 loss 2.0872907348523955e-05
iter 1600 loss 1.9750728986279872e-05
iter 1700 loss 1.8952703099244578e-05
iter 1800 loss 1.831092194671393e-05
iter 1900 loss 1.7486098177727665e-05
iter 2000 loss 1.6994137043680647e-05
iter 2100 loss 1.633059571748038e-05
iter 2200 loss 1.572999951055418e-05
iter 2300 loss 0.0010354888060562977
iter 2400 loss 0.003259907339319718
iter 2500 loss 0.0031296503511451346
iter 2600 loss 0.003009410991874706
iter 2700 loss 0.003304772721563843
iter 2800 loss 0.0033493288694931562
iter 2900 loss 0.0032348356137353674
iter 3000 loss 0.0031271096476031455
iter 3100 loss 0.003026321942833034
iter 3200 loss 0.0029319142146103927
iter 3300 loss 0.0028432105448696968
iter 3400 loss 0.002759676972987789
iter 3500 loss 0.0026809239759198766
iter 3600 loss 0.002606562435032195
iter 3700 loss 0.0025366793026387153
iter 3800 loss 0.0024704700138817723
iter 3900 loss 0.0024072169185677447
iter 4000 loss 0.0023470960892482797
iter 4100 loss 0.0022899464553323176
iter 4200 loss 0.0022354981314149564
iter 4300 loss 0.0021836500740609536
iter 4400 loss 0.0021341916895497998
iter 4500 loss 0.0020870160404403894
iter 4600 loss 0.002041776215424336
iter 4700 loss 0.0019983897825189946
iter 4800 loss 0.00195682483322696
epoch 9 loss 0.0019245232206035537
--- Result:
Test failed! Accuracy = 0.14111743867397308<0.3
Test G: 0/20
--- Time taken: 319.0000069141388


--- TEST: batch_size= 128  num_epochs= 10  lr= 0.0005 

iter 0 loss 0.0003240692603867501
iter 100 loss 2.1112590522786585e-05
iter 200 loss 1.3924813391718972e-05
iter 300 loss 1.051377987407425e-05
iter 400 loss 8.453036592701004e-06
iter 500 loss 7.215941739381734e-06
iter 600 loss 6.714915903307543e-06
iter 700 loss 1.788723313040513e-05
iter 800 loss 1.625001817281761e-05
iter 900 loss 1.4967363628257448e-05
iter 1000 loss 1.3734917975493223e-05
iter 1100 loss 1.2663589847669095e-05
iter 1200 loss 1.7829030410043124e-05
iter 1300 loss 1.70609218392496e-05
iter 1400 loss 1.59908789523333e-05
iter 1500 loss 1.5104228704958774e-05
iter 1600 loss 1.4300594908001778e-05
iter 1700 loss 1.360528598925189e-05
iter 1800 loss 1.2971134950411079e-05
iter 1900 loss 1.238174842869043e-05
iter 2000 loss 1.1906126186689093e-05
iter 2100 loss 1.1431263300786328e-05
iter 2200 loss 1.1013293429585437e-05
iter 2300 loss 0.0010306702522334358
iter 2400 loss 0.0032508556218277173
iter 2500 loss 0.0031209370335592377
iter 2600 loss 0.0030010094083153323
iter 2700 loss 0.003334752792686032
iter 2800 loss 0.003379879656512573
iter 2900 loss 0.0032641890716327954
iter 3000 loss 0.003155475695234918
iter 3100 loss 0.0030537624051333694
iter 3200 loss 0.002958437716782952
iter 3300 loss 0.002868887594313153
iter 3400 loss 0.002784578287359448
iter 3500 loss 0.00270508673350147
iter 3600 loss 0.0026300152992689554
iter 3700 loss 0.002559090461828354
iter 3800 loss 0.002491951582319568
iter 3900 loss 0.0024281216884083556
iter 4000 loss 0.0023674660005457123
iter 4100 loss 0.00230978686554079
iter 4200 loss 0.0022548454912602787
iter 4300 loss 0.0022025159213676106
iter 4400 loss 0.002152549667973346
iter 4500 loss 0.002104816382449076
iter 4600 loss 0.002059161232313906
iter 4700 loss 0.0020153967379974084
iter 4800 loss 0.0019734532170915145
epoch 0 loss 0.0019515106817625885
iter 0 loss 0.00029309355886653066
iter 100 loss 1.5234550659496932e-05
iter 200 loss 1.1855746038236248e-05
iter 300 loss 8.741913492156413e-06
iter 400 loss 6.938366581189733e-06
iter 500 loss 5.861903262170645e-06
iter 600 loss 5.305306165242358e-06
iter 700 loss 1.0041903201181802e-05
iter 800 loss 9.10882219608886e-06
iter 900 loss 8.468174370806954e-06
iter 1000 loss 7.806509510392952e-06
iter 1100 loss 7.231137556366302e-06
iter 1200 loss 1.2126867749033133e-05
iter 1300 loss 1.161420071963811e-05
iter 1400 loss 1.0909888685729713e-05
iter 1500 loss 1.0323271357480895e-05
iter 1600 loss 9.793381189315428e-06
iter 1700 loss 9.326682196669503e-06
iter 1800 loss 8.920623471853351e-06
iter 1900 loss 8.53890523206814e-06
iter 2000 loss 8.268008816988715e-06
iter 2100 loss 7.971208290491709e-06
iter 2200 loss 7.702649100864669e-06
iter 2300 loss 0.000994260864336053
iter 2400 loss 0.0032113464088306116
iter 2500 loss 0.0030830098867832686
iter 2600 loss 0.0029645397869027005
iter 2700 loss 0.0032283419222573855
iter 2800 loss 0.003275012581991222
iter 2900 loss 0.003162626176489003
iter 3000 loss 0.003057293873120587
iter 3100 loss 0.0029587449954668723
iter 3200 loss 0.002866385983469181
iter 3300 loss 0.0027796271929891344
iter 3400 loss 0.002697942174405672
iter 3500 loss 0.0026209236102437443
iter 3600 loss 0.0025481911196664
iter 3700 loss 0.0024795235895347417
iter 3800 loss 0.0024144738364391045
iter 3900 loss 0.0023526287552866174
iter 4000 loss 0.0022938610023627236
iter 4100 loss 0.0022379719860878937
iter 4200 loss 0.002184739457301364
iter 4300 loss 0.002134021212607859
iter 4400 loss 0.002085617680473831
iter 4500 loss 0.002039378478425338
iter 4600 loss 0.0019951102899104897
iter 4700 loss 0.0019527035566548078
iter 4800 loss 0.0019120652315132976
epoch 1 loss 0.0018812534995967698
iter 0 loss 0.00018200158956460655
iter 100 loss 1.0405368455097475e-05
iter 200 loss 1.0875983563136429e-05
iter 300 loss 8.004931161496001e-06
iter 400 loss 6.349592091598091e-06
iter 500 loss 5.367854528739265e-06
iter 600 loss 4.852460217826137e-06
iter 700 loss 9.402032342858549e-06
iter 800 loss 8.523075762349073e-06
iter 900 loss 7.87055680350979e-06
iter 1000 loss 7.243335324653038e-06
iter 1100 loss 6.712273857246345e-06
iter 1200 loss 1.0938846601101304e-05
iter 1300 loss 1.046751327727161e-05
iter 1400 loss 9.840882398991804e-06
iter 1500 loss 9.324183505633021e-06
iter 1600 loss 8.8523129715867e-06
iter 1700 loss 8.44110185090983e-06
iter 1800 loss 8.091695979617514e-06
iter 1900 loss 7.75408583019698e-06
iter 2000 loss 7.526920988869454e-06
iter 2100 loss 7.262646175867005e-06
iter 2200 loss 7.021441019200973e-06
iter 2300 loss 0.0009971881014035407
iter 2400 loss 0.003213019546711751
iter 2500 loss 0.003084614180225486
iter 2600 loss 0.002966081123978399
iter 2700 loss 0.003240649362162713
iter 2800 loss 0.003286676074214181
iter 2900 loss 0.0031738188759007626
iter 3000 loss 0.0030681122019853156
iter 3100 loss 0.0029692136509351786
iter 3200 loss 0.0028765249362039374
iter 3300 loss 0.0027894588729166394
iter 3400 loss 0.002707485255897043
iter 3500 loss 0.0026301939601100725
iter 3600 loss 0.002557203219636297
iter 3700 loss 0.0024883134358756323
iter 3800 loss 0.0024230747592428607
iter 3900 loss 0.0023610120573219168
iter 4000 loss 0.002302035305880539
iter 4100 loss 0.0022459456787099264
iter 4200 loss 0.0021925236257386697
iter 4300 loss 0.00214161046777981
iter 4400 loss 0.002093029342302525
iter 4500 loss 0.0020466275619813686
iter 4600 loss 0.0020022007489338884
iter 4700 loss 0.0019596422251246023
iter 4800 loss 0.0019188590614624434
epoch 2 loss 0.0018863703373375902
iter 0 loss 0.0001179635728476569
iter 100 loss 7.626000896974315e-06
iter 200 loss 5.8092528725770104e-06
iter 300 loss 4.465520583892724e-06
iter 400 loss 3.676491115854113e-06
iter 500 loss 3.2182368555030502e-06
iter 600 loss 3.0372934649650883e-06
iter 700 loss 7.05096247659506e-06
iter 800 loss 6.4432603076579e-06
iter 900 loss 5.964296728477401e-06
iter 1000 loss 5.50916753228547e-06
iter 1100 loss 5.13212606214476e-06
iter 1200 loss 8.86976478652482e-06
iter 1300 loss 8.516469354367315e-06
iter 1400 loss 8.021502719568031e-06
iter 1500 loss 7.618938606605132e-06
iter 1600 loss 7.248404987654024e-06
iter 1700 loss 6.935956472934916e-06
iter 1800 loss 6.680281429562647e-06
iter 1900 loss 6.416154766254578e-06
iter 2000 loss 6.262353205310556e-06
iter 2100 loss 6.0557950753044174e-06
iter 2200 loss 5.864141330074687e-06
iter 2300 loss 0.0009943140023278949
iter 2400 loss 0.0032099857699746004
iter 2500 loss 0.0030816992680717963
iter 2600 loss 0.002963278021477498
iter 2700 loss 0.0032327652786473147
iter 2800 loss 0.003278666110784456
iter 2900 loss 0.003165878576636757
iter 3000 loss 0.0030604346707365573
iter 3100 loss 0.002961781850289708
iter 3200 loss 0.002869322088471553
iter 3300 loss 0.002782471763108975
iter 3400 loss 0.0027007020261080902
iter 3500 loss 0.0026236013103908204
iter 3600 loss 0.0025507921894417774
iter 3700 loss 0.0024820843818635155
iter 3800 loss 0.0024170142171245617
iter 3900 loss 0.0023551058700939202
iter 4000 loss 0.0022962760215864777
iter 4100 loss 0.0022403224808180107
iter 4200 loss 0.0021870329198142493
iter 4300 loss 0.0021362559510325826
iter 4400 loss 0.0020877989635914047
iter 4500 loss 0.002041509533185929
iter 4600 loss 0.001997191129772659
iter 4700 loss 0.001954738513996822
iter 4800 loss 0.0019140560659010838
epoch 3 loss 0.0018810296757917648
iter 0 loss 0.00010463698708917946
iter 100 loss 6.809682836679748e-06
iter 200 loss 7.735487805400421e-06
iter 300 loss 5.795964605936493e-06
iter 400 loss 4.660133996274231e-06
iter 500 loss 3.995267697447305e-06
iter 600 loss 3.660754800353931e-06
iter 700 loss 6.926674206810271e-06
iter 800 loss 6.319481515041271e-06
iter 900 loss 5.81747920828209e-06
iter 1000 loss 5.366002331281455e-06
iter 1100 loss 4.999238611135322e-06
iter 1200 loss 8.214808367052132e-06
iter 1300 loss 7.877836217597273e-06
iter 1400 loss 7.428597858669959e-06
iter 1500 loss 7.066317268404622e-06
iter 1600 loss 6.730071061859193e-06
iter 1700 loss 6.457790372007814e-06
iter 1800 loss 6.2440312212695355e-06
iter 1900 loss 6.003526135639495e-06
iter 2000 loss 5.880567099709857e-06
iter 2100 loss 5.692649523176794e-06
iter 2200 loss 5.515054685255952e-06
iter 2300 loss 0.0009937441692528378
iter 2400 loss 0.0032093672046162552
iter 2500 loss 0.003081104206953456
iter 2600 loss 0.0029627052606052725
iter 2700 loss 0.003231477111957276
iter 2800 loss 0.0032772299438416975
iter 2900 loss 0.0031645338327419784
iter 3000 loss 0.003059134546465234
iter 3100 loss 0.0029605240096788024
iter 3200 loss 0.0028681031283747604
iter 3300 loss 0.002781288945434127
iter 3400 loss 0.002699554843247866
iter 3500 loss 0.0026224880918800926
iter 3600 loss 0.0025497098818110906
iter 3700 loss 0.002481041235992429
iter 3800 loss 0.0024160115452631836
iter 3900 loss 0.002354129157739933
iter 4000 loss 0.00229532413527241
iter 4100 loss 0.002239394614024893
iter 4200 loss 0.002186127935690804
iter 4300 loss 0.002135354264109028
iter 4400 loss 0.0020869071541393157
iter 4500 loss 0.0020406348061082786
iter 4600 loss 0.0019963336347909623
iter 4700 loss 0.0019538984920316545
iter 4800 loss 0.001913233165409283
epoch 4 loss 0.0018800612446497088
iter 0 loss 7.576891948701814e-05
iter 100 loss 5.670717788270489e-06
iter 200 loss 4.03536984292861e-06
iter 300 loss 3.2081886997594208e-06
iter 400 loss 2.7135036119753318e-06
iter 500 loss 2.4360281860188156e-06
iter 600 loss 2.3523098613778877e-06
iter 700 loss 5.238951000375033e-06
iter 800 loss 4.828809207428004e-06
iter 900 loss 4.469274697261462e-06
iter 1000 loss 4.144872118363505e-06
iter 1100 loss 3.887895225337877e-06
iter 1200 loss 6.6999574489570685e-06
iter 1300 loss 6.450536785624891e-06
iter 1400 loss 6.097223551726871e-06
iter 1500 loss 5.819010339773298e-06
iter 1600 loss 5.558002650379705e-06
iter 1700 loss 5.3659888912157685e-06
iter 1800 loss 5.229444073536768e-06
iter 1900 loss 5.041340221208217e-06
iter 2000 loss 4.970422072409033e-06
iter 2100 loss 4.823391864056313e-06
iter 2200 loss 4.6817624099800105e-06
iter 2300 loss 0.0009924774158798003
iter 2400 loss 0.003208181923097644
iter 2500 loss 0.0030799649596757513
iter 2600 loss 0.0029616099339825983
iter 2700 loss 0.0032292776012247475
iter 2800 loss 0.0032749423082916364
iter 2900 loss 0.0031621896895903732
iter 3000 loss 0.0030568674608589287
iter 3100 loss 0.002958328737562658
iter 3200 loss 0.002865974044447477
iter 3300 loss 0.0027792215719494476
iter 3400 loss 0.002697546309732752
iter 3500 loss 0.002620534207055726
iter 3600 loss 0.0025478109243457113
iter 3700 loss 0.0024791913904698985
iter 3800 loss 0.0024142087007898608
iter 3900 loss 0.0023523724018163503
iter 4000 loss 0.0022936105258729274
iter 4100 loss 0.0022377195562115967
iter 4200 loss 0.0021844914170343578
iter 4300 loss 0.0021337771347730975
iter 4400 loss 0.0020853725573988196
iter 4500 loss 0.002039132900881446
iter 4600 loss 0.0019948637215777276
iter 4700 loss 0.0019524601555697065
iter 4800 loss 0.0019118240249220665
epoch 5 loss 0.001878487161711998
iter 0 loss 7.63765347073786e-05
iter 100 loss 5.619726651518546e-06
iter 200 loss 7.601248477333489e-06
iter 300 loss 5.678730305659348e-06
iter 400 loss 4.563934442982222e-06
iter 500 loss 3.9139086129035786e-06
iter 600 loss 3.5701172705913464e-06
iter 700 loss 5.8751725086125724e-06
iter 800 loss 5.376152427650257e-06
iter 900 loss 4.94359992686778e-06
iter 1000 loss 4.568012521750924e-06
iter 1100 loss 4.272665843740309e-06
iter 1200 loss 6.699283093375978e-06
iter 1300 loss 6.429176451344311e-06
iter 1400 loss 6.083733386277409e-06
iter 1500 loss 5.811028148768004e-06
iter 1600 loss 5.552050612631894e-06
iter 1700 loss 5.375979368301785e-06
iter 1800 loss 5.260461015493801e-06
iter 1900 loss 5.0733005910454614e-06
iter 2000 loss 5.005089549782476e-06
iter 2100 loss 4.859164157716856e-06
iter 2200 loss 4.716431521935449e-06
iter 2300 loss 0.0009924190393784684
iter 2400 loss 0.003208234342098557
iter 2500 loss 0.0030800154737813854
iter 2600 loss 0.00296165874981313
iter 2700 loss 0.00322865811688837
iter 2800 loss 0.003274250877176806
iter 2900 loss 0.003161583012341249
iter 3000 loss 0.003056281530432453
iter 3100 loss 0.002957762833363393
iter 3200 loss 0.002865426926069892
iter 3300 loss 0.002778691519673138
iter 3400 loss 0.0026970338898498107
iter 3500 loss 0.002620038562578
iter 3600 loss 0.0025473274887477873
iter 3700 loss 0.002478720633098026
iter 3800 loss 0.0024137529987248766
iter 3900 loss 0.002351928021082819
iter 4000 loss 0.002293178037951682
iter 4100 loss 0.0022372990119687336
iter 4200 loss 0.0021840820700676114
iter 4300 loss 0.002133350848063793
iter 4400 loss 0.002084944596322347
iter 4500 loss 0.00203871217503567
iter 4600 loss 0.0019944507344575252
iter 4700 loss 0.001952055452916952
iter 4800 loss 0.0019114279512416875
epoch 6 loss 0.0018782003213851375
iter 0 loss 5.897195660509169e-05
iter 100 loss 4.948497846298508e-06
iter 200 loss 3.492805255441838e-06
iter 300 loss 2.839368110387092e-06
iter 400 loss 2.431149270837194e-06
iter 500 loss 2.2075559138171093e-06
iter 600 loss 2.145337215946962e-06
iter 700 loss 4.288705028788102e-06
iter 800 loss 3.980142923666356e-06
iter 900 loss 3.6933865490718986e-06
iter 1000 loss 3.440100737851037e-06
iter 1100 loss 3.2468380217130715e-06
iter 1200 loss 5.441301913455479e-06
iter 1300 loss 5.250678706057564e-06
iter 1400 loss 4.981135196662597e-06
iter 1500 loss 4.777520768577238e-06
iter 1600 loss 4.5811693297839e-06
iter 1700 loss 4.475552780913142e-06
iter 1800 loss 4.428213140089106e-06
iter 1900 loss 4.282760647808311e-06
iter 2000 loss 4.251382374329888e-06
iter 2100 loss 4.138105075787069e-06
iter 2200 loss 4.025174946478031e-06
iter 2300 loss 0.0009913281876536304
iter 2400 loss 0.003207269167803055
iter 2500 loss 0.003079087948164369
iter 2600 loss 0.002960767406948363
iter 2700 loss 0.0032271000502701627
iter 2800 loss 0.003272753033310668
iter 2900 loss 0.0031600515645952097
iter 3000 loss 0.0030548005807777443
iter 3100 loss 0.00295632867981736
iter 3200 loss 0.002864035027774777
iter 3300 loss 0.0027773387775865462
iter 3400 loss 0.002695718605115815
iter 3500 loss 0.0026187587863931434
iter 3600 loss 0.0025460860458982946
iter 3700 loss 0.0024775136301663173
iter 3800 loss 0.00241257613618278
iter 3900 loss 0.002350781938369949
iter 4000 loss 0.0022920600417259745
iter 4100 loss 0.0022362064451003135
iter 4200 loss 0.002183014492064243
iter 4300 loss 0.002132321864633685
iter 4400 loss 0.0020839420885146423
iter 4500 loss 0.002037731581205597
iter 4600 loss 0.001993491934910829
iter 4700 loss 0.001951117493382162
iter 4800 loss 0.001910509047576614
epoch 7 loss 0.0018770844372955006
iter 0 loss 6.2622653786093e-05
iter 100 loss 5.021515009799926e-06
iter 200 loss 5.063259780499222e-06
iter 300 loss 3.90806442653208e-06
iter 400 loss 3.2338208336937203e-06
iter 500 loss 2.8489818758774158e-06
iter 600 loss 2.6736639245748617e-06
iter 700 loss 4.53468081668533e-06
iter 800 loss 4.191175074080324e-06
iter 900 loss 3.876612400178927e-06
iter 1000 loss 3.6029587088298693e-06
iter 1100 loss 3.3951807676224793e-06
iter 1200 loss 5.405705450036258e-06
iter 1300 loss 5.207154573029183e-06
iter 1400 loss 4.948251763785626e-06
iter 1500 loss 4.7515137057756046e-06
iter 1600 loss 4.558512004590282e-06
iter 1700 loss 4.4708851716957236e-06
iter 1800 loss 4.442638844432186e-06
iter 1900 loss 4.298214609306959e-06
iter 2000 loss 4.268489673095877e-06
iter 2100 loss 4.156707085745596e-06
iter 2200 loss 4.0436929537435485e-06
iter 2300 loss 0.000991440169029002
iter 2400 loss 0.00320748103569945
iter 2500 loss 0.003079291440430847
iter 2600 loss 0.0029609628167148283
iter 2700 loss 0.0032270832865124344
iter 2800 loss 0.003272663063657584
iter 2900 loss 0.0031599956267796374
iter 3000 loss 0.003054746791513615
iter 3100 loss 0.0029562773258002342
iter 3200 loss 0.002863986492821013
iter 3300 loss 0.002777292649064525
iter 3400 loss 0.002695675531251723
iter 3500 loss 0.0026187183889153772
iter 3600 loss 0.002546044267249405
iter 3700 loss 0.0024774685201448703
iter 3800 loss 0.0024125356939931767
iter 3900 loss 0.00235074199307451
iter 4000 loss 0.002292021578280314
iter 4100 loss 0.0022361695994832943
iter 4200 loss 0.0021829791497878013
iter 4300 loss 0.002132271738673517
iter 4400 loss 0.002083885353069423
iter 4500 loss 0.0020376735501190335
iter 4600 loss 0.0019934335288689284
iter 4700 loss 0.0019510597833694073
iter 4800 loss 0.0019104525417433231
epoch 8 loss 0.0018772146492623625
iter 0 loss 4.934153548674658e-05
iter 100 loss 4.52044542334358e-06
iter 200 loss 3.1822879453835384e-06
iter 300 loss 2.5906851697284364e-06
iter 400 loss 2.2421618458659276e-06
iter 500 loss 2.055050622529707e-06
iter 600 loss 2.004963620358388e-06
iter 700 loss 3.7276160442686656e-06
iter 800 loss 3.479512613378798e-06
iter 900 loss 3.2384638795805673e-06
iter 1000 loss 3.027120118287047e-06
iter 1100 loss 2.8714405827038465e-06
iter 1200 loss 4.705251741180109e-06
iter 1300 loss 4.54953488778155e-06
iter 1400 loss 4.329097378571459e-06
iter 1500 loss 4.167371966750755e-06
iter 1600 loss 4.008046103623657e-06
iter 1700 loss 3.963738638184383e-06
iter 1800 loss 3.980879277022693e-06
iter 1900 loss 3.8586708718142815e-06
iter 2000 loss 3.847423848238522e-06
iter 2100 loss 3.7526292437900253e-06
iter 2200 loss 3.6552761233987487e-06
iter 2300 loss 0.000990587006529286
iter 2400 loss 0.003206718328562722
iter 2500 loss 0.003078558226569558
iter 2600 loss 0.0029602578816972495
iter 2700 loss 0.0032258078969187765
iter 2800 loss 0.003271464664955252
iter 2900 loss 0.003158784227578333
iter 3000 loss 0.0030535752076737022
iter 3100 loss 0.0029551426091976624
iter 3200 loss 0.0028628852919718962
iter 3300 loss 0.002776222476021791
iter 3400 loss 0.002694634742923967
iter 3500 loss 0.002617705764469291
iter 3600 loss 0.0025450630706157374
iter 3700 loss 0.0024765157041950216
iter 3800 loss 0.0024116052430605108
iter 3900 loss 0.0023498358210451366
iter 4000 loss 0.002291137568163542
iter 4100 loss 0.00223530570373743
iter 4200 loss 0.0021821349459657745
iter 4300 loss 0.002131463548417273
iter 4400 loss 0.002083099881288491
iter 4500 loss 0.0020369066754160354
iter 4600 loss 0.0019926846002856456
iter 4700 loss 0.0019503274734904465
iter 4800 loss 0.0019097350248194053
epoch 9 loss 0.0018762954133448955
--- Result:
Test failed! Accuracy = 0.13856206834316254<0.3
Test G: 0/20
--- Time taken: 321.08459424972534


--- TEST: batch_size= 128  num_epochs= 10  lr= 0.0001 

iter 0 loss 5.260531906969845e-05
iter 100 loss 6.182177259850703e-06
iter 200 loss 2.5172483807879117e-05
iter 300 loss 2.3428630468068428e-05
iter 400 loss 1.8020316389240243e-05
iter 500 loss 1.4786477993561175e-05
iter 600 loss 1.2700175729281086e-05
iter 700 loss 1.752710036884343e-05
iter 800 loss 1.6146838048245103e-05
iter 900 loss 1.45494213914972e-05
iter 1000 loss 1.3252790987924793e-05
iter 1100 loss 1.2199836590308132e-05
iter 1200 loss 1.2606361529900918e-05
iter 1300 loss 1.2205585642216414e-05
iter 1400 loss 1.1465761677228445e-05
iter 1500 loss 1.0835578888202228e-05
iter 1600 loss 1.0268347542622587e-05
iter 1700 loss 9.826079875957598e-06
iter 1800 loss 9.44307214202577e-06
iter 1900 loss 9.031197249184268e-06
iter 2000 loss 8.729299434444097e-06
iter 2100 loss 8.406378091807786e-06
iter 2200 loss 8.099346552993169e-06
iter 2300 loss 0.0010037598726701771
iter 2400 loss 0.003220955990267014
iter 2500 loss 0.0030922286285733724
iter 2600 loss 0.0029733999326465705
iter 2700 loss 0.0032766351802004734
iter 2800 loss 0.0033214476629300437
iter 2900 loss 0.0032070626777277434
iter 3000 loss 0.0031002477556185373
iter 3100 loss 0.0030003083656210392
iter 3200 loss 0.002906625844563545
iter 3300 loss 0.002818629431786489
iter 3400 loss 0.002735790267282767
iter 3500 loss 0.0026576821251837316
iter 3600 loss 0.0025839141517974245
iter 3700 loss 0.0025142071296323253
iter 3800 loss 0.0024481938125561543
iter 3900 loss 0.002385474082820144
iter 4000 loss 0.002325878860349774
iter 4100 loss 0.002269190159257056
iter 4200 loss 0.002215204635320149
iter 4300 loss 0.0021637318341361186
iter 4400 loss 0.0021146101504145903
iter 4500 loss 0.002067684519011833
iter 4600 loss 0.002022776246187384
iter 4700 loss 0.0019797706308687405
iter 4800 loss 0.0019385546368629397
epoch 0 loss 0.001905113626252957
iter 0 loss 4.7935001930454746e-05
iter 100 loss 5.123714417284573e-06
iter 200 loss 5.017614400335944e-06
iter 300 loss 4.319364643303691e-06
iter 400 loss 3.6124084928311627e-06
iter 500 loss 3.2042556538730414e-06
iter 600 loss 2.9783703542438853e-06
iter 700 loss 4.392970233146871e-06
iter 800 loss 4.182541797530299e-06
iter 900 loss 3.881553735743599e-06
iter 1000 loss 3.6315546644069222e-06
iter 1100 loss 3.4316557846830916e-06
iter 1200 loss 4.976527290431043e-06
iter 1300 loss 5.064350489803982e-06
iter 1400 loss 4.819361581847327e-06
iter 1500 loss 4.6125127381923406e-06
iter 1600 loss 4.419695738218136e-06
iter 1700 loss 4.285303642781353e-06
iter 1800 loss 4.187259223181476e-06
iter 1900 loss 4.047541748453576e-06
iter 2000 loss 3.979116166536469e-06
iter 2100 loss 3.8795047525991345e-06
iter 2200 loss 3.772236800882929e-06
iter 2300 loss 0.0009703799475229093
iter 2400 loss 0.003186111471335144
iter 2500 loss 0.003058775626873601
iter 2600 loss 0.002941231866698595
iter 2700 loss 0.0031854395011409464
iter 2800 loss 0.0032332602026514824
iter 2900 loss 0.003121921221127037
iter 3000 loss 0.003017941042427358
iter 3100 loss 0.002920655593820698
iter 3200 loss 0.0028294603989037124
iter 3300 loss 0.002743800509322593
iter 3400 loss 0.00266316142376439
iter 3500 loss 0.002587127752481392
iter 3600 loss 0.0025153200622130482
iter 3700 loss 0.002447479745579257
iter 3800 loss 0.0023832264255141985
iter 3900 loss 0.002322173520802261
iter 4000 loss 0.0022641616007309196
iter 4100 loss 0.002208979003191769
iter 4200 loss 0.0021564276042142587
iter 4300 loss 0.002106321598214169
iter 4400 loss 0.002058505516197362
iter 4500 loss 0.002012828759428147
iter 4600 loss 0.001969114077663395
iter 4700 loss 0.0019272513454050002
iter 4800 loss 0.0018871309101816762
epoch 1 loss 0.0018544253743750178
iter 0 loss 4.231965431245044e-05
iter 100 loss 4.517516182915804e-06
iter 200 loss 3.3316853646351435e-06
iter 300 loss 2.8197378490636595e-06
iter 400 loss 2.4578616547186632e-06
iter 500 loss 2.258681159540441e-06
iter 600 loss 2.169800036055991e-06
iter 700 loss 3.5856573626416126e-06
iter 800 loss 3.4518580875672877e-06
iter 900 loss 3.2193740417994896e-06
iter 1000 loss 3.0274807846121355e-06
iter 1100 loss 2.875286570734872e-06
iter 1200 loss 4.266003285642383e-06
iter 1300 loss 4.357402725932937e-06
iter 1400 loss 4.155876208665161e-06
iter 1500 loss 3.989031563779802e-06
iter 1600 loss 3.832116607742815e-06
iter 1700 loss 3.7274174515114753e-06
iter 1800 loss 3.6620566705319488e-06
iter 1900 loss 3.5486044754667122e-06
iter 2000 loss 3.503890755494866e-06
iter 2100 loss 3.4233144645595456e-06
iter 2200 loss 3.33405437952771e-06
iter 2300 loss 0.0009684868880584916
iter 2400 loss 0.003182799665583714
iter 2500 loss 0.0030555942001205777
iter 2600 loss 0.0029381716469138474
iter 2700 loss 0.003185358525083053
iter 2800 loss 0.003233157967295888
iter 2900 loss 0.0031218110954524224
iter 3000 loss 0.0030178334319481736
iter 3100 loss 0.002920550874493685
iter 3200 loss 0.002829358805850878
iter 3300 loss 0.002743701722219452
iter 3400 loss 0.002663065363316934
iter 3500 loss 0.0025870344044463126
iter 3600 loss 0.002515230035884476
iter 3700 loss 0.002447401805591269
iter 3800 loss 0.0023831579852921212
iter 3900 loss 0.0023221081789788186
iter 4000 loss 0.002264098587102791
iter 4100 loss 0.002208917900364728
iter 4200 loss 0.0021563683881674536
iter 4300 loss 0.002106264342746881
iter 4400 loss 0.00205845017262786
iter 4500 loss 0.0020127755186311208
iter 4600 loss 0.0019690629295962404
iter 4700 loss 0.001927202117430438
iter 4800 loss 0.001887083533563898
epoch 2 loss 0.0018542840633222434
iter 0 loss 3.897029091604054e-05
iter 100 loss 4.1865722190741865e-06
iter 200 loss 3.0972351739242796e-06
iter 300 loss 2.6096414493988046e-06
iter 400 loss 2.283292401625205e-06
iter 500 loss 2.106542179724769e-06
iter 600 loss 2.0249240412360115e-06
iter 700 loss 3.345133633662205e-06
iter 800 loss 3.2255297161854253e-06
iter 900 loss 3.010883297373323e-06
iter 1000 loss 2.8344359823715385e-06
iter 1100 loss 2.6961092856342642e-06
iter 1200 loss 3.952576363761009e-06
iter 1300 loss 4.0336144034052626e-06
iter 1400 loss 3.851532341404157e-06
iter 1500 loss 3.702118933421718e-06
iter 1600 loss 3.56132674558746e-06
iter 1700 loss 3.472641583197067e-06
iter 1800 loss 3.423031986372701e-06
iter 1900 loss 3.32126373900555e-06
iter 2000 loss 3.287368679099265e-06
iter 2100 loss 3.2158098919639643e-06
iter 2200 loss 3.1342575072446383e-06
iter 2300 loss 0.0009674309402107385
iter 2400 loss 0.003180769495435572
iter 2500 loss 0.003053644079351866
iter 2600 loss 0.0029362961363732574
iter 2700 loss 0.0031830607970149125
iter 2800 loss 0.0032308970576611076
iter 2900 loss 0.0031196200130762415
iter 3000 loss 0.0030157145432771096
iter 3100 loss 0.0029185001177117454
iter 3200 loss 0.002827372038679044
iter 3300 loss 0.002741775119023462
iter 3400 loss 0.002661195611719771
iter 3500 loss 0.0025852182689515106
iter 3600 loss 0.00251346470024375
iter 3700 loss 0.002445687130556692
iter 3800 loss 0.002381493168371341
iter 3900 loss 0.002320486936063703
iter 4000 loss 0.0022625183572104256
iter 4100 loss 0.0022073765114376887
iter 4200 loss 0.002154864041620504
iter 4300 loss 0.002104795244190081
iter 4400 loss 0.0020570146034303394
iter 4500 loss 0.002011372390349952
iter 4600 loss 0.0019676908460267506
iter 4700 loss 0.0019258596512164265
iter 4800 loss 0.0018857695283060394
epoch 3 loss 0.0018529119800143078
iter 0 loss 3.6709032428916544e-05
iter 100 loss 3.974236465588133e-06
iter 200 loss 2.8928384709339767e-06
iter 300 loss 2.4381349990666983e-06
iter 400 loss 2.1430732242510217e-06
iter 500 loss 1.9859457076605313e-06
iter 600 loss 1.913853134056277e-06
iter 700 loss 3.1250315511343495e-06
iter 800 loss 3.018167626266112e-06
iter 900 loss 2.8210699266793775e-06
iter 1000 loss 2.659581522099355e-06
iter 1100 loss 2.5344234906137996e-06
iter 1200 loss 3.695669125022221e-06
iter 1300 loss 3.7700558459688968e-06
iter 1400 loss 3.604430070887996e-06
iter 1500 loss 3.469550762455596e-06
iter 1600 loss 3.341916239483927e-06
iter 1700 loss 3.266623134018164e-06
iter 1800 loss 3.230309862182372e-06
iter 1900 loss 3.138105860870708e-06
iter 2000 loss 3.1129154224557746e-06
iter 2100 loss 3.0487645583186812e-06
iter 2200 loss 2.9736698114117837e-06
iter 2300 loss 0.0009668267451014784
iter 2400 loss 0.003179408596523263
iter 2500 loss 0.0030523368733322083
iter 2600 loss 0.0029350388254610034
iter 2700 loss 0.0031817719987796743
iter 2800 loss 0.003229617590196421
iter 2900 loss 0.003118377926749014
iter 3000 loss 0.003014513369980211
iter 3100 loss 0.0029173375166672406
iter 3200 loss 0.0028262458006274136
iter 3300 loss 0.002740682985518347
iter 3400 loss 0.002660135675781206
iter 3500 loss 0.0025841886295282926
iter 3600 loss 0.0025124638251513916
iter 3700 loss 0.00244471520082405
iter 3800 loss 0.0023805501618030323
iter 3900 loss 0.0023195687169235224
iter 4000 loss 0.0022616234718848913
iter 4100 loss 0.002206503542642483
iter 4200 loss 0.0021540121397166825
iter 4300 loss 0.002103963394687946
iter 4400 loss 0.002056201785236715
iter 4500 loss 0.002010577952732518
iter 4600 loss 0.0019669139845373785
iter 4700 loss 0.0019250996182895168
iter 4800 loss 0.0018850257466592908
epoch 4 loss 0.0018521132477244565
iter 0 loss 3.502917752484791e-05
iter 100 loss 3.820327202168364e-06
iter 200 loss 2.763507437471411e-06
iter 300 loss 2.3288099396796857e-06
iter 400 loss 2.052278689671469e-06
iter 500 loss 1.9072769454325821e-06
iter 600 loss 1.8400812530462993e-06
iter 700 loss 2.9544730254520735e-06
iter 800 loss 2.8570013108300955e-06
iter 900 loss 2.67408857012363e-06
iter 1000 loss 2.524512919822141e-06
iter 1100 loss 2.4095054456255268e-06
iter 1200 loss 3.4907559058045688e-06
iter 1300 loss 3.5589660823563664e-06
iter 1400 loss 3.406922992442996e-06
iter 1500 loss 3.2837594658512758e-06
iter 1600 loss 3.166495834824619e-06
iter 1700 loss 3.102053549116062e-06
iter 1800 loss 3.076640544626316e-06
iter 1900 loss 2.9919979003535875e-06
iter 2000 loss 2.973491106310359e-06
iter 2100 loss 2.915291471699463e-06
iter 2200 loss 2.8453042209856433e-06
iter 2300 loss 0.0009663529332022015
iter 2400 loss 0.0031783077901228235
iter 2500 loss 0.0030512794876387476
iter 2600 loss 0.0029340218801229517
iter 2700 loss 0.003180669939235837
iter 2800 loss 0.0032285227372322615
iter 2900 loss 0.003117315549377904
iter 3000 loss 0.0030134860340212083
iter 3100 loss 0.0029163431846533946
iter 3200 loss 0.0028252824980328586
iter 3300 loss 0.002739748718079616
iter 3400 loss 0.0026592290030391895
iter 3500 loss 0.002583307877465807
iter 3600 loss 0.0025116076459242615
iter 3700 loss 0.0024438832872096116
iter 3800 loss 0.0023797426442698287
iter 3900 loss 0.0023187822598672997
iter 4000 loss 0.0022608569418214965
iter 4100 loss 0.002205755724087427
iter 4200 loss 0.0021532822917200165
iter 4300 loss 0.0021032506907001387
iter 4400 loss 0.002055505230739276
iter 4500 loss 0.002009896907520162
iter 4600 loss 0.001966247897707093
iter 4700 loss 0.0019244479317012243
iter 4800 loss 0.001884387944617049
epoch 5 loss 0.0018514283865675965
iter 0 loss 3.366566670592874e-05
iter 100 loss 3.7028312362963334e-06
iter 200 loss 2.6579173457895958e-06
iter 300 loss 2.239515452611974e-06
iter 400 loss 1.9789303180567418e-06
iter 500 loss 1.8440424859719511e-06
iter 600 loss 1.7808491437701224e-06
iter 700 loss 2.8126747047047707e-06
iter 800 loss 2.7229085774907593e-06
iter 900 loss 2.5521568915204673e-06
iter 1000 loss 2.4120482844314305e-06
iter 1100 loss 2.3054952919158706e-06
iter 1200 loss 3.3172705371740286e-06
iter 1300 loss 3.380493484065726e-06
iter 1400 loss 3.239813962393988e-06
iter 1500 loss 3.1264099032391304e-06
iter 1600 loss 3.017982657156057e-06
iter 1700 loss 2.9629369189239625e-06
iter 1800 loss 2.946980924362343e-06
iter 1900 loss 2.8686572095308663e-06
iter 2000 loss 2.8558354322786705e-06
iter 2100 loss 2.8026985315982777e-06
iter 2200 loss 2.737140917003462e-06
iter 2300 loss 0.0009659553058505877
iter 2400 loss 0.003177367744336363
iter 2500 loss 0.003050376418692505
iter 2600 loss 0.002933153341112292
iter 2700 loss 0.0031797366954106126
iter 2800 loss 0.003227594400658512
iter 2900 loss 0.003116415051724987
iter 3000 loss 0.003012615303688018
iter 3100 loss 0.0029155003993905767
iter 3200 loss 0.0028244659438244946
iter 3300 loss 0.0027389568245509564
iter 3400 loss 0.002658460453070405
iter 3500 loss 0.002582561342309174
iter 3600 loss 0.002510881936480711
iter 3700 loss 0.002443177943230317
iter 3800 loss 0.0023790578711553635
iter 3900 loss 0.002318115294992881
iter 4000 loss 0.002260206760028449
iter 4100 loss 0.0022051214126449937
iter 4200 loss 0.0021526632451602155
iter 4300 loss 0.0021026461818717387
iter 4400 loss 0.0020549143643071397
iter 4500 loss 0.0020093192055794947
iter 4600 loss 0.0019656828151344914
iter 4700 loss 0.0019238950484996126
iter 4800 loss 0.0018838467876956088
epoch 6 loss 0.0018508465467644824
iter 0 loss 3.248732537031174e-05
iter 100 loss 3.6052479680449217e-06
iter 200 loss 2.5746106831180232e-06
iter 300 loss 2.1698011631124123e-06
iter 400 loss 1.922007240161231e-06
iter 500 loss 1.7944400005563547e-06
iter 600 loss 1.7340732607429612e-06
iter 700 loss 2.6948144181428733e-06
iter 800 loss 2.6110601683445592e-06
iter 900 loss 2.4503376612801174e-06
iter 1000 loss 2.318152985219774e-06
iter 1100 loss 2.2189278624154113e-06
iter 1200 loss 3.1698178022781755e-06
iter 1300 loss 3.2287598916780297e-06
iter 1400 loss 3.097543349015085e-06
iter 1500 loss 2.9924636109863204e-06
iter 1600 loss 2.8916314293043325e-06
iter 1700 loss 2.8447932410995465e-06
iter 1800 loss 2.8371343961411993e-06
iter 1900 loss 2.764117250866804e-06
iter 2000 loss 2.7560944419582567e-06
iter 2100 loss 2.707335157176289e-06
iter 2200 loss 2.6454806485145467e-06
iter 2300 loss 0.0009656084661376802
iter 2400 loss 0.003176539968963834
iter 2500 loss 0.0030495813035510844
iter 2600 loss 0.002932388593259135
iter 2700 loss 0.00317890552558711
iter 2800 loss 0.0032267672468612512
iter 2900 loss 0.0031156129802396623
iter 3000 loss 0.0030118397271895925
iter 3100 loss 0.00291474978656974
iter 3200 loss 0.0028237386814054925
iter 3300 loss 0.0027382514529291327
iter 3400 loss 0.0026577758757925464
iter 3500 loss 0.0025818963312982113
iter 3600 loss 0.002510235458030792
iter 3700 loss 0.002442549459997297
iter 3800 loss 0.0023784475072474516
iter 3900 loss 0.0023175207631380666
iter 4000 loss 0.002259627190872495
iter 4100 loss 0.0022045559347707295
iter 4200 loss 0.00215211135752856
iter 4300 loss 0.0021021071529207474
iter 4400 loss 0.0020543874304483237
iter 4500 loss 0.002008803875929881
iter 4600 loss 0.0019651786791916965
iter 4700 loss 0.0019234017805987468
iter 4800 loss 0.0018833639296646136
epoch 7 loss 0.0018503285889032897
iter 0 loss 3.144112997688353e-05
iter 100 loss 3.522397058545586e-06
iter 200 loss 2.5088999573154295e-06
iter 300 loss 2.1149743795444694e-06
iter 400 loss 1.877450516080887e-06
iter 500 loss 1.755317382735021e-06
iter 600 loss 1.6969105730877254e-06
iter 700 loss 2.5961786468737027e-06
iter 800 loss 2.517570651485748e-06
iter 900 loss 2.365448527284414e-06
iter 1000 loss 2.240070513651701e-06
iter 1100 loss 2.146915522184831e-06
iter 1200 loss 3.043971179896205e-06
iter 1300 loss 3.09913408258452e-06
iter 1400 loss 2.9762159948838884e-06
iter 1500 loss 2.878469213785894e-06
iter 1600 loss 2.78413072785581e-06
iter 1700 loss 2.7444789416444086e-06
iter 1800 loss 2.7441846717665753e-06
iter 1900 loss 2.6756572757453434e-06
iter 2000 loss 2.671855568273688e-06
iter 2100 loss 2.6266509449924652e-06
iter 2200 loss 2.567866037848371e-06
iter 2300 loss 0.0009653010027913621
iter 2400 loss 0.0031758012728557083
iter 2500 loss 0.003048871757688293
iter 2600 loss 0.002931706212547766
iter 2700 loss 0.0031781635048467633
iter 2800 loss 0.0032260284549692608
iter 2900 loss 0.0031148967833006277
iter 3000 loss 0.003011147226084396
iter 3100 loss 0.0029140794785543884
iter 3200 loss 0.002823089214468181
iter 3300 loss 0.002737621503700165
iter 3400 loss 0.0026571645013371906
iter 3500 loss 0.0025813024178328076
iter 3600 loss 0.0025096580577427326
iter 3700 loss 0.0024419881439313733
iter 3800 loss 0.0023779021428793627
iter 3900 loss 0.002316989523538731
iter 4000 loss 0.0022591093011030427
iter 4100 loss 0.002204050632275349
iter 4200 loss 0.002151618167645038
iter 4300 loss 0.0021016254296873406
iter 4400 loss 0.0020539164640131172
iter 4500 loss 0.0020083432330174713
iter 4600 loss 0.00196472806569215
iter 4700 loss 0.0019229608696650724
iter 4800 loss 0.0018829323353418278
epoch 8 loss 0.0018498668791562722
iter 0 loss 3.052056854357943e-05
iter 100 loss 3.4518076110525905e-06
iter 200 loss 2.454243080232552e-06
iter 300 loss 2.0692957918694797e-06
iter 400 loss 1.839953412067792e-06
iter 500 loss 1.722477308332433e-06
iter 600 loss 1.6660946014519275e-06
iter 700 loss 2.511572485119241e-06
iter 800 loss 2.437396112648738e-06
iter 900 loss 2.292684961378086e-06
iter 1000 loss 2.173100436881508e-06
iter 1100 loss 2.0850062727682972e-06
iter 1200 loss 2.9342299993096806e-06
iter 1300 loss 2.9857675982929115e-06
iter 1400 loss 2.8702646183107345e-06
iter 1500 loss 2.7788648808391368e-06
iter 1600 loss 2.690248669342423e-06
iter 1700 loss 2.6571293905141478e-06
iter 1800 loss 2.6634715617433926e-06
iter 1900 loss 2.5989244566972767e-06
iter 2000 loss 2.5988671606968495e-06
iter 2100 loss 2.5568138140643755e-06
iter 2200 loss 2.500704697770564e-06
iter 2300 loss 0.0009650289208013137
iter 2400 loss 0.0031751391129939393
iter 2500 loss 0.0030482357198705996
iter 2600 loss 0.0029310945362940434
iter 2700 loss 0.003177496034052889
iter 2800 loss 0.003225363397284219
iter 2900 loss 0.003114252245641289
iter 3000 loss 0.003010524013480632
iter 3100 loss 0.0029134762669947504
iter 3200 loss 0.0028225048290519803
iter 3300 loss 0.0027370546505990777
iter 3400 loss 0.0026566143149365953
iter 3500 loss 0.002580767933784718
iter 3600 loss 0.0025091384616009677
iter 3700 loss 0.0024414830130552448
iter 3800 loss 0.0023774112959373643
iter 3900 loss 0.002316511358724741
iter 4000 loss 0.002258643147476997
iter 4100 loss 0.002203595796652247
iter 4200 loss 0.0021511742036455844
iter 4300 loss 0.002101191773091912
iter 4400 loss 0.0020534924642117572
iter 4500 loss 0.002007928496920351
iter 4600 loss 0.0019643222853442887
iter 4700 loss 0.0019225638011421645
iter 4800 loss 0.0018825436671327138
epoch 9 loss 0.0018494520344226818
--- Result:
Test failed! Accuracy = 0.1379232257604599<0.3
Test G: 0/20
--- Time taken: 321.1737070083618


--- TEST: batch_size= 256  num_epochs= 2  lr= 0.001 

iter 0 loss 1.55641209858004e-05
iter 100 loss 1.919233071556945e-06
iter 200 loss 1.3264502870729144e-06
iter 300 loss 1.1015774913082177e-06
iter 400 loss 9.862281433498694e-07
iter 500 loss 9.601481773000535e-07
iter 600 loss 9.159898914782909e-07
iter 700 loss 8.75953375307887e-07
iter 800 loss 8.509585590639445e-07
iter 900 loss 8.22018286383533e-07
iter 1000 loss 8.063080604524321e-07
iter 1100 loss 4.176739081665984e-06
iter 1200 loss 3.9227154192429425e-06
iter 1300 loss 3.695358799295627e-06
iter 1400 loss 3.4829629500692433e-06
iter 1500 loss 3.299024348389824e-06
iter 1600 loss 3.1388586040914998e-06
iter 1700 loss 3.0007082778662163e-06
iter 1800 loss 2.8729595421748393e-06
iter 1900 loss 2.7623227422582285e-06
iter 2000 loss 2.6610018603096507e-06
iter 2100 loss 2.573457098317711e-06
iter 2200 loss 0.0009082302543359993
iter 2300 loss 0.002152552524064239
iter 2400 loss 0.0031781487139507275
epoch 0 loss 0.0031222461305161664
iter 0 loss 2.181908712373115e-05
iter 100 loss 1.7752303232789406e-06
iter 200 loss 1.1613570117349275e-06
iter 300 loss 9.385642803872512e-07
iter 400 loss 8.314294292401322e-07
iter 500 loss 2.1972554733254295e-06
iter 600 loss 2.1112973814439847e-06
iter 700 loss 1.9765082935060157e-06
iter 800 loss 1.8208508291518252e-06
iter 900 loss 1.6739598658993431e-06
iter 1000 loss 1.5609701464299078e-06
iter 1100 loss 1.9556402178116016e-06
iter 1200 loss 1.9022271227961594e-06
iter 1300 loss 1.8234039005054855e-06
iter 1400 loss 1.7338403593753093e-06
iter 1500 loss 1.6562361072854384e-06
iter 1600 loss 1.5895633346860486e-06
iter 1700 loss 1.5344452289347619e-06
iter 1800 loss 1.4845556678721216e-06
iter 1900 loss 1.4385940927850401e-06
iter 2000 loss 1.396780964808039e-06
iter 2100 loss 1.3627385829738728e-06
iter 2200 loss 0.000904357537392429
iter 2300 loss 0.0021448494371716104
iter 2400 loss 0.003169260010064051
epoch 1 loss 0.003113514984358635
--- Result:
Test failed! Accuracy = 0.1369853913784027<0.3
Test G: 0/20
--- Time taken: 48.40520095825195


--- TEST: batch_size= 256  num_epochs= 2  lr= 0.0005 

iter 0 loss 1.2292925021029077e-05
iter 100 loss 1.7826507262113166e-06
iter 200 loss 1.23375559215036e-06
iter 300 loss 1.0183291588311531e-06
iter 400 loss 9.037653676829254e-07
iter 500 loss 1.114855126752173e-06
iter 600 loss 1.0610058590083431e-06
iter 700 loss 1.0017667740497052e-06
iter 800 loss 9.506172879847507e-07
iter 900 loss 9.002987060832409e-07
iter 1000 loss 8.620448294312847e-07
iter 1100 loss 1.5976510313243873e-06
iter 1200 loss 1.5529137112972995e-06
iter 1300 loss 1.4924276981270041e-06
iter 1400 loss 1.4225444507379152e-06
iter 1500 loss 1.361581555971827e-06
iter 1600 loss 1.3084788714670723e-06
iter 1700 loss 1.2629177714351242e-06
iter 1800 loss 1.2212038281725654e-06
iter 1900 loss 1.1818831127087328e-06
iter 2000 loss 1.1457890096430307e-06
iter 2100 loss 1.1148555868204485e-06
iter 2200 loss 0.0008974907793248243
iter 2300 loss 0.002137246996007416
iter 2400 loss 0.003161150845804922
epoch 0 loss 0.003105548262877736
iter 0 loss 1.0565012416918762e-05
iter 100 loss 1.4894982871395552e-06
iter 200 loss 1.0361564618936097e-06
iter 300 loss 8.571260110084856e-07
iter 400 loss 7.661392130030617e-07
iter 500 loss 1.2502139650600085e-06
iter 600 loss 1.2311684443181569e-06
iter 700 loss 1.1711723496136594e-06
iter 800 loss 1.0982640890815515e-06
iter 900 loss 1.0282937208251635e-06
iter 1000 loss 9.743470507380373e-07
iter 1100 loss 1.208271714662521e-06
iter 1200 loss 1.1791807309294455e-06
iter 1300 loss 1.1393446505673615e-06
iter 1400 loss 1.094163058728389e-06
iter 1500 loss 1.0542012621222242e-06
iter 1600 loss 1.0197929196824223e-06
iter 1700 loss 9.917423901585193e-07
iter 1800 loss 9.662018299205526e-07
iter 1900 loss 9.417328563827373e-07
iter 2000 loss 9.191559313379705e-07
iter 2100 loss 9.01014578476458e-07
iter 2200 loss 0.0008835350702949955
iter 2300 loss 0.002121790550642805
iter 2400 loss 0.0031452644930045457
epoch 1 loss 0.0030899428995748578
--- Result:
Test failed! Accuracy = 0.13993173837661743<0.3
Test G: 0/20
--- Time taken: 48.31553912162781


--- TEST: batch_size= 256  num_epochs= 2  lr= 0.0001 

iter 0 loss 9.7439396995469e-06
iter 100 loss 1.6277298621137354e-06
iter 200 loss 1.1743323392494034e-06
iter 300 loss 9.859715604674107e-07
iter 400 loss 8.844715332027289e-07
iter 500 loss 1.4866175681804261e-06
iter 600 loss 1.8539785915261581e-06
iter 700 loss 1.8398620996068633e-06
iter 800 loss 1.7159877391352435e-06
iter 900 loss 1.5820718169998467e-06
iter 1000 loss 1.4758920258884189e-06
iter 1100 loss 1.6207632222558937e-06
iter 1200 loss 1.6042748810144195e-06
iter 1300 loss 1.5480364235771862e-06
iter 1400 loss 1.4728959957007272e-06
iter 1500 loss 1.406679007277459e-06
iter 1600 loss 1.349346515857329e-06
iter 1700 loss 1.2998801987848205e-06
iter 1800 loss 1.2548925091636928e-06
iter 1900 loss 1.2131463397117808e-06
iter 2000 loss 1.1743402495440754e-06
iter 2100 loss 1.1402184956066973e-06
iter 2200 loss 0.0008865174102437415
iter 2300 loss 0.002124649591612565
iter 2400 loss 0.0031477036433883478
epoch 0 loss 0.003092337969038675
iter 0 loss 9.185685485135764e-06
iter 100 loss 1.4934613303840607e-06
iter 200 loss 1.0792881658378078e-06
iter 300 loss 9.093257910816619e-07
iter 400 loss 8.194249726658742e-07
iter 500 loss 9.420695121066555e-07
iter 600 loss 1.0691399154155096e-06
iter 700 loss 1.0607883071143324e-06
iter 800 loss 1.0095668580595954e-06
iter 900 loss 9.521308817530128e-07
iter 1000 loss 9.07543883849476e-07
iter 1100 loss 1.0678477415892785e-06
iter 1200 loss 1.0710600881539964e-06
iter 1300 loss 1.0460057034441e-06
iter 1400 loss 1.007029733053249e-06
iter 1500 loss 9.719639707364851e-07
iter 1600 loss 9.419051923707262e-07
iter 1700 loss 9.1697912756301e-07
iter 1800 loss 8.938628746102388e-07
iter 1900 loss 8.715457595118277e-07
iter 2000 loss 8.5036199588908e-07
iter 2100 loss 8.322858953795368e-07
iter 2200 loss 0.0008710022170626229
iter 2300 loss 0.002108628809342921
iter 2400 loss 0.0031320935435132807
epoch 1 loss 0.003077003210007424
--- Result:
Test failed! Accuracy = 0.14050501585006714<0.3
Test G: 0/20
--- Time taken: 48.15682578086853


--- TEST: batch_size= 256  num_epochs= 5  lr= 0.001 

iter 0 loss 8.862158210831694e-06
iter 100 loss 1.136646183608313e-06
iter 200 loss 7.833020747715528e-07
iter 300 loss 6.484136693709169e-07
iter 400 loss 5.809255303988823e-07
iter 500 loss 5.579856748679054e-07
iter 600 loss 5.298867456141622e-07
iter 700 loss 5.065835293541506e-07
iter 800 loss 4.943355368969934e-07
iter 900 loss 4.810495781160723e-07
iter 1000 loss 4.731790921470418e-07
iter 1100 loss 1.5685072362785426e-06
iter 1200 loss 1.4886290216651214e-06
iter 1300 loss 1.415799354266141e-06
iter 1400 loss 1.346211200363027e-06
iter 1500 loss 1.2859052126548815e-06
iter 1600 loss 1.2340815360553753e-06
iter 1700 loss 1.1903392025171314e-06
iter 1800 loss 1.1493836488957961e-06
iter 1900 loss 1.1121821133807764e-06
iter 2000 loss 1.0795252403179542e-06
iter 2100 loss 1.0519593923834242e-06
iter 2200 loss 0.0008957576226655332
iter 2300 loss 0.0021329172093174195
iter 2400 loss 0.003154336499138707
epoch 0 loss 0.003098857742265706
iter 0 loss 1.19815003927215e-05
iter 100 loss 1.3827612957753121e-06
iter 200 loss 9.38831399354913e-07
iter 300 loss 7.716860057332097e-07
iter 400 loss 6.883461721084397e-07
iter 500 loss 1.3589115457386312e-06
iter 600 loss 1.2731077807441271e-06
iter 700 loss 1.1913270034998904e-06
iter 800 loss 1.110074032347077e-06
iter 900 loss 1.035139971592352e-06
iter 1000 loss 9.777633082123046e-07
iter 1100 loss 1.1589913625288463e-06
iter 1200 loss 1.1278662392908803e-06
iter 1300 loss 1.088427081683468e-06
iter 1400 loss 1.0451385363826768e-06
iter 1500 loss 1.0074237455168906e-06
iter 1600 loss 9.755541149308208e-07
iter 1700 loss 9.504815710626857e-07
iter 1800 loss 9.267645365473303e-07
iter 1900 loss 9.042013444557837e-07
iter 2000 loss 8.842754598559765e-07
iter 2100 loss 8.692898986730925e-07
iter 2200 loss 0.0009021636029342176
iter 2300 loss 0.002137735642435604
iter 2400 loss 0.003158846836836265
epoch 1 loss 0.003103288352874327
iter 0 loss 8.459312994091306e-06
iter 100 loss 1.3602217664036757e-06
iter 200 loss 9.424947557790937e-07
iter 300 loss 7.830958859705842e-07
iter 400 loss 7.031384757453481e-07
iter 500 loss 8.827363576265797e-07
iter 600 loss 8.500832413900361e-07
iter 700 loss 8.134624716391738e-07
iter 800 loss 7.77290935314689e-07
iter 900 loss 7.403507154243911e-07
iter 1000 loss 7.143610690247231e-07
iter 1100 loss 8.520451162066333e-07
iter 1200 loss 8.303922732787628e-07
iter 1300 loss 8.085987235875953e-07
iter 1400 loss 7.871050789282609e-07
iter 1500 loss 7.677652199016643e-07
iter 1600 loss 7.518316966297363e-07
iter 1700 loss 7.412800663213661e-07
iter 1800 loss 7.312070020406913e-07
iter 1900 loss 7.206303989658831e-07
iter 2000 loss 7.113178105765778e-07
iter 2100 loss 7.054347776729243e-07
iter 2200 loss 0.0008981107415296988
iter 2300 loss 0.002133821145646658
iter 2400 loss 0.003155143007357159
epoch 2 loss 0.0030996504438553243
iter 0 loss 1.0214366739091929e-05
iter 100 loss 1.4432931625198745e-06
iter 200 loss 9.976416463224037e-07
iter 300 loss 8.266078774821144e-07
iter 400 loss 7.416964152432916e-07
iter 500 loss 1.027455668602846e-06
iter 600 loss 9.570225332374197e-07
iter 700 loss 9.009809755911225e-07
iter 800 loss 8.537792541233528e-07
iter 900 loss 8.090573002513606e-07
iter 1000 loss 7.779436910967743e-07
iter 1100 loss 1.00204851678361e-06
iter 1200 loss 9.693500864186695e-07
iter 1300 loss 9.377254053443252e-07
iter 1400 loss 9.082268578410973e-07
iter 1500 loss 8.81769700049078e-07
iter 1600 loss 8.595989680071352e-07
iter 1700 loss 8.43858853261221e-07
iter 1800 loss 8.295902764945541e-07
iter 1900 loss 8.154214784496025e-07
iter 2000 loss 8.024151945168453e-07
iter 2100 loss 7.927855894567806e-07
iter 2200 loss 0.0008980911507875079
iter 2300 loss 0.002133989619547306
iter 2400 loss 0.0031554895628213265
epoch 3 loss 0.0030999916854498747
iter 0 loss 8.370006980840117e-06
iter 100 loss 1.4413325013067313e-06
iter 200 loss 1.003196543866009e-06
iter 300 loss 8.346969819971631e-07
iter 400 loss 7.520625871464227e-07
iter 500 loss 9.9719959952401e-07
iter 600 loss 9.405885460775831e-07
iter 700 loss 8.919318810617868e-07
iter 800 loss 8.479661059726858e-07
iter 900 loss 8.051814065006344e-07
iter 1000 loss 7.760380594039258e-07
iter 1100 loss 9.102597533461361e-07
iter 1200 loss 8.862059109182243e-07
iter 1300 loss 8.617590999833224e-07
iter 1400 loss 8.387408024751483e-07
iter 1500 loss 8.176243284651415e-07
iter 1600 loss 8.006462881852796e-07
iter 1700 loss 7.893686878963285e-07
iter 1800 loss 7.793856072925795e-07
iter 1900 loss 7.691445224020556e-07
iter 2000 loss 7.594483420227793e-07
iter 2100 loss 7.522964397348598e-07
iter 2200 loss 0.0008978659465053891
iter 2300 loss 0.00213402554545692
iter 2400 loss 0.003155741913876384
epoch 4 loss 0.00310024024670539
--- Result:
Test failed! Accuracy = 0.14195819199085236<0.3
Test G: 0/20
--- Time taken: 114.10204815864563


--- TEST: batch_size= 256  num_epochs= 5  lr= 0.0005 

iter 0 loss 1.0260156159347389e-05
iter 100 loss 1.865658461075228e-06
iter 200 loss 1.30957733740638e-06
iter 300 loss 1.0845505250526731e-06
iter 400 loss 9.660292625552498e-07
iter 500 loss 1.0559506645739806e-06
iter 600 loss 1.0033004896217903e-06
iter 700 loss 9.50928320554559e-07
iter 800 loss 9.03038810040502e-07
iter 900 loss 8.582675411452915e-07
iter 1000 loss 8.256211941869452e-07
iter 1100 loss 9.526114973467257e-07
iter 1200 loss 9.284303873800999e-07
iter 1300 loss 9.009724455153306e-07
iter 1400 loss 8.720421473764021e-07
iter 1500 loss 8.46545972621186e-07
iter 1600 loss 8.239273956600519e-07
iter 1700 loss 8.046584056048552e-07
iter 1800 loss 7.866790907806608e-07
iter 1900 loss 7.692928302906745e-07
iter 2000 loss 7.53251505809418e-07
iter 2100 loss 7.395217961975665e-07
iter 2200 loss 0.0008949346689614409
iter 2300 loss 0.0021312413697387577
iter 2400 loss 0.0031528905615483905
epoch 0 loss 0.003097437268664817
iter 0 loss 8.550302482035477e-06
iter 100 loss 1.503818846276322e-06
iter 200 loss 1.0637418270322377e-06
iter 300 loss 8.877576486615893e-07
iter 400 loss 7.98129402293644e-07
iter 500 loss 9.259787420542814e-07
iter 600 loss 8.894306436146287e-07
iter 700 loss 8.502592412179013e-07
iter 800 loss 8.109083078397476e-07
iter 900 loss 7.730087128840063e-07
iter 1000 loss 7.457080271053853e-07
iter 1100 loss 8.165810997758889e-07
iter 1200 loss 8.005769551886719e-07
iter 1300 loss 7.818618394602912e-07
iter 1400 loss 7.616706878997048e-07
iter 1500 loss 7.432602898878662e-07
iter 1600 loss 7.275761487701031e-07
iter 1700 loss 7.151352883056862e-07
iter 1800 loss 7.035968721401307e-07
iter 1900 loss 6.924008348982829e-07
iter 2000 loss 6.817574164924709e-07
iter 2100 loss 6.736088222552954e-07
iter 2200 loss 0.0008813689941695018
iter 2300 loss 0.0021170739331652817
iter 2400 loss 0.003138746736534895
epoch 1 loss 0.0030835436659438047
iter 0 loss 7.15659189154394e-06
iter 100 loss 1.3917217163700369e-06
iter 200 loss 9.965930082637611e-07
iter 300 loss 8.381308426968255e-07
iter 400 loss 7.568209518655733e-07
iter 500 loss 8.215126102099194e-07
iter 600 loss 7.92609919702343e-07
iter 700 loss 7.622676451004147e-07
iter 800 loss 7.326235612488565e-07
iter 900 loss 7.026580506165643e-07
iter 1000 loss 6.823398136675933e-07
iter 1100 loss 7.472117369765276e-07
iter 1200 loss 7.354071020861717e-07
iter 1300 loss 7.215115105043432e-07
iter 1400 loss 7.062768206680739e-07
iter 1500 loss 6.922124105506607e-07
iter 1600 loss 6.805596698517618e-07
iter 1700 loss 6.72232480571648e-07
iter 1800 loss 6.648801797104981e-07
iter 1900 loss 6.573161884532009e-07
iter 2000 loss 6.496767312164511e-07
iter 2100 loss 6.443684867787827e-07
iter 2200 loss 0.0008822566231390366
iter 2300 loss 0.0021175304296279646
iter 2400 loss 0.003138968630490674
epoch 2 loss 0.0030837624421983707
iter 0 loss 6.894963917147834e-06
iter 100 loss 1.3652264388469174e-06
iter 200 loss 9.782660754919462e-07
iter 300 loss 8.242867566391502e-07
iter 400 loss 7.458811463020583e-07
iter 500 loss 8.071233834559104e-07
iter 600 loss 7.783029526023125e-07
iter 700 loss 7.489155329959157e-07
iter 800 loss 7.205087647054618e-07
iter 900 loss 6.915436421252189e-07
iter 1000 loss 6.72399002703989e-07
iter 1100 loss 7.333056931686908e-07
iter 1200 loss 7.227473393973684e-07
iter 1300 loss 7.100012873838299e-07
iter 1400 loss 6.960774292501855e-07
iter 1500 loss 6.832925178023313e-07
iter 1600 loss 6.726910871266813e-07
iter 1700 loss 6.654906169540071e-07
iter 1800 loss 6.595479166657901e-07
iter 1900 loss 6.532824584230958e-07
iter 2000 loss 6.468710888173852e-07
iter 2100 loss 6.422839440099117e-07
iter 2200 loss 0.0008818595379093044
iter 2300 loss 0.0021169985963504882
iter 2400 loss 0.003138373384643754
epoch 3 loss 0.003083178216759859
iter 0 loss 6.684996606054483e-06
iter 100 loss 1.3511745307125851e-06
iter 200 loss 9.688629596559478e-07
iter 300 loss 8.175218274036796e-07
iter 400 loss 7.409878697527935e-07
iter 500 loss 7.941934486661704e-07
iter 600 loss 7.67152542924088e-07
iter 700 loss 7.392594536145971e-07
iter 800 loss 7.12018660086121e-07
iter 900 loss 6.837973737033488e-07
iter 1000 loss 6.657787403432545e-07
iter 1100 loss 7.230172114936403e-07
iter 1200 loss 7.133104820076658e-07
iter 1300 loss 7.014476302711332e-07
iter 1400 loss 6.885421384076232e-07
iter 1500 loss 6.765459013284666e-07
iter 1600 loss 6.668533458010134e-07
iter 1700 loss 6.604951283801585e-07
iter 1800 loss 6.554679193621203e-07
iter 1900 loss 6.501997178486237e-07
iter 2000 loss 6.445386216755274e-07
iter 2100 loss 6.405425609700302e-07
iter 2200 loss 0.0008817503697521826
iter 2300 loss 0.0021168370441688534
iter 2400 loss 0.0031381942523705385
epoch 4 loss 0.00308300264389747
--- Result:
Test failed! Accuracy = 0.141211599111557<0.3
Test G: 0/20
--- Time taken: 113.13574194908142


--- TEST: batch_size= 256  num_epochs= 5  lr= 0.0001 

iter 0 loss 6.560696419910528e-06
iter 100 loss 1.5955076078891652e-06
iter 200 loss 1.1805483817493696e-06
iter 300 loss 1.0023797092472884e-06
iter 400 loss 9.063205746195153e-07
iter 500 loss 1.0294020146652103e-06
iter 600 loss 1.101765155189833e-06
iter 700 loss 1.0750840655439629e-06
iter 800 loss 1.0196296062950455e-06
iter 900 loss 9.6180551410784e-07
iter 1000 loss 9.185328759007939e-07
iter 1100 loss 9.492718475676809e-07
iter 1200 loss 9.357183791388957e-07
iter 1300 loss 9.115274630650414e-07
iter 1400 loss 8.826142247987999e-07
iter 1500 loss 8.561735976696991e-07
iter 1600 loss 8.330484915913383e-07
iter 1700 loss 8.133782453911367e-07
iter 1800 loss 7.956307077963186e-07
iter 1900 loss 7.787746910841898e-07
iter 2000 loss 7.623402624907702e-07
iter 2100 loss 7.477559637699008e-07
iter 2200 loss 0.0008826101210351842
iter 2300 loss 0.002118026543104196
iter 2400 loss 0.0031391077519880533
epoch 0 loss 0.0030838976382166674
iter 0 loss 6.13410520600155e-06
iter 100 loss 1.4273280782509745e-06
iter 200 loss 1.0606523696833618e-06
iter 300 loss 9.049429531101893e-07
iter 400 loss 8.237912336080354e-07
iter 500 loss 8.248127206986361e-07
iter 600 loss 8.397273157200663e-07
iter 700 loss 8.190285436159344e-07
iter 800 loss 7.878523821118472e-07
iter 900 loss 7.549733573669214e-07
iter 1000 loss 7.308485938078452e-07
iter 1100 loss 7.719224907434725e-07
iter 1200 loss 7.662725631267498e-07
iter 1300 loss 7.52487765428495e-07
iter 1400 loss 7.347818125437372e-07
iter 1500 loss 7.178984945283465e-07
iter 1600 loss 7.033069294468974e-07
iter 1700 loss 6.913554884177692e-07
iter 1800 loss 6.805471539544352e-07
iter 1900 loss 6.699869953054808e-07
iter 2000 loss 6.593115413466933e-07
iter 2100 loss 6.502163826836767e-07
iter 2200 loss 0.0008690594977479085
iter 2300 loss 0.0021042309124023836
iter 2400 loss 0.0031257922629852634
epoch 1 loss 0.003070817089084008
iter 0 loss 5.98000315221725e-06
iter 100 loss 1.374026327435912e-06
iter 200 loss 1.0223687063195365e-06
iter 300 loss 8.73986654931232e-07
iter 400 loss 7.95972417668927e-07
iter 500 loss 7.995591920394099e-07
iter 600 loss 8.050044556142902e-07
iter 700 loss 7.836805315499586e-07
iter 800 loss 7.554576103650695e-07
iter 900 loss 7.25717652020165e-07
iter 1000 loss 7.042029245745801e-07
iter 1100 loss 7.427342380002395e-07
iter 1200 loss 7.378672551762055e-07
iter 1300 loss 7.259222106862368e-07
iter 1400 loss 7.104039382508066e-07
iter 1500 loss 6.956631297345919e-07
iter 1600 loss 6.829086171065244e-07
iter 1700 loss 6.728938718791674e-07
iter 1800 loss 6.636018712565295e-07
iter 1900 loss 6.546753169359415e-07
iter 2000 loss 6.454182929563172e-07
iter 2100 loss 6.376227183921461e-07
iter 2200 loss 0.0008692414391611956
iter 2300 loss 0.0021042206234084134
iter 2400 loss 0.0031256458510681177
epoch 2 loss 0.0030706737856907034
iter 0 loss 5.912967935728375e-06
iter 100 loss 1.3480140017299443e-06
iter 200 loss 1.0028851499614281e-06
iter 300 loss 8.563472748661007e-07
iter 400 loss 7.800366240343989e-07
iter 500 loss 7.820332509205498e-07
iter 600 loss 7.85367167913751e-07
iter 700 loss 7.652171138755104e-07
iter 800 loss 7.384841915375951e-07
iter 900 loss 7.099804923623737e-07
iter 1000 loss 6.89829500889881e-07
iter 1100 loss 7.258616757149737e-07
iter 1200 loss 7.216408168154155e-07
iter 1300 loss 7.107518755654118e-07
iter 1400 loss 6.964586834364686e-07
iter 1500 loss 6.827573827043529e-07
iter 1600 loss 6.709895973482667e-07
iter 1700 loss 6.619682048108683e-07
iter 1800 loss 6.538123706162139e-07
iter 1900 loss 6.459703045733542e-07
iter 2000 loss 6.376861163688232e-07
iter 2100 loss 6.307809654827309e-07
iter 2200 loss 0.0008691478033859048
iter 2300 loss 0.00210399246598578
iter 2400 loss 0.0031253211791415934
epoch 3 loss 0.003070355258012542
iter 0 loss 5.8678119785327e-06
iter 100 loss 1.3310290075135679e-06
iter 200 loss 9.887115084549706e-07
iter 300 loss 8.446222480104377e-07
iter 400 loss 7.699117189194727e-07
iter 500 loss 7.697428502038809e-07
iter 600 loss 7.720335286886649e-07
iter 700 loss 7.527121052659853e-07
iter 800 loss 7.26781118299393e-07
iter 900 loss 6.990667268814352e-07
iter 1000 loss 6.798194780381967e-07
iter 1100 loss 7.142005573088308e-07
iter 1200 loss 7.105571250491943e-07
iter 1300 loss 7.002638496014361e-07
iter 1400 loss 6.867754393502916e-07
iter 1500 loss 6.736686903372814e-07
iter 1600 loss 6.626983698505121e-07
iter 1700 loss 6.544578768182011e-07
iter 1800 loss 6.472594337204083e-07
iter 1900 loss 6.401980984452355e-07
iter 2000 loss 6.325891466344367e-07
iter 2100 loss 6.261925578537397e-07
iter 2200 loss 0.0008690583817334214
iter 2300 loss 0.0021037959262808084
iter 2400 loss 0.003125045857314634
epoch 4 loss 0.0030700851007019504
--- Result:
Test failed! Accuracy = 0.14322471618652344<0.3
Test G: 0/20
--- Time taken: 113.52554225921631


--- TEST: batch_size= 256  num_epochs= 10  lr= 0.001 

iter 0 loss 5.82079155719839e-06
iter 100 loss 1.031734715263948e-06
iter 200 loss 7.317182462373067e-07
iter 300 loss 6.155901781915228e-07
iter 400 loss 5.571014848063762e-07
iter 500 loss 5.287306770539859e-07
iter 600 loss 5.065028480700301e-07
iter 700 loss 4.876221876571947e-07
iter 800 loss 4.763799380925525e-07
iter 900 loss 4.648935382321783e-07
iter 1000 loss 4.593120403702282e-07
iter 1100 loss 5.866994634963751e-07
iter 1200 loss 5.769086355633391e-07
iter 1300 loss 5.680753056732659e-07
iter 1400 loss 5.592854822414076e-07
iter 1500 loss 5.518024264784508e-07
iter 1600 loss 5.457521134312907e-07
iter 1700 loss 5.417239322281552e-07
iter 1800 loss 5.370180983342053e-07
iter 1900 loss 5.328007404234262e-07
iter 2000 loss 5.299299457506051e-07
iter 2100 loss 5.2830165198599e-07
iter 2200 loss 0.0008956427323444097
iter 2300 loss 0.0021305547135244225
iter 2400 loss 0.003150465744730419
epoch 0 loss 0.003095059617877346
iter 0 loss 8.20174318505451e-06
iter 100 loss 1.2852373067676802e-06
iter 200 loss 8.947660556890723e-07
iter 300 loss 7.462300325632926e-07
iter 400 loss 6.716537190693413e-07
iter 500 loss 1.0554694530943569e-06
iter 600 loss 9.679285918796593e-07
iter 700 loss 9.02215071595461e-07
iter 800 loss 8.4906673031896e-07
iter 900 loss 8.037406093907007e-07
iter 1000 loss 7.705874653791826e-07
iter 1100 loss 8.419502097090666e-07
iter 1200 loss 8.204384382469418e-07
iter 1300 loss 7.98349803806931e-07
iter 1400 loss 7.763382959177051e-07
iter 1500 loss 7.570729402075614e-07
iter 1600 loss 7.412203211107424e-07
iter 1700 loss 7.294192078338248e-07
iter 1800 loss 7.180727816259871e-07
iter 1900 loss 7.069123416771367e-07
iter 2000 loss 6.973549232171699e-07
iter 2100 loss 6.907977859736483e-07
iter 2200 loss 0.0009001022920039196
iter 2300 loss 0.0021338030464574513
iter 2400 loss 0.0031536243707624896
epoch 1 loss 0.0030981609228063166
iter 0 loss 7.136289241316263e-06
iter 100 loss 1.3634470327239193e-06
iter 200 loss 9.551383715431859e-07
iter 300 loss 7.987744946986155e-07
iter 400 loss 7.207366728128538e-07
iter 500 loss 7.478549810615416e-07
iter 600 loss 7.301333502373942e-07
iter 700 loss 7.055356058913438e-07
iter 800 loss 6.824666437193075e-07
iter 900 loss 6.576648111457428e-07
iter 1000 loss 6.415576320522815e-07
iter 1100 loss 6.74138357592749e-07
iter 1200 loss 6.651728527119094e-07
iter 1300 loss 6.551169871124723e-07
iter 1400 loss 6.455400673504709e-07
iter 1500 loss 6.36307277829436e-07
iter 1600 loss 6.290588412245872e-07
iter 1700 loss 6.252304979591793e-07
iter 1800 loss 6.216005379855147e-07
iter 1900 loss 6.173613893416772e-07
iter 2000 loss 6.138656921177125e-07
iter 2100 loss 6.123448701141138e-07
iter 2200 loss 0.0008972517290799066
iter 2300 loss 0.002131236687429226
iter 2400 loss 0.003151364075448709
epoch 2 loss 0.003095940570281132
iter 0 loss 8.611696102889255e-06
iter 100 loss 1.4603805418641293e-06
iter 200 loss 1.0170608338717215e-06
iter 300 loss 8.467878404437602e-07
iter 400 loss 7.617966980093057e-07
iter 500 loss 7.987695990558207e-07
iter 600 loss 7.620361598605201e-07
iter 700 loss 7.308766526251782e-07
iter 800 loss 7.05143394250765e-07
iter 900 loss 6.784759213388819e-07
iter 1000 loss 6.62210967843353e-07
iter 1100 loss 7.920643344241431e-07
iter 1200 loss 7.747795790181772e-07
iter 1300 loss 7.572753281710189e-07
iter 1400 loss 7.415413297438136e-07
iter 1500 loss 7.271347313927278e-07
iter 1600 loss 7.151028539827508e-07
iter 1700 loss 7.074274478790193e-07
iter 1800 loss 7.010315285680371e-07
iter 1900 loss 6.943590639457828e-07
iter 2000 loss 6.883521162991996e-07
iter 2100 loss 6.841645809352568e-07
iter 2200 loss 0.0008971788698934599
iter 2300 loss 0.0021315421707312444
iter 2400 loss 0.0031519783498762284
epoch 3 loss 0.0030965445313378705
iter 0 loss 8.119195626932196e-06
iter 100 loss 1.4994912009011512e-06
iter 200 loss 1.045966431530672e-06
iter 300 loss 8.710222358895045e-07
iter 400 loss 7.840914597302044e-07
iter 500 loss 9.130148561994727e-07
iter 600 loss 8.676259478485316e-07
iter 700 loss 8.280219077951525e-07
iter 800 loss 7.9391674198144e-07
iter 900 loss 7.588720502180281e-07
iter 1000 loss 7.365195653665478e-07
iter 1100 loss 7.968264817251221e-07
iter 1200 loss 7.816076665778752e-07
iter 1300 loss 7.653266898766373e-07
iter 1400 loss 7.504518810926923e-07
iter 1500 loss 7.362889628394774e-07
iter 1600 loss 7.249031178697243e-07
iter 1700 loss 7.179231644768997e-07
iter 1800 loss 7.123748077506678e-07
iter 1900 loss 7.064788635577604e-07
iter 2000 loss 7.008087203845845e-07
iter 2100 loss 6.968115624056892e-07
iter 2200 loss 0.0008970969826949676
iter 2300 loss 0.002131884302263502
iter 2400 loss 0.0031526468290676065
epoch 4 loss 0.0030972016407760346
iter 0 loss 9.228631824953482e-06
iter 100 loss 1.5539028982623288e-06
iter 200 loss 1.0799844174027783e-06
iter 300 loss 8.971513518471483e-07
iter 400 loss 8.059899255782883e-07
iter 500 loss 8.504182365149591e-07
iter 600 loss 8.165573934035612e-07
iter 700 loss 7.847971280719706e-07
iter 800 loss 7.559278630890546e-07
iter 900 loss 7.251185958528097e-07
iter 1000 loss 7.069870664790588e-07
iter 1100 loss 7.783061967017076e-07
iter 1200 loss 7.645856439639685e-07
iter 1300 loss 7.499194310583098e-07
iter 1400 loss 7.369899226784785e-07
iter 1500 loss 7.244830125257381e-07
iter 1600 loss 7.14646938803174e-07
iter 1700 loss 7.09091479415921e-07
iter 1800 loss 7.050858239983547e-07
iter 1900 loss 7.008536909462212e-07
iter 2000 loss 6.963406002600421e-07
iter 2100 loss 6.930616645073307e-07
iter 2200 loss 0.0008970029117260102
iter 2300 loss 0.002132213124109483
iter 2400 loss 0.0031532954551009443
epoch 5 loss 0.0030978392027233947
iter 0 loss 8.458168849756476e-06
iter 100 loss 1.561874810677452e-06
iter 200 loss 1.0877201206294648e-06
iter 300 loss 9.039383580467223e-07
iter 400 loss 8.132803340369828e-07
iter 500 loss 9.187348802085895e-07
iter 600 loss 8.763109142141343e-07
iter 700 loss 8.379071842201261e-07
iter 800 loss 8.038718702254585e-07
iter 900 loss 7.682127496313767e-07
iter 1000 loss 7.46609274712953e-07
iter 1100 loss 8.315768912448935e-07
iter 1200 loss 8.1438935379554e-07
iter 1300 loss 7.966502538300491e-07
iter 1400 loss 7.811237549893038e-07
iter 1500 loss 7.661347645348242e-07
iter 1600 loss 7.545905909877595e-07
iter 1700 loss 7.476745993060084e-07
iter 1800 loss 7.426890963397969e-07
iter 1900 loss 7.375219052754084e-07
iter 2000 loss 7.318290820460314e-07
iter 2100 loss 7.273492904509565e-07
iter 2200 loss 0.0008969977670218396
iter 2300 loss 0.002132602436167518
iter 2400 loss 0.00315398200020016
epoch 6 loss 0.0030985139603767705
iter 0 loss 9.428852536075283e-06
iter 100 loss 1.5989930690497797e-06
iter 200 loss 1.111086106534028e-06
iter 300 loss 9.21394919290126e-07
iter 400 loss 8.286898786709141e-07
iter 500 loss 9.004318483679519e-07
iter 600 loss 8.645600799872274e-07
iter 700 loss 8.29899881854818e-07
iter 800 loss 7.976135880743869e-07
iter 900 loss 7.627761996903546e-07
iter 1000 loss 7.421628886213576e-07
iter 1100 loss 8.045764542858854e-07
iter 1200 loss 7.899949525040692e-07
iter 1300 loss 7.745804567512822e-07
iter 1400 loss 7.61197279576614e-07
iter 1500 loss 7.480846388094011e-07
iter 1600 loss 7.382946869331827e-07
iter 1700 loss 7.3296115339411e-07
iter 1800 loss 7.296962643503988e-07
iter 1900 loss 7.260983053483922e-07
iter 2000 loss 7.21552110761841e-07
iter 2100 loss 7.178039375830044e-07
iter 2200 loss 0.0008969667943170369
iter 2300 loss 0.0021329325032135495
iter 2400 loss 0.0031545842331188274
epoch 7 loss 0.003099105790742682
iter 0 loss 8.562943548895419e-06
iter 100 loss 1.5913448872215487e-06
iter 200 loss 1.1087025231248827e-06
iter 300 loss 9.209279279423564e-07
iter 400 loss 8.29659683106065e-07
iter 500 loss 8.974528867441108e-07
iter 600 loss 8.61838943815965e-07
iter 700 loss 8.273590128997336e-07
iter 800 loss 7.956358439436242e-07
iter 900 loss 7.609745474664719e-07
iter 1000 loss 7.408249894330959e-07
iter 1100 loss 8.220239195769658e-07
iter 1200 loss 8.062568181115259e-07
iter 1300 loss 7.898640366113402e-07
iter 1400 loss 7.756222859689355e-07
iter 1500 loss 7.618278283268963e-07
iter 1600 loss 7.515968415473077e-07
iter 1700 loss 7.45981988963314e-07
iter 1800 loss 7.426216374992598e-07
iter 1900 loss 7.389786776792086e-07
iter 2000 loss 7.342321062448933e-07
iter 2100 loss 7.300094042606532e-07
iter 2200 loss 0.0008969734204443539
iter 2300 loss 0.0021332598801241313
iter 2400 loss 0.0031551524159139236
epoch 8 loss 0.0030996641388192996
iter 0 loss 9.43864415603457e-06
iter 100 loss 1.617550435527153e-06
iter 200 loss 1.1244304588371192e-06
iter 300 loss 9.323712403249337e-07
iter 400 loss 8.392816485659811e-07
iter 500 loss 9.176665049742228e-07
iter 600 loss 8.805262913507815e-07
iter 700 loss 8.448718534064016e-07
iter 800 loss 8.115831936426983e-07
iter 900 loss 7.754253373378912e-07
iter 1000 loss 7.540219436172216e-07
iter 1100 loss 8.188828817618131e-07
iter 1200 loss 8.036417454538932e-07
iter 1300 loss 7.876561335146747e-07
iter 1400 loss 7.740210193359658e-07
iter 1500 loss 7.605733622582317e-07
iter 1600 loss 7.508750464597317e-07
iter 1700 loss 7.45630040395696e-07
iter 1800 loss 7.427437724336672e-07
iter 1900 loss 7.395166911149866e-07
iter 2000 loss 7.349703613688708e-07
iter 2100 loss 7.307688170367986e-07
iter 2200 loss 0.0008969783855398109
iter 2300 loss 0.0021335411897459283
iter 2400 loss 0.003155641091489493
epoch 9 loss 0.003100144323090617
--- Result:
Test failed! Accuracy = 0.13785196840763092<0.3
Test G: 0/20
--- Time taken: 221.22838306427002


--- TEST: batch_size= 256  num_epochs= 10  lr= 0.0005 

iter 0 loss 8.58483326737769e-06
iter 100 loss 2.0053638131189876e-06
iter 200 loss 1.4219163588437659e-06
iter 300 loss 1.1776959016353666e-06
iter 400 loss 1.0466728179011186e-06
iter 500 loss 1.0451491259722998e-06
iter 600 loss 9.91229061066206e-07
iter 700 loss 9.399798087385888e-07
iter 800 loss 8.955337486813267e-07
iter 900 loss 8.540229872249642e-07
iter 1000 loss 8.243229621081076e-07
iter 1100 loss 8.875486209498864e-07
iter 1200 loss 8.64908125982632e-07
iter 1300 loss 8.409275788705629e-07
iter 1400 loss 8.18110218527004e-07
iter 1500 loss 7.971375022845688e-07
iter 1600 loss 7.778254964263729e-07
iter 1700 loss 7.612073831302466e-07
iter 1800 loss 7.457526288277623e-07
iter 1900 loss 7.308310881934301e-07
iter 2000 loss 7.171086669591429e-07
iter 2100 loss 7.050681969132057e-07
iter 2200 loss 0.0008924105260292714
iter 2300 loss 0.0021287793351279873
iter 2400 loss 0.003150242174637898
epoch 0 loss 0.0030948374449754176
iter 0 loss 7.106287739588879e-06
iter 100 loss 1.529795055587486e-06
iter 200 loss 1.0906647550872373e-06
iter 300 loss 9.124471742542667e-07
iter 400 loss 8.213754475596906e-07
iter 500 loss 8.709675269332758e-07
iter 600 loss 8.371181464238433e-07
iter 700 loss 8.011015337694533e-07
iter 800 loss 7.678179015348949e-07
iter 900 loss 7.360244061256687e-07
iter 1000 loss 7.136326169492841e-07
iter 1100 loss 7.427288237033147e-07
iter 1200 loss 7.294480172140273e-07
iter 1300 loss 7.148264567448308e-07
iter 1400 loss 7.004907162815921e-07
iter 1500 loss 6.869501548700814e-07
iter 1600 loss 6.748922713875704e-07
iter 1700 loss 6.651908409203679e-07
iter 1800 loss 6.564739268509208e-07
iter 1900 loss 6.481832082843502e-07
iter 2000 loss 6.402396618840136e-07
iter 2100 loss 6.341954912410669e-07
iter 2200 loss 0.0008808872249622449
iter 2300 loss 0.00211613842310375
iter 2400 loss 0.0031374871238154583
epoch 1 loss 0.0030823080987564685
iter 0 loss 6.454064532590564e-06
iter 100 loss 1.419725140970126e-06
iter 200 loss 1.0222410374147904e-06
iter 300 loss 8.611645591562867e-07
iter 400 loss 7.78547735137494e-07
iter 500 loss 7.928146645692954e-07
iter 600 loss 7.652369337065254e-07
iter 700 loss 7.370126333947865e-07
iter 800 loss 7.111807401193896e-07
iter 900 loss 6.843967185986802e-07
iter 1000 loss 6.668848190368696e-07
iter 1100 loss 7.002042593688414e-07
iter 1200 loss 6.907639374654827e-07
iter 1300 loss 6.794891064130833e-07
iter 1400 loss 6.683171505516814e-07
iter 1500 loss 6.576097134740526e-07
iter 1600 loss 6.483739547129494e-07
iter 1700 loss 6.417749501270918e-07
iter 1800 loss 6.361766252070566e-07
iter 1900 loss 6.306974650154603e-07
iter 2000 loss 6.250719931615875e-07
iter 2100 loss 6.209297486348052e-07
iter 2200 loss 0.0008815022012929911
iter 2300 loss 0.002116351102655454
iter 2400 loss 0.0031375055081219764
epoch 2 loss 0.0030823268775343566
iter 0 loss 6.374932581820758e-06
iter 100 loss 1.3979223794113651e-06
iter 200 loss 1.007075937521412e-06
iter 300 loss 8.499253713019899e-07
iter 400 loss 7.692856884556507e-07
iter 500 loss 7.88349832314661e-07
iter 600 loss 7.61472408680561e-07
iter 700 loss 7.340667886511825e-07
iter 800 loss 7.087131239466807e-07
iter 900 loss 6.821631820021623e-07
iter 1000 loss 6.651414360085191e-07
iter 1100 loss 6.98344397559422e-07
iter 1200 loss 6.899790122763388e-07
iter 1300 loss 6.795490855919365e-07
iter 1400 loss 6.69087792016512e-07
iter 1500 loss 6.588995331574898e-07
iter 1600 loss 6.503513628370687e-07
iter 1700 loss 6.446054825555631e-07
iter 1800 loss 6.400843427487973e-07
iter 1900 loss 6.356831874541357e-07
iter 2000 loss 6.306898424371503e-07
iter 2100 loss 6.270619244431402e-07
iter 2200 loss 0.0008812149944096336
iter 2300 loss 0.002115972101852406
iter 2400 loss 0.003137098236231059
epoch 3 loss 0.003081927277282549
iter 0 loss 6.306498107733205e-06
iter 100 loss 1.3886185958636087e-06
iter 200 loss 1.0002116009197936e-06
iter 300 loss 8.447289144026571e-07
iter 400 loss 7.653560748424791e-07
iter 500 loss 7.816365235678726e-07
iter 600 loss 7.561085755269944e-07
iter 700 loss 7.298733376578528e-07
iter 800 loss 7.054047995701281e-07
iter 900 loss 6.79123843919681e-07
iter 1000 loss 6.62691823700206e-07
iter 1100 loss 6.950233488211576e-07
iter 1200 loss 6.87294687295707e-07
iter 1300 loss 6.775099050350111e-07
iter 1400 loss 6.677721682292659e-07
iter 1500 loss 6.582427000488967e-07
iter 1600 loss 6.502657868452211e-07
iter 1700 loss 6.45129666615494e-07
iter 1800 loss 6.414639442498852e-07
iter 1900 loss 6.377601116751456e-07
iter 2000 loss 6.333662338954315e-07
iter 2100 loss 6.30176992428814e-07
iter 2200 loss 0.0008811585936330194
iter 2300 loss 0.0021159116754581567
iter 2400 loss 0.0031370572052845623
epoch 4 loss 0.003081887332155455
iter 0 loss 6.284157279878855e-06
iter 100 loss 1.3870234699136897e-06
iter 200 loss 9.9884247287834e-07
iter 300 loss 8.432731795624913e-07
iter 400 loss 7.646489011778813e-07
iter 500 loss 7.801902444350458e-07
iter 600 loss 7.555367446367199e-07
iter 700 loss 7.298919202661217e-07
iter 800 loss 7.057931263441109e-07
iter 900 loss 6.795527970708896e-07
iter 1000 loss 6.634589200726426e-07
iter 1100 loss 6.956270641348453e-07
iter 1200 loss 6.881873965760601e-07
iter 1300 loss 6.786056629496825e-07
iter 1400 loss 6.690389968913823e-07
iter 1500 loss 6.597434298532956e-07
iter 1600 loss 6.520919034406565e-07
iter 1700 loss 6.472530397949157e-07
iter 1800 loss 6.440028202352656e-07
iter 1900 loss 6.408598820714395e-07
iter 2000 loss 6.368807768917807e-07
iter 2100 loss 6.338731135580012e-07
iter 2200 loss 0.0008811280652409134
iter 2300 loss 0.002115920694058633
iter 2400 loss 0.0031371108572819926
epoch 5 loss 0.0030819402701657596
iter 0 loss 6.268331162573304e-06
iter 100 loss 1.387443051449322e-06
iter 200 loss 9.991413422829653e-07
iter 300 loss 8.429220079081663e-07
iter 400 loss 7.648219335371726e-07
iter 500 loss 7.801763946097893e-07
iter 600 loss 7.56301563284771e-07
iter 700 loss 7.312630671318472e-07
iter 800 loss 7.071326186586534e-07
iter 900 loss 6.807720480283306e-07
iter 1000 loss 6.648996815727571e-07
iter 1100 loss 6.965893977889255e-07
iter 1200 loss 6.893297691332356e-07
iter 1300 loss 6.799812872525104e-07
iter 1400 loss 6.70516523678478e-07
iter 1500 loss 6.61316106516854e-07
iter 1600 loss 6.539019963165765e-07
iter 1700 loss 6.49435793745067e-07
iter 1800 loss 6.466187223830084e-07
iter 1900 loss 6.4392925605501e-07
iter 2000 loss 6.402191352737836e-07
iter 2100 loss 6.373863643899345e-07
iter 2200 loss 0.0008811158422557333
iter 2300 loss 0.0021159685243530447
iter 2400 loss 0.003137215289155288
epoch 6 loss 0.0030820430607107667
iter 0 loss 6.2664707911608275e-06
iter 100 loss 1.3878717915290643e-06
iter 200 loss 9.99653317947703e-07
iter 300 loss 8.429761411612328e-07
iter 400 loss 7.653502922649589e-07
iter 500 loss 7.804345353246256e-07
iter 600 loss 7.570149581225207e-07
iter 700 loss 7.322858845004514e-07
iter 800 loss 7.081184346273911e-07
iter 900 loss 6.816313948792545e-07
iter 1000 loss 6.658741439865039e-07
iter 1100 loss 6.973041516359113e-07
iter 1200 loss 6.901358359328143e-07
iter 1300 loss 6.809777337723068e-07
iter 1400 loss 6.716741785295133e-07
iter 1500 loss 6.62632103352084e-07
iter 1600 loss 6.554089089241813e-07
iter 1700 loss 6.511248616165917e-07
iter 1800 loss 6.486985404809982e-07
iter 1900 loss 6.463682680250144e-07
iter 2000 loss 6.429013859079445e-07
iter 2100 loss 6.401512830164209e-07
iter 2200 loss 0.0008811164505479022
iter 2300 loss 0.0021160384417785254
iter 2400 loss 0.0031373469140684976
epoch 7 loss 0.003082172557284353
iter 0 loss 6.268798642850015e-06
iter 100 loss 1.3888814674099211e-06
iter 200 loss 1.0007259456259233e-06
iter 300 loss 8.436521908949613e-07
iter 400 loss 7.663791523344189e-07
iter 500 loss 7.812474001854323e-07
iter 600 loss 7.583550352906294e-07
iter 700 loss 7.33834693485145e-07
iter 800 loss 7.096767758077213e-07
iter 900 loss 6.830353849606817e-07
iter 1000 loss 6.673006935813504e-07
iter 1100 loss 6.985064902081411e-07
iter 1200 loss 6.913528306228684e-07
iter 1300 loss 6.822797899111903e-07
iter 1400 loss 6.730684316757908e-07
iter 1500 loss 6.641425659919398e-07
iter 1600 loss 6.570071027956855e-07
iter 1700 loss 6.529970289908848e-07
iter 1800 loss 6.508923402071215e-07
iter 1900 loss 6.488692133966721e-07
iter 2000 loss 6.455838307926294e-07
iter 2100 loss 6.428731681175684e-07
iter 2200 loss 0.0008811247946828187
iter 2300 loss 0.0021161197318473263
iter 2400 loss 0.0031374915066865487
epoch 8 loss 0.0030823147806414886
iter 0 loss 6.279975423240103e-06
iter 100 loss 1.3905873147720767e-06
iter 200 loss 1.0022480115928298e-06
iter 300 loss 8.446871516750141e-07
iter 400 loss 7.674521369305768e-07
iter 500 loss 7.823547755646733e-07
iter 600 loss 7.598964555370174e-07
iter 700 loss 7.354770730622287e-07
iter 800 loss 7.113036333200686e-07
iter 900 loss 6.845421498045421e-07
iter 1000 loss 6.687932348079364e-07
iter 1100 loss 6.996591991579287e-07
iter 1200 loss 6.926197088444032e-07
iter 1300 loss 6.835287501024163e-07
iter 1400 loss 6.74375819801063e-07
iter 1500 loss 6.655139369894539e-07
iter 1600 loss 6.584996156267563e-07
iter 1700 loss 6.546640577566154e-07
iter 1800 loss 6.52814048793201e-07
iter 1900 loss 6.510437934600635e-07
iter 2000 loss 6.479231751359441e-07
iter 2100 loss 6.452403566701014e-07
iter 2200 loss 0.0008811369780839695
iter 2300 loss 0.0021162053823999857
iter 2400 loss 0.003137639920047759
epoch 9 loss 0.003082460743038611
--- Result:
Test failed! Accuracy = 0.13779863715171814<0.3
Test G: 0/20
--- Time taken: 221.89721083641052


--- TEST: batch_size= 256  num_epochs= 10  lr= 0.0001 

iter 0 loss 6.289755219768267e-06
iter 100 loss 1.6643163570562938e-06
iter 200 loss 1.2453154216751472e-06
iter 300 loss 1.0574690123422046e-06
iter 400 loss 9.559841002517356e-07
iter 500 loss 1.001803709954685e-06
iter 600 loss 1.028277813040068e-06
iter 700 loss 9.98243637355925e-07
iter 800 loss 9.516034361433058e-07
iter 900 loss 9.033057654631356e-07
iter 1000 loss 8.67936215832764e-07
iter 1100 loss 8.81186617173937e-07
iter 1200 loss 8.667186004692169e-07
iter 1300 loss 8.462396841405293e-07
iter 1400 loss 8.239395814855612e-07
iter 1500 loss 8.03020585956133e-07
iter 1600 loss 7.845187960024228e-07
iter 1700 loss 7.689936941589332e-07
iter 1800 loss 7.550481115051348e-07
iter 1900 loss 7.419384611992414e-07
iter 2000 loss 7.283081263535544e-07
iter 2100 loss 7.158425407917625e-07
iter 2200 loss 0.0008811366578198058
iter 2300 loss 0.0021161791584924814
iter 2400 loss 0.00313708398532503
epoch 0 loss 0.0030819113592208043
iter 0 loss 5.858951226400677e-06
iter 100 loss 1.4706267127549648e-06
iter 200 loss 1.1006049091624326e-06
iter 300 loss 9.389196306938326e-07
iter 400 loss 8.540653335498554e-07
iter 500 loss 8.382888555041757e-07
iter 600 loss 8.366876189092985e-07
iter 700 loss 8.130733639221329e-07
iter 800 loss 7.83001140211213e-07
iter 900 loss 7.516988492596619e-07
iter 1000 loss 7.295559118840818e-07
iter 1100 loss 7.4926157558399e-07
iter 1200 loss 7.415213637412021e-07
iter 1300 loss 7.29024886847494e-07
iter 1400 loss 7.142085007339292e-07
iter 1500 loss 6.999485475841758e-07
iter 1600 loss 6.873739922087541e-07
iter 1700 loss 6.769771025127841e-07
iter 1800 loss 6.679007733237238e-07
iter 1900 loss 6.59563048511896e-07
iter 2000 loss 6.508432142052798e-07
iter 2100 loss 6.428313222841963e-07
iter 2200 loss 0.0008687604607239546
iter 2300 loss 0.0021036377720697994
iter 2400 loss 0.0031249895925207485
epoch 1 loss 0.003070030365410241
iter 0 loss 5.6908866099547595e-06
iter 100 loss 1.4109072054398525e-06
iter 200 loss 1.0567495689827238e-06
iter 300 loss 9.025208631530186e-07
iter 400 loss 8.222019223773403e-07
iter 500 loss 8.085584185974148e-07
iter 600 loss 8.026895797359516e-07
iter 700 loss 7.79679891892713e-07
iter 800 loss 7.522570012867042e-07
iter 900 loss 7.235369102157394e-07
iter 1000 loss 7.036495832506591e-07
iter 1100 loss 7.224756689920122e-07
iter 1200 loss 7.159006903552242e-07
iter 1300 loss 7.049962723707509e-07
iter 1400 loss 6.919199176212231e-07
iter 1500 loss 6.789742508188672e-07
iter 1600 loss 6.67930821362212e-07
iter 1700 loss 6.590793967165704e-07
iter 1800 loss 6.515250631030998e-07
iter 1900 loss 6.445413708377415e-07
iter 2000 loss 6.370765357973403e-07
iter 2100 loss 6.30317423161698e-07
iter 2200 loss 0.0008689363154725903
iter 2300 loss 0.0021036431813254573
iter 2400 loss 0.0031248793641301463
epoch 2 loss 0.0030699226232160965
iter 0 loss 5.6047642829071265e-06
iter 100 loss 1.3773382934772652e-06
iter 200 loss 1.0296648753041634e-06
iter 300 loss 8.788712238887353e-07
iter 400 loss 8.011763847527273e-07
iter 500 loss 7.877722714102582e-07
iter 600 loss 7.82114892511537e-07
iter 700 loss 7.60397498241675e-07
iter 800 loss 7.344534838914578e-07
iter 900 loss 7.070695341004167e-07
iter 1000 loss 6.881188060074268e-07
iter 1100 loss 7.064728599336995e-07
iter 1200 loss 7.00855026611752e-07
iter 1300 loss 6.910115138614452e-07
iter 1400 loss 6.791643598387096e-07
iter 1500 loss 6.672397450706924e-07
iter 1600 loss 6.571732928211301e-07
iter 1700 loss 6.494374732826265e-07
iter 1800 loss 6.429226899382592e-07
iter 1900 loss 6.36856687123892e-07
iter 2000 loss 6.302231708084956e-07
iter 2100 loss 6.242399564790585e-07
iter 2200 loss 0.0008688569613956758
iter 2300 loss 0.0021034539492280017
iter 2400 loss 0.0031246155903470038
epoch 3 loss 0.003069663931378897
iter 0 loss 5.5498312576673925e-06
iter 100 loss 1.3544518459062908e-06
iter 200 loss 1.0111844401319502e-06
iter 300 loss 8.624663744821829e-07
iter 400 loss 7.866700800599361e-07
iter 500 loss 7.733649599395575e-07
iter 600 loss 7.683018809125232e-07
iter 700 loss 7.475206814514566e-07
iter 800 loss 7.22323863883094e-07
iter 900 loss 6.957362495515784e-07
iter 1000 loss 6.778311914146073e-07
iter 1100 loss 6.959372737472076e-07
iter 1200 loss 6.909380654229793e-07
iter 1300 loss 6.818027641127056e-07
iter 1400 loss 6.70800702167134e-07
iter 1500 loss 6.596960612590905e-07
iter 1600 loss 6.502959588774891e-07
iter 1700 loss 6.43256001729569e-07
iter 1800 loss 6.376139668548202e-07
iter 1900 loss 6.322333591823502e-07
iter 2000 loss 6.261941607398381e-07
iter 2100 loss 6.207639825385197e-07
iter 2200 loss 0.0008687867136493646
iter 2300 loss 0.002103302529006588
iter 2400 loss 0.0031244072264689023
epoch 4 loss 0.0030694595615733313
iter 0 loss 5.506070010596886e-06
iter 100 loss 1.3377526924798959e-06
iter 200 loss 9.970548073094203e-07
iter 300 loss 8.508697465230552e-07
iter 400 loss 7.763825953958516e-07
iter 500 loss 7.632795565129895e-07
iter 600 loss 7.584829160529963e-07
iter 700 loss 7.382196001679445e-07
iter 800 loss 7.136659845646397e-07
iter 900 loss 6.878925107225774e-07
iter 1000 loss 6.704635484322795e-07
iter 1100 loss 6.882568900253237e-07
iter 1200 loss 6.837696219055979e-07
iter 1300 loss 6.751806624078989e-07
iter 1400 loss 6.647513154967405e-07
iter 1500 loss 6.5432363485276e-07
iter 1600 loss 6.453670066993034e-07
iter 1700 loss 6.388344542385049e-07
iter 1800 loss 6.339330599672416e-07
iter 1900 loss 6.2911400495743e-07
iter 2000 loss 6.235167021394023e-07
iter 2100 loss 6.185136157612555e-07
iter 2200 loss 0.0008687317894024946
iter 2300 loss 0.002103182152917302
iter 2400 loss 0.0031242399807444306
epoch 5 loss 0.003069295540375099
iter 0 loss 5.477207650983473e-06
iter 100 loss 1.325977514665787e-06
iter 200 loss 9.87139263889364e-07
iter 300 loss 8.426533976371895e-07
iter 400 loss 7.69245567491743e-07
iter 500 loss 7.563012931025364e-07
iter 600 loss 7.513772827916967e-07
iter 700 loss 7.314274615427543e-07
iter 800 loss 7.072741644488621e-07
iter 900 loss 6.820235307814972e-07
iter 1000 loss 6.650655112022835e-07
iter 1100 loss 6.825118441435728e-07
iter 1200 loss 6.785141788314407e-07
iter 1300 loss 6.703982540658104e-07
iter 1400 loss 6.603159149147526e-07
iter 1500 loss 6.504136131946196e-07
iter 1600 loss 6.418350026828174e-07
iter 1700 loss 6.357000805696693e-07
iter 1800 loss 6.312387768679454e-07
iter 1900 loss 6.269928190297233e-07
iter 2000 loss 6.217256253401738e-07
iter 2100 loss 6.170837269594669e-07
iter 2200 loss 0.0008686876261567644
iter 2300 loss 0.0021030828016216935
iter 2400 loss 0.0031241003123121513
epoch 6 loss 0.003069158612605347
iter 0 loss 5.451138349599205e-06
iter 100 loss 1.315539388614714e-06
iter 200 loss 9.792114745898144e-07
iter 300 loss 8.360413409395294e-07
iter 400 loss 7.634904301751856e-07
iter 500 loss 7.507013975045195e-07
iter 600 loss 7.453687524838992e-07
iter 700 loss 7.257752036716239e-07
iter 800 loss 7.02070600438331e-07
iter 900 loss 6.771359846741135e-07
iter 1000 loss 6.605931956529729e-07
iter 1100 loss 6.777319252975348e-07
iter 1200 loss 6.740550999153594e-07
iter 1300 loss 6.66227154639926e-07
iter 1400 loss 6.565159933863929e-07
iter 1500 loss 6.469571292510808e-07
iter 1600 loss 6.38787833174912e-07
iter 1700 loss 6.329924727696728e-07
iter 1800 loss 6.289395484124686e-07
iter 1900 loss 6.251939756096036e-07
iter 2000 loss 6.202221658574377e-07
iter 2100 loss 6.158169469661728e-07
iter 2200 loss 0.0008686484489582626
iter 2300 loss 0.0021029952468982235
iter 2400 loss 0.003123976986160007
epoch 7 loss 0.0030690376908113506
iter 0 loss 5.423670700110961e-06
iter 100 loss 1.3077384634547488e-06
iter 200 loss 9.72967936277698e-07
iter 300 loss 8.306019418776855e-07
iter 400 loss 7.586991271755028e-07
iter 500 loss 7.456749655463506e-07
iter 600 loss 7.402318679569831e-07
iter 700 loss 7.211087240460213e-07
iter 800 loss 6.976971907035417e-07
iter 900 loss 6.729725022762006e-07
iter 1000 loss 6.568112210517937e-07
iter 1100 loss 6.736895952446476e-07
iter 1200 loss 6.703039881270719e-07
iter 1300 loss 6.626866990412346e-07
iter 1400 loss 6.533176565929513e-07
iter 1500 loss 6.440522237372267e-07
iter 1600 loss 6.362263773313125e-07
iter 1700 loss 6.306902839291682e-07
iter 1800 loss 6.268758501107041e-07
iter 1900 loss 6.234463135732394e-07
iter 2000 loss 6.188006080827282e-07
iter 2100 loss 6.146148721801719e-07
iter 2200 loss 0.000868613277597486
iter 2300 loss 0.0021029170495257404
iter 2400 loss 0.003123866831669164
epoch 8 loss 0.003068929675783123
iter 0 loss 5.408309334598016e-06
iter 100 loss 1.3023995522714115e-06
iter 200 loss 9.683924406601536e-07
iter 300 loss 8.26678650843446e-07
iter 400 loss 7.551433954088445e-07
iter 500 loss 7.419776540097996e-07
iter 600 loss 7.364578624702319e-07
iter 700 loss 7.175090697855091e-07
iter 800 loss 6.943527606079365e-07
iter 900 loss 6.698080371920994e-07
iter 1000 loss 6.539652123261807e-07
iter 1100 loss 6.705675718886917e-07
iter 1200 loss 6.67445408202355e-07
iter 1300 loss 6.599995206365982e-07
iter 1400 loss 6.509419378965541e-07
iter 1500 loss 6.419160620116672e-07
iter 1600 loss 6.344196794383478e-07
iter 1700 loss 6.291034091794639e-07
iter 1800 loss 6.255221366364857e-07
iter 1900 loss 6.223249915720632e-07
iter 2000 loss 6.178558701629827e-07
iter 2100 loss 6.138372228664622e-07
iter 2200 loss 0.0008685822437534025
iter 2300 loss 0.0021028471519806465
iter 2400 loss 0.003123767899385421
epoch 9 loss 0.0030688326415040917
--- Result:
Test failed! Accuracy = 0.13779863715171814<0.3
Test G: 0/20
--- Time taken: 221.2050838470459


--- TEST: batch_size= 1024  num_epochs= 2  lr= 0.001 

iter 0 loss 1.9846575014526024e-06
iter 100 loss 7.024700759829291e-07
iter 200 loss 5.922129688751829e-07
iter 300 loss 6.023674973070126e-07
iter 400 loss 5.739851564131933e-07
iter 500 loss 5.529845636461513e-07
iter 600 loss 5.361778115482097e-07
epoch 0 loss 5.34718451836419e-07
iter 0 loss 1.460455223423196e-06
iter 100 loss 5.422724601061472e-07
iter 200 loss 4.5787911331352017e-07
iter 300 loss 4.3102457264926655e-07
iter 400 loss 4.1521802774113077e-07
iter 500 loss 4.0657985585243435e-07
iter 600 loss 4.028658528876442e-07
epoch 1 loss 4.0278050529923645e-07
--- Result:
Test failed! Accuracy = 0.13119381666183472<0.3
Test G: 0/20
--- Time taken: 31.487875938415527


--- TEST: batch_size= 1024  num_epochs= 2  lr= 0.0005 

iter 0 loss 1.1440913567639655e-06
iter 100 loss 4.7666225916397914e-07
iter 200 loss 4.04650503769155e-07
iter 300 loss 3.8504925156345516e-07
iter 400 loss 3.7058964969622946e-07
iter 500 loss 3.639555371227109e-07
iter 600 loss 3.635980332465033e-07
epoch 0 loss 3.639734094652767e-07
iter 0 loss 1.056552946465672e-06
iter 100 loss 4.5521657952040995e-07
iter 200 loss 3.8582776162920666e-07
iter 300 loss 3.5970162034467393e-07
iter 400 loss 3.459914661983785e-07
iter 500 loss 3.4093482311876354e-07
iter 600 loss 3.421710447849223e-07
epoch 1 loss 3.4275535277374846e-07
--- Result:
Test failed! Accuracy = 0.13007010519504547<0.3
Test G: 0/20
--- Time taken: 32.07999396324158


--- TEST: batch_size= 1024  num_epochs= 2  lr= 0.0001 

iter 0 loss 9.918323939928086e-07
iter 100 loss 4.475067397483018e-07
iter 200 loss 3.821552158017798e-07
iter 300 loss 3.549178774485872e-07
iter 400 loss 3.400418916441057e-07
iter 500 loss 3.338358844204065e-07
iter 600 loss 3.3505272574662366e-07
epoch 0 loss 3.357333401097396e-07
iter 0 loss 9.760012744663982e-07
iter 100 loss 4.421205379359061e-07
iter 200 loss 3.7680533719863904e-07
iter 300 loss 3.4960610763094116e-07
iter 400 loss 3.351762675044354e-07
iter 500 loss 3.295603631021824e-07
iter 600 loss 3.312673979209842e-07
epoch 1 loss 3.320135853927272e-07
--- Result:
Test failed! Accuracy = 0.12909354269504547<0.3
Test G: 0/20
--- Time taken: 32.09383797645569


--- TEST: batch_size= 1024  num_epochs= 5  lr= 0.001 

iter 0 loss 9.642442364565795e-07
iter 100 loss 4.0696895407878885e-07
iter 200 loss 3.444610393361658e-07
iter 300 loss 3.2937359736649984e-07
iter 400 loss 3.1827556094561573e-07
iter 500 loss 3.1563083440911674e-07
iter 600 loss 3.1907526350042584e-07
epoch 0 loss 3.199247761635732e-07
iter 0 loss 8.977780225905008e-07
iter 100 loss 4.028806628330941e-07
iter 200 loss 3.387300844326037e-07
iter 300 loss 3.149228635181295e-07
iter 400 loss 3.028398305615244e-07
iter 500 loss 2.9958379589365294e-07
iter 600 loss 3.035063454514062e-07
epoch 1 loss 3.045300733809679e-07
iter 0 loss 8.578511483392504e-07
iter 100 loss 3.9324245087957726e-07
iter 200 loss 3.2974296527980875e-07
iter 300 loss 3.048786898540564e-07
iter 400 loss 2.9225941467867365e-07
iter 500 loss 2.888052899956829e-07
iter 600 loss 2.933629268220568e-07
epoch 2 loss 2.945842964885989e-07
iter 0 loss 8.325908993356279e-07
iter 100 loss 3.8628637425837483e-07
iter 200 loss 3.233528781293985e-07
iter 300 loss 2.981749643148128e-07
iter 400 loss 2.8527335194455036e-07
iter 500 loss 2.817720564179782e-07
iter 600 loss 2.869351114882594e-07
epoch 3 loss 2.883144598758032e-07
iter 0 loss 8.188549145415891e-07
iter 100 loss 3.82723617304232e-07
iter 200 loss 3.1965539500927646e-07
iter 300 loss 2.943626713868131e-07
iter 400 loss 2.8136576217934987e-07
iter 500 loss 2.7795197535035907e-07
iter 600 loss 2.8359200848399647e-07
epoch 4 loss 2.8513143616322207e-07
--- Result:
Test failed! Accuracy = 0.12759524583816528<0.3
Test G: 0/20
--- Time taken: 72.06104397773743


--- TEST: batch_size= 1024  num_epochs= 5  lr= 0.0005 

iter 0 loss 8.128018862407771e-07
iter 100 loss 3.871969596784225e-07
iter 200 loss 3.257593787688464e-07
iter 300 loss 2.9881586508101e-07
iter 400 loss 2.8408919756844386e-07
iter 500 loss 2.7913217310580657e-07
iter 600 loss 2.84092740174532e-07
epoch 0 loss 2.857573449094038e-07
iter 0 loss 7.895195039964165e-07
iter 100 loss 3.8274552143730486e-07
iter 200 loss 3.212632026276621e-07
iter 300 loss 2.939473296973848e-07
iter 400 loss 2.798759100983545e-07
iter 500 loss 2.758453725149622e-07
iter 600 loss 2.8152716081898803e-07
epoch 1 loss 2.83284436932449e-07
iter 0 loss 7.864928193157539e-07
iter 100 loss 3.8099353157059064e-07
iter 200 loss 3.196449733623148e-07
iter 300 loss 2.9254609471256745e-07
iter 400 loss 2.7868534322467153e-07
iter 500 loss 2.7482296723986087e-07
iter 600 loss 2.808670257346586e-07
epoch 2 loss 2.827259895324692e-07
iter 0 loss 7.833497761566832e-07
iter 100 loss 3.7995732416468195e-07
iter 200 loss 3.185966581940876e-07
iter 300 loss 2.9154399844613535e-07
iter 400 loss 2.7772150919130503e-07
iter 500 loss 2.7400876020495556e-07
iter 600 loss 2.803488733332887e-07
epoch 3 loss 2.8227881207828346e-07
iter 0 loss 7.803230914760206e-07
iter 100 loss 3.7905942833438727e-07
iter 200 loss 3.17645064860382e-07
iter 300 loss 2.9068693736032016e-07
iter 400 loss 2.7693911995339744e-07
iter 500 loss 2.7339322690397944e-07
iter 600 loss 2.80034497282122e-07
epoch 4 loss 2.820353145717102e-07
--- Result:
Test failed! Accuracy = 0.12162885069847107<0.3
Test G: 0/20
--- Time taken: 71.77121114730835


--- TEST: batch_size= 1024  num_epochs= 5  lr= 0.0001 

iter 0 loss 7.796247132318967e-07
iter 100 loss 3.8752084559980564e-07
iter 200 loss 3.28054669925544e-07
iter 300 loss 2.99771590856385e-07
iter 400 loss 2.8412406817083174e-07
iter 500 loss 2.783519182138391e-07
iter 600 loss 2.834944180284055e-07
epoch 0 loss 2.8546718804996244e-07
iter 0 loss 7.762487825857534e-07
iter 100 loss 3.8608467467284834e-07
iter 200 loss 3.258132421502632e-07
iter 300 loss 2.9724990551670113e-07
iter 400 loss 2.820129168129169e-07
iter 500 loss 2.767309354566352e-07
iter 600 loss 2.8246566453336845e-07
epoch 1 loss 2.8450519121752327e-07
iter 0 loss 7.753174600111379e-07
iter 100 loss 3.84739562119054e-07
iter 200 loss 3.242216551266731e-07
iter 300 loss 2.958440277370604e-07
iter 400 loss 2.808124760530974e-07
iter 500 loss 2.759590179114488e-07
iter 600 loss 2.8194131300722063e-07
epoch 2 loss 2.840046640517097e-07
iter 0 loss 7.749681572022382e-07
iter 100 loss 3.840502912036826e-07
iter 200 loss 3.2343396936704805e-07
iter 300 loss 2.94992765010454e-07
iter 400 loss 2.8002253596081745e-07
iter 500 loss 2.7541690863625867e-07
iter 600 loss 2.816697421076015e-07
epoch 3 loss 2.8375925849949274e-07
iter 0 loss 7.740368346276227e-07
iter 100 loss 3.8349587595290034e-07
iter 200 loss 3.229260270398218e-07
iter 300 loss 2.944950023937996e-07
iter 400 loss 2.795197148831638e-07
iter 500 loss 2.7505976251342543e-07
iter 600 loss 2.8153163194166516e-07
epoch 4 loss 2.8364989259890535e-07
--- Result:
Test failed! Accuracy = 0.12065228819847107<0.3
Test G: 0/20
--- Time taken: 73.27048230171204


--- TEST: batch_size= 1024  num_epochs= 10  lr= 0.001 

iter 0 loss 7.7333839954008e-07
iter 100 loss 3.581796330862663e-07
iter 200 loss 3.0190874672348513e-07
iter 300 loss 2.7987034189456997e-07
iter 400 loss 2.690234406205568e-07
iter 500 loss 2.6815103843135966e-07
iter 600 loss 2.775593507222356e-07
epoch 0 loss 2.8005242782220566e-07
iter 0 loss 7.733390248176875e-07
iter 100 loss 3.736882140491625e-07
iter 200 loss 3.1237395511600686e-07
iter 300 loss 2.864236653185406e-07
iter 400 loss 2.7361417516064316e-07
iter 500 loss 2.713098390740162e-07
iter 600 loss 2.7997658282353544e-07
epoch 1 loss 2.8246191652242e-07
iter 0 loss 7.859115385144833e-07
iter 100 loss 3.7805319738983583e-07
iter 200 loss 3.151441689366418e-07
iter 300 loss 2.881803353085218e-07
iter 400 loss 2.7466974979416364e-07
iter 500 loss 2.718570594348573e-07
iter 600 loss 2.8039188041611624e-07
epoch 2 loss 2.8292814787200116e-07
iter 0 loss 7.913830017969303e-07
iter 100 loss 3.81360075100992e-07
iter 200 loss 3.174330867017515e-07
iter 300 loss 2.895722903219372e-07
iter 400 loss 2.75636199369007e-07
iter 500 loss 2.7252069588337986e-07
iter 600 loss 2.810808802764246e-07
epoch 3 loss 2.8369122874236056e-07
iter 0 loss 7.972037678882771e-07
iter 100 loss 3.8372641147316696e-07
iter 200 loss 3.1908432721699607e-07
iter 300 loss 2.906134532666486e-07
iter 400 loss 2.7641743090067563e-07
iter 500 loss 2.731473872865205e-07
iter 600 loss 2.8181346404784796e-07
epoch 4 loss 2.8450518284451556e-07
iter 0 loss 7.996485464900616e-07
iter 100 loss 3.8530781227005846e-07
iter 200 loss 3.203388310699231e-07
iter 300 loss 2.9148328141266715e-07
iter 400 loss 2.770439258220085e-07
iter 500 loss 2.737236550882392e-07
iter 600 loss 2.825450791875519e-07
epoch 5 loss 2.853223753702025e-07
iter 0 loss 8.053527835727436e-07
iter 100 loss 3.874332479215197e-07
iter 200 loss 3.219987586594448e-07
iter 300 loss 2.924985306985628e-07
iter 400 loss 2.7784693001327364e-07
iter 500 loss 2.743854329498468e-07
iter 600 loss 2.833663777072239e-07
epoch 6 loss 2.8624054927590307e-07
iter 0 loss 8.10009396445821e-07
iter 100 loss 3.894192202146274e-07
iter 200 loss 3.234142742536251e-07
iter 300 loss 2.934801319853428e-07
iter 400 loss 2.785703886794814e-07
iter 500 loss 2.7507881243363844e-07
iter 600 loss 2.842029788583079e-07
epoch 7 loss 2.8716482019647697e-07
iter 0 loss 8.14083819022926e-07
iter 100 loss 3.912668738970445e-07
iter 200 loss 3.2469078471094055e-07
iter 300 loss 2.9440719871585e-07
iter 400 loss 2.792726537518433e-07
iter 500 loss 2.7572362715820777e-07
iter 600 loss 2.8498882966591893e-07
epoch 8 loss 2.8804088613022196e-07
iter 0 loss 8.18158241600031e-07
iter 100 loss 3.93395771368995e-07
iter 200 loss 3.260362190862386e-07
iter 300 loss 2.9532382433953716e-07
iter 400 loss 2.7999408012407263e-07
iter 500 loss 2.764307163726223e-07
iter 600 loss 2.85850418630633e-07
epoch 9 loss 2.889674438354873e-07
--- Result:
Test failed! Accuracy = 0.12651166319847107<0.3
Test G: 0/20
--- Time taken: 140.04972314834595


--- TEST: batch_size= 1024  num_epochs= 10  lr= 0.0005 

iter 0 loss 8.226983254644438e-07
iter 100 loss 4.0076334819264723e-07
iter 200 loss 3.3490346791199163e-07
iter 300 loss 3.0345510416913704e-07
iter 400 loss 2.8639721196760546e-07
iter 500 loss 2.810515449236428e-07
iter 600 loss 2.893022014044612e-07
epoch 0 loss 2.925917504746168e-07
iter 0 loss 8.060512755037053e-07
iter 100 loss 3.989133855980927e-07
iter 200 loss 3.3238113791869623e-07
iter 300 loss 3.004967655791662e-07
iter 400 loss 2.838712049707938e-07
iter 500 loss 2.792098145785652e-07
iter 600 loss 2.88028219938383e-07
epoch 1 loss 2.913993990597056e-07
iter 0 loss 8.065168231041753e-07
iter 100 loss 3.9761206836590305e-07
iter 200 loss 3.3131776143619477e-07
iter 300 loss 2.9940493673551455e-07
iter 400 loss 2.829204313508044e-07
iter 500 loss 2.785696460659983e-07
iter 600 loss 2.876940823814179e-07
epoch 2 loss 2.911254126876538e-07
iter 0 loss 8.076808626356069e-07
iter 100 loss 3.9686516413327117e-07
iter 200 loss 3.307177295181786e-07
iter 300 loss 2.9886308331328734e-07
iter 400 loss 2.8254592819580003e-07
iter 500 loss 2.7837654999326737e-07
iter 600 loss 2.877310792658056e-07
epoch 3 loss 2.9119838577549173e-07
iter 0 loss 8.07797221114015e-07
iter 100 loss 3.967176256326026e-07
iter 200 loss 3.305978379809057e-07
iter 300 loss 2.9876793932757407e-07
iter 400 loss 2.824530276922175e-07
iter 500 loss 2.783547070869857e-07
iter 600 loss 2.8786144042527673e-07
epoch 4 loss 2.9138091478467484e-07
iter 0 loss 8.094269787761732e-07
iter 100 loss 3.967925455400642e-07
iter 200 loss 3.3065980975079944e-07
iter 300 loss 2.9876755237685897e-07
iter 400 loss 2.824298026903568e-07
iter 500 loss 2.783956035441978e-07
iter 600 loss 2.880346105431967e-07
epoch 5 loss 2.915950725978165e-07
iter 0 loss 8.10358187663951e-07
iter 100 loss 3.9713833306189255e-07
iter 200 loss 3.3083124695685593e-07
iter 300 loss 2.9891877650273474e-07
iter 400 loss 2.825314121798131e-07
iter 500 loss 2.7847553744623236e-07
iter 600 loss 2.8822405166414916e-07
epoch 6 loss 2.9181894763525557e-07
iter 0 loss 8.100089416984702e-07
iter 100 loss 3.975486674114496e-07
iter 200 loss 3.3111214917487055e-07
iter 300 loss 2.9911177043807485e-07
iter 400 loss 2.826588592357697e-07
iter 500 loss 2.786137949368515e-07
iter 600 loss 2.8848167592218223e-07
epoch 7 loss 2.921074128680436e-07
iter 0 loss 8.109401505862479e-07
iter 100 loss 3.978495013566632e-07
iter 200 loss 3.3136409179199866e-07
iter 300 loss 2.993488548485027e-07
iter 400 loss 2.8288762486620424e-07
iter 500 loss 2.788245502515819e-07
iter 600 loss 2.88831308536409e-07
epoch 8 loss 2.9250028946018643e-07
iter 0 loss 8.12569965091825e-07
iter 100 loss 3.9818952602515065e-07
iter 200 loss 3.315668048975594e-07
iter 300 loss 2.9957433696948033e-07
iter 400 loss 2.830449741474951e-07
iter 500 loss 2.78982094084518e-07
iter 600 loss 2.890571657173977e-07
epoch 9 loss 2.9276036561501573e-07
--- Result:
Test failed! Accuracy = 0.12260541319847107<0.3
Test G: 0/20
--- Time taken: 140.25950407981873


--- TEST: batch_size= 1024  num_epochs= 10  lr= 0.0001 

iter 0 loss 8.136176461448485e-07
iter 100 loss 4.064296411676275e-07
iter 200 loss 3.4203316868206005e-07
iter 300 loss 3.091277384354789e-07
iter 400 loss 2.914344227513101e-07
iter 500 loss 2.850435898990018e-07
iter 600 loss 2.933515548752265e-07
epoch 0 loss 2.9700638144854586e-07
iter 0 loss 8.096596388895705e-07
iter 100 loss 4.048701388757475e-07
iter 200 loss 3.4007090586548016e-07
iter 300 loss 3.072554251838274e-07
iter 400 loss 2.896568398801649e-07
iter 500 loss 2.836577600546861e-07
iter 600 loss 2.9226042771618156e-07
epoch 1 loss 2.9593654174235936e-07
iter 0 loss 8.10707319942594e-07
iter 100 loss 4.0500729866862186e-07
iter 200 loss 3.391673825507184e-07
iter 300 loss 3.0640918908675817e-07
iter 400 loss 2.888143522288959e-07
iter 500 loss 2.831200645972215e-07
iter 600 loss 2.9201442443779394e-07
epoch 2 loss 2.9571990542590863e-07
iter 0 loss 8.109401505862479e-07
iter 100 loss 4.0447248229701483e-07
iter 200 loss 3.3853955222125774e-07
iter 300 loss 3.059385008394283e-07
iter 400 loss 2.88520266209851e-07
iter 500 loss 2.8309148404390607e-07
iter 600 loss 2.9216008900897357e-07
epoch 3 loss 2.9589119409092076e-07
iter 0 loss 8.096595820461516e-07
iter 100 loss 4.0399529614723455e-07
iter 200 loss 3.381793028121812e-07
iter 300 loss 3.0565423152849534e-07
iter 400 loss 2.8824679245083457e-07
iter 500 loss 2.8292162490273797e-07
iter 600 loss 2.9215253450914887e-07
epoch 4 loss 2.959110091857485e-07
iter 0 loss 8.09077505437017e-07
iter 100 loss 4.0384891336894853e-07
iter 200 loss 3.3780399442195577e-07
iter 300 loss 3.052566408018191e-07
iter 400 loss 2.878342585679145e-07
iter 500 loss 2.826688109608566e-07
iter 600 loss 2.920427050317818e-07
epoch 5 loss 2.958267935209376e-07
iter 0 loss 8.0942663771566e-07
iter 100 loss 4.035884182715086e-07
iter 200 loss 3.374981863023878e-07
iter 300 loss 3.04869878463163e-07
iter 400 loss 2.8746759353209124e-07
iter 500 loss 2.824076314206746e-07
iter 600 loss 2.919198973080947e-07
epoch 6 loss 2.957282878532304e-07
iter 0 loss 8.101251296466216e-07
iter 100 loss 4.03344061921173e-07
iter 200 loss 3.3728041463988276e-07
iter 300 loss 3.0458754239501697e-07
iter 400 loss 2.8719382914160473e-07
iter 500 loss 2.822080293576973e-07
iter 600 loss 2.918228524689306e-07
epoch 7 loss 2.956482645301106e-07
iter 0 loss 8.091938070720062e-07
iter 100 loss 4.031377419787442e-07
iter 200 loss 3.37137936327705e-07
iter 300 loss 3.044042172230674e-07
iter 400 loss 2.870338666932191e-07
iter 500 loss 2.82096493408004e-07
iter 600 loss 2.918015448647009e-07
epoch 8 loss 2.9563854707322964e-07
iter 0 loss 8.100087143247947e-07
iter 100 loss 4.0286111151103663e-07
iter 200 loss 3.369166892981633e-07
iter 300 loss 3.041996201956706e-07
iter 400 loss 2.8687593678688176e-07
iter 500 loss 2.8199750572811917e-07
iter 600 loss 2.9178604881433457e-07
epoch 9 loss 2.9562616262733315e-07
--- Result:
Test failed! Accuracy = 0.12358197569847107<0.3
Test G: 0/20
--- Time taken: 139.5122730731964

